In [ ]:
'''
Student Names + (IDs): 

Abdelrahman Mohamed Kamal Abdelaziz (1220255)
Mazen Ahmed Fouad Abdelwahab (1220269)
Mohamed Hesham Ibrahim Hassanain (1220278)
Ahmed Walaa Abdlelkhalek Abdelrahman (1220216)

'''
#Import(s)


import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import cv2
from skimage.feature import canny
from skimage.color import rgb2gray

In [ ]:
def show_images(images,titles=None):
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [ ]:
# Plate Localization

#-------------------------------------------------------------------------------------------------------------------------------

Path1 = 'Localization Test/One Step All Angles No Headlights/Straight.jpeg'
Path2 = 'Localization Test/Two Step All Angles No Headlights/Straight.jpeg'
Path3 = 'Localization Test/Three Step All Angles No Headlights/Straight.jpeg'
Path4 = 'Localization Test/One Step All Angles With Headlights/Straight.jpeg'
Path5 = 'Localization Test/Two Step All Angles With Headlights/Straight.jpeg'

Path11 = 'Localization Test/One Step All Angles No Headlights/Elevated.jpeg'
Path12 = 'Localization Test/Two Step All Angles No Headlights/Elevated.jpeg'
Path13 = 'Localization Test/Three Step All Angles No Headlights/Elevated.jpeg'
Path14 = 'Localization Test/One Step All Angles With Headlights/Elevated.jpeg'
Path15 = 'Localization Test/Two Step All Angles With Headlights/Elevated.jpeg'

Path111 = 'Localization Test/One Step All Angles No Headlights/Slight Right.jpeg'
Path112 = 'Localization Test/Two Step All Angles No Headlights/Slight Right.jpeg'
Path113 = 'Localization Test/Three Step All Angles No Headlights/Slight Right.jpeg'

Path1111 = 'Localization Test/One Step All Angles No Headlights/Slight Left.jpeg'
Path1112 = 'Localization Test/Two Step All Angles No Headlights/Slight Left.jpeg'
Path1113 = 'Localization Test/Three Step All Angles No Headlights/Slight Left.jpeg'
Path1114 = 'Localization Test/One Step All Angles With Headlights/Left.jpeg'

Path11111 = 'Localization Test/One Step All Angles No Headlights/Extreme Left.jpeg'
Path11112 = 'Localization Test/One Step All Angles No Headlights/Extreme Right.jpeg'
Path11113 = 'Localization Test/Two Step All Angles No Headlights/Extreme Left.jpeg'
Path11114 = 'Localization Test/Two Step All Angles No Headlights/Extreme Right.jpeg'
Path11115 = 'Localization Test/Three Step All Angles No Headlights/Extreme Left.jpeg'
Path11116 = 'Localization Test/Three Step All Angles No Headlights/Extreme Right.jpeg'
   
#-------------------------------------------------------------------------------------------------------------------------------

def Plate_Detection(image_path):

    # Original Image
    Original_Img = io.imread(image_path)[:,:,:3]
    Height, Width, _ = Original_Img.shape

    # Read Image and convert to grayscale
    img = cv2.imread(image_path)[:,:,:3]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE to improve local contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray_clahe = clahe.apply(gray)

    # Apply Gaussian Blur to reduce noise
    gray_blur = cv2.GaussianBlur(gray_clahe, (5,5), 0)

    # Adaptive Thresholding to handle dark and bright regions
    thresh = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 19, 9)

    # Morphological closing to connect fragmented regions
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    plate_candidates = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h
        area_ratio = (w * h) / (Height * Width)
        if (1.8 < aspect_ratio < 3.7) and (0.01 < area_ratio < 0.3):
            plate_candidates.append((x, y, w, h))

    if not plate_candidates:
        print("No plate candidate found.")
        return None

    # Choose the best candidate
    Best_Score = -1
    Best_Candidate = None

    for (x, y, w, h) in plate_candidates:

        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        Current_Image = Original_Img[y:y+h, x:x+w]

        # --- Edge Density ---
        CannyEdges = canny(rgb2gray(Current_Image), sigma=1.0)
        edge_ratio = np.count_nonzero(CannyEdges) / (h * w)

        # --- Color / brightness variance ---
        # use brightness (mean over channels) as a simple measure
        brightness = Current_Image.mean(axis=2)
        brightness_var = np.var(brightness)

        # show_images([Current_Image], [f"Candidate Plate Edge Ratio: {edge_ratio:.4f} , Brightness Var: {brightness_var:.4f}"])

        # --- Combined score ---
        # Edge density is still the main factor, color helps down-weight bad ones
        score = brightness_var

        if score > Best_Score:
            Best_Score = score
            Best_Candidate = (x, y, w, h)


    # Crop the best candidate
    x, y, w, h = Best_Candidate
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # show_images([img], ["Best Plate Candidate"])
    plate_img = Original_Img[y:y+h, x:x+w] 
    
    return plate_img

#-------------------------------------------------------------------------------------------------------------------------------
# Testing the Plate Detection Function on Different Images
#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img1 = io.imread(Path1)
Detected_Plate1 = Plate_Detection(Path1)
if Detected_Plate1 is not None:
    show_images([Origignal_Img1, Detected_Plate1], ["One Step No Headlights", "Plate Image"])

Origignal_Img2 = io.imread(Path2)
Detected_Plate2 = Plate_Detection(Path2)
if Detected_Plate2 is not None:
    show_images([Origignal_Img2, Detected_Plate2], ["Two Steps No Headlights", "Plate Image"])

Origignal_Img3 = io.imread(Path3)
Detected_Plate3 = Plate_Detection(Path3)
if Detected_Plate3 is not None:
    show_images([Origignal_Img3, Detected_Plate3], ["Three Steps No Headlights", "Plate Image"])

Origignal_Img4 = io.imread(Path4)
Detected_Plate4 = Plate_Detection(Path4)
if Detected_Plate4 is not None:
    show_images([Origignal_Img4, Detected_Plate4], ["One Step Headlights", "Plate Image"])

Origignal_Img5 = io.imread(Path5)
Detected_Plate5 = Plate_Detection(Path5)
if Detected_Plate5 is not None:
    show_images([Origignal_Img5, Detected_Plate5], ["Two Steps Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img11 = io.imread(Path11)
Detected_Plate11 = Plate_Detection(Path11)
if Detected_Plate11 is not None:
    show_images([Origignal_Img11, Detected_Plate11], ["One Step Elevated No Headlights", "Plate Image"]) 

Origignal_Img12 = io.imread(Path12)
Detected_Plate12 = Plate_Detection(Path12)
if Detected_Plate12 is not None:
    show_images([Origignal_Img12, Detected_Plate12], ["Two Step Elevated No Headlights", "Plate Image"])

Origignal_Img13 = io.imread(Path13)
Detected_Plate13 = Plate_Detection(Path13)
if Detected_Plate13 is not None:
    show_images([Origignal_Img13, Detected_Plate13], ["Three Step Elevated No Headlights", "Plate Image"])

Origignal_Img14 = io.imread(Path14)
Detected_Plate14 = Plate_Detection(Path14) 
if Detected_Plate14 is not None:
    show_images([Origignal_Img14, Detected_Plate14], ["One Step Elevated Headlights", "Plate Image"])

Origignal_Img15 = io.imread(Path15)
Detected_Plate15 = Plate_Detection(Path15)  
if Detected_Plate15 is not None:
    show_images([Origignal_Img15, Detected_Plate15], ["Two Step Elevated Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img111 = io.imread(Path111)
Detected_Plate111 = Plate_Detection(Path111)
if Detected_Plate111 is not None:
    show_images([Origignal_Img111, Detected_Plate111], ["One Step Slight Right No Headlights", "Plate Image"])

Origignal_Img112 = io.imread(Path112)
Detected_Plate112 = Plate_Detection(Path112)
if Detected_Plate112 is not None:
    show_images([Origignal_Img112, Detected_Plate112], ["Two Step Slight Right No Headlights", "Plate Image"])

Origignal_Img113 = io.imread(Path113)
Detected_Plate113 = Plate_Detection(Path113)
if Detected_Plate113 is not None:
    show_images([Origignal_Img113, Detected_Plate113], ["Three Step Slight Right No Headlights", "Plate Image"])

# Origignal_Img114 = io.imread(Path114)
# Detected_Plate114 = Plate_Detection(Path114)
# if Detected_Plate114 is not None:
#     show_images([Origignal_Img114, Detected_Plate114], ["One Step Slight Right Headlights", "Plate Image"])

# Origignal_Img115 = io.imread(Path115)
# Detected_Plate115 = Plate_Detection(Path115)
# if Detected_Plate115 is not None:
#     show_images([Origignal_Img115, Detected_Plate115], ["Two Step Slight Right Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img1111 = io.imread(Path1111)
Detected_Plate1111 = Plate_Detection(Path1111)
if Detected_Plate1111 is not None:
    show_images([Origignal_Img1111, Detected_Plate1111], ["One Step Slight Left No Headlights", "Plate Image"])

Origignal_Img1112 = io.imread(Path1112)
Detected_Plate1112 = Plate_Detection(Path1112)
if Detected_Plate1112 is not None:
    show_images([Origignal_Img1112, Detected_Plate1112], ["Two Step Slight Left No Headlights", "Plate Image"])

Origignal_Img1113 = io.imread(Path1113)
Detected_Plate1113 = Plate_Detection(Path1113)
if Detected_Plate1113 is not None:
    show_images([Origignal_Img1113, Detected_Plate1113], ["Three Step Slight Left No Headlights", "Plate Image"])

Origignal_Img1114 = io.imread(Path1114)
Detected_Plate1114 = Plate_Detection(Path1114)
if Detected_Plate1114 is not None:
    show_images([Origignal_Img1114, Detected_Plate1114], ["One Step Slight Left Headlights", "Plate Image"])

# Origignal_Img1115 = io.imread(Path1115)
# Detected_Plate1115 = Plate_Detection(Path1115)
# if Detected_Plate1115 is not None:
#     show_images([Origignal_Img1115, Detected_Plate1115], ["Two Step Slight Left Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Original_Img11111 = io.imread(Path11111)
Detected_Plate11111 = Plate_Detection(Path11111)
if Detected_Plate11111 is not None:
    show_images([Original_Img11111, Detected_Plate11111], ["One Step Extreme Left No Headlights", "Plate Image"])

Original_Img11112 = io.imread(Path11112)
Detected_Plate11112 = Plate_Detection(Path11112)
if Detected_Plate11112 is not None:
    show_images([Original_Img11112, Detected_Plate11112], ["One Step Extreme Right No Headlights", "Plate Image"])

Original_Img11113 = io.imread(Path11113)
Detected_Plate11113 = Plate_Detection(Path11113)
if Detected_Plate11113 is not None:
    show_images([Original_Img11113, Detected_Plate11113], ["Two Step Extreme Left No Headlights", "Plate Image"])

Original_Img11114 = io.imread(Path11114)
Detected_Plate11114 = Plate_Detection(Path11114)
if Detected_Plate11114 is not None:
    show_images([Original_Img11114, Detected_Plate11114], ["Two Step Extreme Right No Headlights", "Plate Image"])

Original_Img11115 = io.imread(Path11115)
Detected_Plate11115 = Plate_Detection(Path11115)
if Detected_Plate11115 is not None:
    show_images([Original_Img11115, Detected_Plate11115], ["Three Step Extreme Left No Headlights", "Plate Image"])

Original_Img11116 = io.imread(Path11116)
Detected_Plate11116 = Plate_Detection(Path11116)
if Detected_Plate11116 is not None:
    show_images([Original_Img11116, Detected_Plate11116], ["Three Step Extreme Right No Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Original_Img11117 = io.imread('trella.png')[:,:,:3]
Detected_Plate11117 = Plate_Detection('trella.png')
if Detected_Plate11117 is not None:
    show_images([Original_Img11117, Detected_Plate11117], ["Trella", "Plate Image"])

# import os

# BASE_PATH = "Localization Test/Vehicles"

# START_ID = 1
# END_ID   = 2087

# for i in range(START_ID, END_ID + 1):
#     fname = f"{i:04d}.jpg"        # 0001.jpg, 0002.jpg, ...
#     fpath = os.path.join(BASE_PATH, fname)

#     if not os.path.exists(fpath):
#         print(f"[SKIP] Missing: {fpath}")
#         continue

#     try:
#         img = io.imread(fpath)    # RGB (as you want)
#         title = f"Vehicle_{fname}"
#         print(f"[RUN] {title}")
#         Detected_Plate11116=Plate_Detection(fpath)
#         if Detected_Plate11116 is not None:
#             show_images([img, Detected_Plate11116], ["img", "detect"])

#     except Exception as e:
#         print(f"[ERROR] {fname}: {e}")

In [ ]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

def strip_corners_from_mask_edges(comp01, debug=False):
    """
    comp01: binary mask (H,W) of ONLY the strip component (0/1 or 0/255)
    Returns: (4,2) float32 corners in FULL image coords:
             [tl, tr, br, bl]  (already ordered)
    """

    comp = (comp01 > 0)
    H, W = comp.shape

    ys, xs = np.where(comp)
    if xs.size < 30:
        return None

    # x-span where the strip exists
    x_left  = int(np.percentile(xs, 2))
    x_right = int(np.percentile(xs, 98))
    if x_right <= x_left + 5:
        return None

    # For each x, find top and bottom y in that column
    top_pts = []
    bot_pts = []

    for x in range(x_left, x_right + 1):
        col = np.where(comp[:, x])[0]
        if col.size == 0:
            continue
        top_pts.append([x, float(col[0])])      # smallest y
        bot_pts.append([x, float(col[-1])])     # largest y

    top_pts = np.array(top_pts, dtype=np.float32)
    bot_pts = np.array(bot_pts, dtype=np.float32)

    if top_pts.shape[0] < 20 or bot_pts.shape[0] < 20:
        return None

    # robust trimming to ignore jagged ends / noise
    def _trim_points(pts, keep=0.80):
        n = pts.shape[0]
        k0 = int((1.0-keep)*0.5*n)
        k1 = int(n - k0)
        return pts[k0:k1]

    top_pts = _trim_points(top_pts, keep=0.80)
    bot_pts = _trim_points(bot_pts, keep=0.80)

    # Fit y = a*x + b using least squares (cv2.fitLine is OK too, but keep it simple)
    def _fit_line_y(ax_pts):
        x = ax_pts[:, 0]
        y = ax_pts[:, 1]
        A = np.stack([x, np.ones_like(x)], axis=1)
        a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a), float(b)

    a_top, b_top = _fit_line_y(top_pts)
    a_bot, b_bot = _fit_line_y(bot_pts)

    # Evaluate at x_left/x_right
    tl = np.array([x_left,  a_top*x_left  + b_top], dtype=np.float32)
    tr = np.array([x_right, a_top*x_right + b_top], dtype=np.float32)
    bl = np.array([x_left,  a_bot*x_left  + b_bot], dtype=np.float32)
    br = np.array([x_right, a_bot*x_right + b_bot], dtype=np.float32)

    quad = np.array([tl, tr, br, bl], dtype=np.float32)

    # sanity: clamp y into image
    quad[:, 1] = np.clip(quad[:, 1], 0, H-1)

    # final order (safe)
    quad = _order_quad_tl_tr_br_bl(quad)

    if debug:
        area = abs(cv2.contourArea(quad))
        print(f"[mask-edge corners] xL={x_left} xR={x_right} area={area:.1f} "
              f"top: y={tl[1]:.1f}->{tr[1]:.1f}, bot: y={bl[1]:.1f}->{br[1]:.1f}")

    return quad


def cv2_getPerspectiveTransform_like(src_pts, dst_pts):
    """
    Drop-in replacement for cv2.getPerspectiveTransform(src, dst)
    where src and dst are (4,2).

    Returns:
        H (3,3) float64
    """
    src = np.asarray(src_pts, dtype=np.float64).reshape(4, 2)
    dst = np.asarray(dst_pts, dtype=np.float64).reshape(4, 2)

    # Solve for homography with h33 = 1
    A = np.zeros((8, 8), dtype=np.float64)
    b = np.zeros((8,), dtype=np.float64)

    for i in range(4):
        x, y = src[i]
        u, v = dst[i]

        # u = (h11*x + h12*y + h13) / (h31*x + h32*y + 1)
        A[2*i, 0] = x
        A[2*i, 1] = y
        A[2*i, 2] = 1
        A[2*i, 6] = -u * x
        A[2*i, 7] = -u * y
        b[2*i] = u

        # v = (h21*x + h22*y + h23) / (h31*x + h32*y + 1)
        A[2*i+1, 3] = x
        A[2*i+1, 4] = y
        A[2*i+1, 5] = 1
        A[2*i+1, 6] = -v * x
        A[2*i+1, 7] = -v * y
        b[2*i+1] = v

    p = np.linalg.solve(A, b)

    H = np.array([
        [p[0], p[1], p[2]],
        [p[3], p[4], p[5]],
        [p[6], p[7], 1.0]
    ], dtype=np.float64)

    return H


def _replicate_index(i, n):
    if i < 0:
        return 0
    if i >= n:
        return n - 1
    return i


def cv2_warpPerspective_like(img, H, dsize, flags=None, borderMode=None, borderValue=0):
    """
    Drop-in replacement for cv2.warpPerspective(img, H, dsize, flags=..., borderMode=...)

    Implements:
      - inverse mapping (like OpenCV default)
      - INTER_LINEAR
      - BORDER_REPLICATE

    Args:
      img: (H,W) or (H,W,C)
      H: (3,3) homography mapping src->dst (like cv2)
      dsize: (outW, outH)
      flags: ignored except assumed INTER_LINEAR
      borderMode: ignored except assumed BORDER_REPLICATE
    """
    src = np.asarray(img)
    outW, outH = int(dsize[0]), int(dsize[1])

    Hm = np.asarray(H, dtype=np.float64)
    Hinv = np.linalg.inv(Hm)  # OpenCV does inverse mapping internally

    inH, inW = src.shape[:2]
    hasC = (src.ndim == 3)
    C = src.shape[2] if hasC else 1

    # work in float for interpolation, then cast back
    src_f = src.astype(np.float64)

    if not hasC:
        out = np.empty((outH, outW), dtype=np.float64)
    else:
        out = np.empty((outH, outW, C), dtype=np.float64)

    # For speed: precompute row terms that depend on v
    for v in range(outH):
        # These are coefficients for u in numerator/denom
        a0 = Hinv[0, 0]
        a1 = Hinv[0, 1] * v + Hinv[0, 2]
        b0 = Hinv[1, 0]
        b1 = Hinv[1, 1] * v + Hinv[1, 2]
        c0 = Hinv[2, 0]
        c1 = Hinv[2, 1] * v + Hinv[2, 2]

        for u in range(outW):
            denom = c0 * u + c1
            if denom == 0:
                # replicate border -> clamp to nearest valid (choose 0,0)
                if not hasC:
                    out[v, u] = src_f[0, 0]
                else:
                    out[v, u, :] = src_f[0, 0, :]
                continue

            x = (a0 * u + a1) / denom
            y = (b0 * u + b1) / denom

            # bilinear sampling with BORDER_REPLICATE
            x0 = int(np.floor(x))
            y0 = int(np.floor(y))
            x1 = x0 + 1
            y1 = y0 + 1

            ax = x - x0
            ay = y - y0

            x0c = _replicate_index(x0, inW)
            x1c = _replicate_index(x1, inW)
            y0c = _replicate_index(y0, inH)
            y1c = _replicate_index(y1, inH)

            if not hasC:
                p00 = src_f[y0c, x0c]
                p10 = src_f[y0c, x1c]
                p01 = src_f[y1c, x0c]
                p11 = src_f[y1c, x1c]

                top = (1 - ax) * p00 + ax * p10
                bot = (1 - ax) * p01 + ax * p11
                out[v, u] = (1 - ay) * top + ay * bot
            else:
                p00 = src_f[y0c, x0c, :]
                p10 = src_f[y0c, x1c, :]
                p01 = src_f[y1c, x0c, :]
                p11 = src_f[y1c, x1c, :]

                top = (1 - ax) * p00 + ax * p10
                bot = (1 - ax) * p01 + ax * p11
                out[v, u, :] = (1 - ay) * top + ay * bot

    # cast back to original dtype like cv2
    if np.issubdtype(src.dtype, np.integer):
        info = np.iinfo(src.dtype)
        out = np.clip(out, info.min, info.max).astype(src.dtype)
    else:
        out = out.astype(src.dtype)

    return out


# ================================================================
#  DEBUG VIEW
# ================================================================
def _show_mask_debug(mask, title):
    plt.figure(figsize=(4, 2))
    plt.imshow(mask, cmap="gray")
    plt.title(title)
    plt.axis("off")
    plt.show()

# ======================================================================
#  HELPER: draw candidate rectangles on debug image
# ======================================================================
def draw_candidates_debug(img, candidates, best_idx):
    debug_img = img.copy()

    for i, c in enumerate(candidates):
        color = (0,255,0) if i==best_idx else \
                (random.randint(50,255), random.randint(50,255), random.randint(50,255))

        x0, y0, x1, y1 = c["x0"], c["y0"], c["x1"], c["y1"]
        cv2.rectangle(debug_img, (x0,y0), (x1,y1), color, 2)
        cv2.putText(debug_img, f"{c['score']:.2f}", (x0, y0-4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return debug_img

def _order_quad_tl_tr_br_bl(pts4):
    pts = np.array(pts4, dtype=np.float32)
    s = pts.sum(axis=1)              # x+y
    d = pts[:,0] - pts[:,1]          # x-y
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmax(d)]
    bl = pts[np.argmin(d)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def deskew_stage2B_strip_perspective_if_needed(
    img_rgb,
    strip_corners_xy,     # 4x2 corners IN img_rgb coords
    debug=False,
    trap_ratio_thr=0.08,  # how trapezoid before we apply
    min_strip_area=80.0   # ignore tiny / unstable quads
):
    """
    Stage 2B: Projective correction using ONLY the 4 corners of the blue strip.
    Returns dict with 'rgb' and 'did_apply'.
    """

    if strip_corners_xy is None:
        return {"rgb": img_rgb, "did_apply": False, "reason": "no_corners"}

    quad = np.array(strip_corners_xy, dtype=np.float32)
    if quad.shape != (4, 2):
        return {"rgb": img_rgb, "did_apply": False, "reason": "bad_shape"}

    quad = _order_quad_tl_tr_br_bl(quad)

    # quick sanity: area of quad
    area = float(abs(cv2.contourArea(quad)))
    if area < min_strip_area:
        return {"rgb": img_rgb, "did_apply": False, "reason": "tiny_quad"}

    tl, tr, br, bl = quad

    def _dist(a, b):
        return float(np.hypot(a[0]-b[0], a[1]-b[1]))

    top_w    = _dist(tl, tr)
    bot_w    = _dist(bl, br)
    left_h   = _dist(tl, bl)
    right_h  = _dist(tr, br)

    # trapezoid-ness measures (1.0 is perfect rectangle)
    w_ratio = (top_w + 1e-6) / (bot_w + 1e-6)
    h_ratio = (left_h + 1e-6) / (right_h + 1e-6)

    need = (abs(w_ratio - 1.0) > trap_ratio_thr) or (abs(h_ratio - 1.0) > trap_ratio_thr)

    if debug:
        print(f"[2B] area={area:.1f} top/bot={w_ratio:.3f} left/right={h_ratio:.3f} -> apply={need}")

    if not need:
        return {"rgb": img_rgb, "did_apply": False, "reason": "already_rect"}

    # target rectangle size based on measured strip size
    dst_w = int(max(top_w, bot_w))
    dst_h = int(max(left_h, right_h))
    dst_w = max(dst_w, 10)
    dst_h = max(dst_h, 6)

    dst = np.array([
        [0,      0],
        [dst_w-1,0],
        [dst_w-1,dst_h-1],
        [0,      dst_h-1]
    ], dtype=np.float32)
    Himg, Wimg = img_rgb.shape[:2]
    Hm = cv2_getPerspectiveTransform_like(quad, dst)
    warped = cv2_warpPerspective_like(img_rgb, Hm, (Wimg, Himg))   #,flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE    HERE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    return {"rgb": warped, "did_apply": True, "reason": "warped"}



# ================================================================
#  TIERS
# ================================================================
BLUE_PARAMS_STRICT = dict(BLUE_SAT_THRESH=140, BLUE_VAL_THRESH=110, BLUE_HUE_MIN=80, BLUE_HUE_MAX=120, BLUE_DELTA=12)
BLUE_PARAMS_MID    = dict(BLUE_SAT_THRESH=115, BLUE_VAL_THRESH=95,  BLUE_HUE_MIN=80, BLUE_HUE_MAX=120, BLUE_DELTA=14)
BLUE_PARAMS_LENIENT= dict(BLUE_SAT_THRESH=60,  BLUE_VAL_THRESH=80,  BLUE_HUE_MIN=75, BLUE_HUE_MAX=125, BLUE_DELTA=18)
BLUE_PARAMS_VERY_LENIENT = dict(BLUE_SAT_THRESH=30, BLUE_VAL_THRESH=60, BLUE_HUE_MIN=70, BLUE_HUE_MAX=135, BLUE_DELTA=22)

BLUE_PARAMS_ULTRA_LENIENT = dict(
    BLUE_SAT_THRESH=18,
    BLUE_VAL_THRESH=45,
    BLUE_HUE_MIN=65,
    BLUE_HUE_MAX=145,
    BLUE_DELTA=26
)

BLUE_PARAMS_MAX_LENIENT = dict(
    BLUE_SAT_THRESH=8,
    BLUE_VAL_THRESH=35,
    BLUE_HUE_MIN=55,
    BLUE_HUE_MAX=160,
    BLUE_DELTA=32
)

BLUE_PARAMS_NUCLEAR = dict(
    BLUE_SAT_THRESH = 3,
    BLUE_VAL_THRESH = 25,
    BLUE_HUE_MIN    = 45,
    BLUE_HUE_MAX    = 170,
    BLUE_DELTA      = 40
)

BLUE_PARAM_TIERS = [
    ("STRICT",       BLUE_PARAMS_STRICT),
    ("MID",          BLUE_PARAMS_MID),
    ("LENIENT",      BLUE_PARAMS_LENIENT),
    ("VERY_LENIENT", BLUE_PARAMS_VERY_LENIENT),
    ("ULTRA_LENIENT", BLUE_PARAMS_ULTRA_LENIENT),
    ("MAX_LENIENT",   BLUE_PARAMS_MAX_LENIENT),
    ("NUCLEAR",       BLUE_PARAMS_NUCLEAR),
]

TIER_BONUS = {
    "STRICT": 0.18,
    "MID": 0.12,
    "LENIENT": 0.06,
    "VERY_LENIENT": 0.00,
    "ULTRA_LENIENT": -0.06,
    "MAX_LENIENT":   -0.10,
    "NUCLEAR":       -0.10,
}

# ================================================================
#  SCORING HYPERPARAMS
# ================================================================
HUE_CENTER = 100
HUE_SIGMA  = 18.0

WHITE_S_MAX = 90
WHITE_V_MIN = 120

W_CC_DOM   = 0.22
W_ROW_BAND = 0.18
W_BLUE     = 0.26
W_SAT      = 0.10
W_BELOW    = 0.18
W_TOP_PRIOR= 0.06


# ================================================================
def _adaptive_blue_mask_with_params(img_rgb, params, debug=False, tag=""):
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    H, S, V = cv2.split(hsv)

    sat_thr = params["BLUE_SAT_THRESH"]
    val_thr = params["BLUE_VAL_THRESH"]
    hue_min = params["BLUE_HUE_MIN"]
    hue_max = params["BLUE_HUE_MAX"]
    delta   = params["BLUE_DELTA"]

    colored = (S > sat_thr) & (V > val_thr)
    H_colored = H[colored]

    hue_mask = (H_colored >= hue_min) & (H_colored <= hue_max)
    blue_hues = H_colored[hue_mask]

    if blue_hues.size == 0:
        if debug:
            print(f"[{tag}] No blue hues found in candidate window.")
        return np.zeros_like(H, dtype=np.uint8), None, None

    hist, bin_edges = np.histogram(blue_hues, bins=36, range=(0, 180))
    peak_bin = int(np.argmax(hist))
    h_peak = 0.5 * (bin_edges[peak_bin] + bin_edges[peak_bin + 1])

    h_low  = max(0,   int(h_peak - delta))
    h_high = min(179, int(h_peak + delta))

    lower_blue = np.array([h_low,  sat_thr, val_thr], dtype=np.uint8)
    upper_blue = np.array([h_high, 255,     255],     dtype=np.uint8)

    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    if debug:
        n = int(mask.sum() // 255)
        print(f"[{tag}] Peak={h_peak:.1f}, H=({h_low},{h_high}), S>{sat_thr}, V>{val_thr}, pixels={n}")

    return mask, lower_blue, upper_blue


def _hue_distance_circular(h, center):
    d = np.abs(h.astype(np.float32) - float(center))
    return np.minimum(d, 180.0 - d)


def _score_component(mask, hsv, comp_mask, bbox, tier_name, debug=False):
    Himg, Wimg = mask.shape[:2]
    x, y, w, h = bbox
    area = float(comp_mask.sum())
    total = float((mask > 0).sum()) + 1e-6

    cc_dom = area / total

    y_center = y + 0.5*h
    top_prior = 1.0 - np.clip(y_center / (0.65 * Himg + 1e-6), 0.0, 1.0)

    sub = mask[y:y+h, x:x+w] > 0
    if sub.size == 0:
        return -1e9, {}
    row_counts = sub.sum(axis=1).astype(np.float32)
    if row_counts.max() <= 0:
        row_band = 0.0
    else:
        peak = row_counts.max()
        mean = row_counts.mean() + 1e-6
        peakiness = float(peak / mean)
        thr = 0.6 * peak
        strong = row_counts >= thr
        longest = 0
        cur = 0
        for v in strong:
            if v:
                cur += 1
                longest = max(longest, cur)
            else:
                cur = 0
        run_frac = float(longest / (h + 1e-6))
        row_band = np.tanh((peakiness - 1.5) / 2.0) * 0.6 + run_frac * 0.4
        row_band = float(np.clip(row_band, 0.0, 1.0))

    Hc = hsv[...,0][comp_mask]
    Sc = hsv[...,1][comp_mask]
    if Hc.size == 0:
        blue_score = 0.0
        sat_score  = 0.0
    else:
        d = _hue_distance_circular(Hc, HUE_CENTER)
        mean_d = float(d.mean())
        blue_score = float(np.exp(- (mean_d**2) / (2.0 * (HUE_SIGMA**2))))
        sat_score = float(np.clip(Sc.mean() / 255.0, 0.0, 1.0))

    y0 = int(y + h)
    y1 = int(min(Himg, y + h + 1.5*h))
    if y1 <= y0 + 2:
        below_white = 0.0
    else:
        below = hsv[y0:y1, x:x+w]
        S_b = below[...,1].astype(np.uint8)
        V_b = below[...,2].astype(np.uint8)
        white = (S_b < WHITE_S_MAX) & (V_b > WHITE_V_MIN)
        below_white = float(white.mean())

    tier_bonus = TIER_BONUS.get(tier_name, 0.0)

    score = (
        W_CC_DOM    * cc_dom +
        W_ROW_BAND  * row_band +
        W_BLUE      * blue_score +
        W_SAT       * sat_score +
        W_BELOW     * below_white +
        W_TOP_PRIOR * top_prior +
        tier_bonus
    )

    if debug:
        print(f"    [{tier_name}] bbox={x,y,w,h} "
              f"score={score:.3f} | cc={cc_dom:.2f} row={row_band:.2f} blue={blue_score:.2f} "
              f"sat={sat_score:.2f} belowW={below_white:.2f} top={top_prior:.2f} +bonus={tier_bonus:.2f}")

    return float(score), dict(bbox=(x,y,w,h))


def adaptive_blue_mask(img_rgb, debug=False):
    # NOTE: keep everything minimal
    Himg, Wimg = img_rgb.shape[:2]

    # ------------------------------------------------------------
    # NEW: probe STRICT + MID, and if both are "near empty", rescue
    # ------------------------------------------------------------
    RESCUE_WHITE_THR = 200  # you can tune this
    did_rescue = False

    # quick probe on ORIGINAL img
    maskS, _, _ = _adaptive_blue_mask_with_params(img_rgb, BLUE_PARAMS_STRICT, debug=False, tag="PROBE_STRICT")
    maskM, _, _ = _adaptive_blue_mask_with_params(img_rgb, BLUE_PARAMS_MID,    debug=False, tag="PROBE_MID")

    nS = int(maskS.sum() // 255) if maskS is not None else 0
    nM = int(maskM.sum() // 255) if maskM is not None else 0

    img_work = img_rgb

    if (nS < RESCUE_WHITE_THR) and (nM < RESCUE_WHITE_THR):
        did_rescue = True
        if debug:
            print(f"[RESCUE TRIGGER] STRICT={nS} MID={nM} < {RESCUE_WHITE_THR} -> calling rescue_blue_strip_rgb_extreme()")

        img_rescued = rescue_blue_strip_rgb_extreme(img_rgb, debug=True)

        if debug:
            plt.figure(figsize=(10,4))
            plt.subplot(1,2,1); plt.imshow(img_rgb);     plt.title("Original (before rescue)"); plt.axis("off")
            plt.subplot(1,2,2); plt.imshow(img_rescued); plt.title("After rescue_blue_strip_rgb_extreme"); plt.axis("off")
            plt.tight_layout()
            plt.show()

        img_work = img_rescued

    # now proceed normally, BUT using img_work (original or rescued)
    hsv = cv2.cvtColor(img_work, cv2.COLOR_RGB2HSV)

    best = dict(
        score=-1e9,
        mask=None, lb=None, ub=None, tier=None, bbox=None,
        did_rescue=did_rescue,
        best_comp_id=None,
        best_labels=None
    )

    for tier_name, params in BLUE_PARAM_TIERS:
        mask, lb, ub = _adaptive_blue_mask_with_params(img_work, params, debug=debug, tag=tier_name)

        # keep your dilation exactly as-is
        mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3)), iterations=1)

        if debug:
            _show_mask_debug(mask, f"{tier_name} mask (raw+dilated)")

        if lb is None or mask.sum() == 0:
            continue

        bin_mask = (mask > 0).astype(np.uint8)
        num, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_mask, connectivity=8)

        for comp_id in range(1, num):
            x, y, w, h, area = stats[comp_id]
            if area < 20:
                continue

            comp_mask = (labels == comp_id)
            score, details = _score_component(mask, hsv, comp_mask, (x,y,w,h), tier_name, debug=debug)

            if score > best["score"]:
                best.update(
                    score=score,
                    mask=mask, lb=lb, ub=ub, tier=tier_name, bbox=(x,y,w,h),
                    best_comp_id=comp_id,
                    best_labels=labels
                )

    if best["mask"] is None:
        return np.zeros((Himg, Wimg), dtype=np.uint8), None, None, None, None, None, img_work

    # -----------------------------
    # NEW: compute 4 rotated corners of BEST strip component
    # -----------------------------
# -----------------------------
# NEW: compute 4 corners of BEST strip component
# Try TRUE quad from contour; fallback to minAreaRect rectangle
# -----------------------------



    # strip_corners = None
    # strip_corners_src = "none"

    # if best["best_labels"] is not None and best["best_comp_id"] is not None:
    #     comp = (best["best_labels"] == best["best_comp_id"]).astype(np.uint8) * 255

    #     contours, _ = cv2.findContours(comp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #     if contours:
    #         cnt = max(contours, key=cv2.contourArea)
    #         if cv2.contourArea(cnt) >= 30:

    #             # try to approximate to 4-point polygon (real trapezoid if perspective exists)
    #             peri = cv2.arcLength(cnt, True)
    #             approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)

    #             if len(approx) == 4:
    #                 strip_corners = approx.reshape(4, 2).astype(np.float32)
    #                 strip_corners_src = "approx4"
    #             else:
    #                 # fallback: minAreaRect gives rectangle (always w_ratio≈1, h_ratio≈1)
    #                 rect = cv2.minAreaRect(cnt)
    #                 strip_corners = cv2.boxPoints(rect).astype(np.float32)
    #                 strip_corners_src = f"minAreaRect(len={len(approx)})"



    strip_corners = None
    strip_corners_src = "none"

    if best["best_labels"] is not None and best["best_comp_id"] is not None:
        comp = (best["best_labels"] == best["best_comp_id"]).astype(np.uint8) * 255

        # NEW PRIMARY METHOD: mask edge lines -> trapezoid corners
        strip_corners = strip_corners_from_mask_edges(comp, debug=debug)
        if strip_corners is not None:
            strip_corners_src = "mask_edges"

        else:
            # fallback 1: approxPolyDP if it returns 4 points
            contours, _ = cv2.findContours(comp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                cnt = max(contours, key=cv2.contourArea)
                peri = cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)

                if len(approx) == 4:
                    strip_corners = approx.reshape(4, 2).astype(np.float32)
                    strip_corners_src = "approx4"
                else:
                    # last fallback: minAreaRect (rectangle)
                    rect = cv2.minAreaRect(cnt)
                    strip_corners = cv2.boxPoints(rect).astype(np.float32)
                    strip_corners_src = f"minAreaRect(len={len(approx)})"



    if debug:
        x,y,w,h = best["bbox"]
        print(f"\n✅ CHOSEN OVERALL: tier={best['tier']} score={best['score']:.3f} bbox={(x,y,w,h)} rescue_used={did_rescue}\n")
        if strip_corners is not None:
            print(f"[STRIP CORNERS] src={strip_corners_src}\n", strip_corners)


    return best["mask"], best["lb"], best["ub"], best["tier"], best["bbox"], strip_corners, img_work



#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

def crop_left_right_by_blue_adaptive(img_bgr, debug=False, enable_perspective=True):

    H, W = img_bgr.shape[:2]


    mask, lb, ub, tier, bbox, strip_corners, img_work = adaptive_blue_mask(img_bgr, False)   #HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    if debug:
        print("✅ Tier used:", tier)

    if lb is None:
        if debug:
            print("❌ No blue detected")
        return None, None, mask, None

    # IMPORTANT: continue on img_work (original or rescued)
    hsv = cv2.cvtColor(img_work, cv2.COLOR_RGB2HSV)

    row_counts = mask.sum(axis=1)//255
    max_row = row_counts.max()
    if max_row == 0:
        return None, None, mask, None

    row_thresh = 0.30*max_row
    blue_rows = row_counts > row_thresh

    bands = []
    cur_start = None
    for y, flag in enumerate(blue_rows):
        if flag and cur_start is None:
            cur_start = y
        elif not flag and cur_start is not None:
            bands.append((cur_start, y))
            cur_start = None
    if cur_start is not None:
        bands.append((cur_start, len(blue_rows)))

    # =====================================================
    # ✅ NEW: two-pass gating thresholds (only used for filtering)
    # =====================================================
    PASS1_MIN_BAND_H_FRAC = 0.02
    PASS1_MIN_BAND_W_FRAC = 0.10

    PASS2_MIN_BAND_H_FRAC = 0.008
    PASS2_MIN_BAND_W_FRAC = 0.05

    def _collect_candidates(min_band_h_frac, min_band_w_frac):
        candidates = []

        for (y0_band, y1_band) in bands:
            band_h = y1_band - y0_band
            if band_h < min_band_h_frac * H:
                continue

            submask = mask[y0_band:y1_band, :]

            ys2, xs2 = np.where(submask > 0)
            if xs2.size == 0:
                continue

            x0_band = int(np.percentile(xs2, 1))
            x1_band = int(np.percentile(xs2, 99))
            band_w  = x1_band - x0_band

            if band_w < min_band_w_frac * W or band_w > 0.95 * W:
                continue

            ar_band = band_w / (band_h + 1e-6)
            s_ar_band = np.tanh((ar_band - 2.0) / 2.0)
            s_ar_band = float(np.clip(s_ar_band, 0.0, 1.0))

            region_hsv = hsv[:, x0_band:x1_band]
            _, Sc, Vc = cv2.split(region_hsv)

            S_MAX = 130
            V_MIN = 80
            white_mask = (Sc < S_MAX) & (Vc > V_MIN)
            white_ratio = float(white_mask.mean())

            s_white = white_ratio

            k = 3.0
            y0_pl = max(0,   int(y0_band - 0.2*band_h))
            y1_pl = min(H-1, int(y0_band + k*band_h))
            plate_h = y1_pl - y0_pl
            ar_plate = band_w / (plate_h + 1e-6)

            expected_ar = 2.0
            sigma = 0.8
            s_ar_plate = float(np.exp(-((ar_plate - expected_ar)**2) / (2 * sigma**2)))

            score = (
                0.20*s_ar_band +
                0.50*s_white   +
                0.30*s_ar_plate
            )

            candidates.append({
                "x0": x0_band, "y0": y0_band,
                "x1": x1_band, "y1": y1_band,
                "width": band_w,
                "height": band_h,
                "score": float(score),
                "white_ratio": white_ratio,
                "s_white": s_white,
                "s_ar_band": s_ar_band,
                "s_ar_plate": s_ar_plate
            })

        return candidates

    candidates = _collect_candidates(PASS1_MIN_BAND_H_FRAC, PASS1_MIN_BAND_W_FRAC)

    if not candidates:
        candidates = _collect_candidates(PASS2_MIN_BAND_H_FRAC, PASS2_MIN_BAND_W_FRAC)
        if debug and candidates:
            print("[2-PASS] Pass1 empty -> using lenient gates for far/small plate case")

    if not candidates:
        return img_work, (0,0,W-1,H-1), mask, img_work

    candidates_sorted = sorted(
        candidates,
        key=lambda c: (c["score"], c["white_ratio"], c["width"], c["height"]),
        reverse=True
    )

    best = candidates_sorted[0]
    best_idx = candidates.index(best)

    margin = int(0.02*W)
    x0 = max(0, best["x0"] - margin)
    x1 = min(W-1, best["x1"] + margin)

    y_strip_top = int(best["y0"])
    strip_h     = int(best["height"])

    plate_h = int(3.5 * strip_h)

    y_top = max(0, y_strip_top)
    y_bottom = y_top + plate_h + int(0.10 * strip_h)
    y_bottom = min(H - 1, y_bottom)

        # =====================================================
    # ✅ NEW: tiny safety padding (top/left/right) so we don't cut plate edges                             MAW OK OR NOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
    # =====================================================
    # PAD_X_FRAC = 0.010   # ~1% of image width
    # PAD_TOP_FRAC = 0.010 # ~1% of image height
    # PAD_X_PX_MIN = 2
    # PAD_TOP_PX_MIN = 2

    # pad_x  = max(PAD_X_PX_MIN,  int(PAD_X_FRAC  * W))
    # # pad_ty = max(PAD_TOP_PX_MIN, int(PAD_TOP_FRAC * H))
    # pad_ty = max(2, int(0.15 * strip_h))  # e.g., 15% of strip height


    # # expand crop slightly
    # x0 = max(0, x0 - pad_x)     # left
    # x1 = min(W - 1, x1 + pad_x) # right
    # y_top = max(0, y_top - pad_ty)  # top only (keep bottom as-is)

    # # (optional) keep bottom inside bounds (already clamped, but safe)
    # y_bottom = min(H - 1, y_bottom)


    cropped = img_work[y_top:y_bottom+1, x0:x1]


    # ===================== DEBUG: Stage 2B inspection =====================
    if debug:
        print("\n================= [DEBUG 2B] =================")
        print("[DEBUG 2B] img_work used instead of original:", (img_work is not img_bgr))
        print("[DEBUG 2B] crop x0,x1,y_top,y_bottom:", x0, x1, y_top, y_bottom)
        print("[DEBUG 2B] cropped shape:", cropped.shape)

        if strip_corners is None:
            print("[DEBUG 2B] strip_corners = None  (no corners computed!)")
        else:
            print("[DEBUG 2B] strip_corners (FULL coords):\n", strip_corners)

            # convert to cropped coords
            strip_corners_crop = strip_corners.copy().astype(np.float32)
            strip_corners_crop[:, 0] -= float(x0)
            strip_corners_crop[:, 1] -= float(y_top)

            print("[DEBUG 2B] strip_corners (CROPPED coords):\n", strip_corners_crop)

            # order and compute trapezoid measures like Stage 2B
            quad = _order_quad_tl_tr_br_bl(strip_corners_crop)
            tl, tr, br, bl = quad

            def _dist(a, b):
                return float(np.hypot(a[0]-b[0], a[1]-b[1]))

            top_w  = _dist(tl, tr)
            bot_w  = _dist(bl, br)
            left_h = _dist(tl, bl)
            right_h= _dist(tr, br)

            w_ratio = (top_w + 1e-6) / (bot_w + 1e-6)
            h_ratio = (left_h + 1e-6) / (right_h + 1e-6)


                        # =====================================================
            # ✅ NEW: cap extreme ratios (if too crazy -> skip perspective)
            # =====================================================
            MAX_RATIO_CAP = 2.0  # your requested cap

            w_ratio_sym = max(w_ratio, 1.0 / (w_ratio + 1e-6))   # symmetric ratio >= 1
            h_ratio_sym = max(h_ratio, 1.0 / (h_ratio + 1e-6))   # symmetric ratio >= 1

            if (w_ratio_sym > MAX_RATIO_CAP) or (h_ratio_sym > MAX_RATIO_CAP):
                if debug:
                    print(f"[2B] SKIP: extreme ratio (w_sym={w_ratio_sym:.3f}, h_sym={h_ratio_sym:.3f}) > {MAX_RATIO_CAP}")
                    enable_perspective=False


            area = float(abs(cv2.contourArea(quad)))
            print(f"[DEBUG 2B] quad area={area:.1f} top_w={top_w:.1f} bot_w={bot_w:.1f} w_ratio={w_ratio:.3f}")
            print(f"[DEBUG 2B] left_h={left_h:.1f} right_h={right_h:.1f} h_ratio={h_ratio:.3f}")

            # draw overlay on cropped image
            overlay = cropped.copy()
            qint = np.round(quad).astype(np.int32)
            cv2.polylines(overlay, [qint], True, (255, 0, 0), 2)  # blue polygon

            # label corners
            pts = {"TL": tl, "TR": tr, "BR": br, "BL": bl}
            for name, p in pts.items():
                px, py = int(round(p[0])), int(round(p[1]))
                cv2.circle(overlay, (px, py), 4, (0, 255, 0), -1)
                cv2.putText(overlay, name, (px + 5, py - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            plt.figure(figsize=(10,4))
            plt.subplot(1,2,1); plt.imshow(cropped);  plt.title("Cropped (before 2B)"); plt.axis("off")
            plt.subplot(1,2,2); plt.imshow(overlay);  plt.title("Cropped + strip corners"); plt.axis("off")
            plt.tight_layout()
            plt.show()

        print("================================================\n")
    # =================== END DEBUG: Stage 2B inspection ===================


    # =====================================================
    # ✅ NEW: Stage 2B plug-in (using strip corners only)
    # Convert strip_corners from FULL img_work coords -> CROPPED coords
    # =====================================================
    if enable_perspective and (strip_corners is not None):
        strip_corners_crop = strip_corners.copy()
        strip_corners_crop[:, 0] -= float(x0)
        strip_corners_crop[:, 1] -= float(y_top)

        out2b = deskew_stage2B_strip_perspective_if_needed(
            cropped, strip_corners_crop, debug=debug
        )
        if out2b["did_apply"]:
            cropped = out2b["rgb"]
            if debug:
                print("[2B] applied:", out2b.get("reason", ""))



    debug_img = draw_candidates_debug(img_work, candidates, best_idx)

    # if debug:
    #     print("\n========= CANDIDATE BREAKDOWN =========")
    #     for i, c in enumerate(candidates):
    #         print(f"[{i}] score={c['score']:.3f}, "
    #               f"white_ratio={c['white_ratio']:.3f}, "                                     HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #               f"s_white={c['s_white']:.2f}, "
    #               f"ARband={c['s_ar_band']:.2f}, "
    #               f"ARplate={c['s_ar_plate']:.2f}, "
    #               f"range=({c['x0']},{c['x1']}) "
    #               f"vertical=({c['y0']},{c['y1']})")
    #     print("Chosen:", best)
    #     print("========================================\n")

    return cropped, (best["x0"], best["y0"], best["x1"], best["y1"]), mask, debug_img




def debug_blue_crop(plate_img, title, enable_perspective=True):

    if plate_img is None:
        print(f"{title}: plate_img is None, skipping.")
        return None

    cropped, bbox, mask, debug_img = crop_left_right_by_blue_adaptive(
    plate_img, debug=True, enable_perspective=enable_perspective
)



    images = [plate_img]
    titles = [f"{title} - Original"]

    if mask is not None:
        images.append(mask)
        titles.append("Blue mask")

    if debug_img is not None:
        images.append(debug_img)
        titles.append("All candidates")
    else:
        print(f"{title}: debug_img is None (likely no candidates / no blue).")

    if cropped is not None:
        images.append(cropped)
        titles.append("Final crop")
    else:
        print(f"{title}: cropped is None (no blue detected).")

    show_images(images, titles)
    plt.close('all')

    return cropped


def rescue_blue_strip_rgb_extreme(img_rgb, debug=False):

    """
    MUCH harsher rescue:
      - normalize + CLAHE + stronger gamma on V
      - build a better seed (prefers blue-ish + horizontal band)
      - inside seed: PUSH S/V strongly toward target (S~100, V~90+)
      - optional stronger hue pull toward blue center
    """

    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV).astype(np.float32)
    H = hsv[..., 0]
    S = hsv[..., 1]
    V = hsv[..., 2]

    # -----------------------------
    # (1) Robust normalize V (stronger)
    # -----------------------------
    v1 = np.percentile(V, 1)
    v2 = np.percentile(V, 99)
    if v2 > v1 + 1e-3:
        Vn = (V - v1) * (255.0 / (v2 - v1))
        Vn = np.clip(Vn, 0, 255)
    else:
        Vn = V.copy()
    
    V8 = Vn.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(5, 5))
    Vc = clahe.apply(V8).astype(np.float32)

    gamma = 0.3   # was 0.65, now harsher
    Vg = 255.0 * np.power(np.clip(Vc / 255.0, 0, 1), gamma)

    # slight extra lift to midtones
    Vg = np.clip(Vg + 10.0, 0, 255)

    hsv2 = hsv.copy()
    hsv2[..., 2] = Vg
    # -----------------------------
    hsv2_u8 = hsv2.astype(np.uint8)



    lower_seed = np.array([70,  0,  0], dtype=np.uint8)
    upper_seed = np.array([135, 255, 255], dtype=np.uint8)
    seed0 = cv2.inRange(hsv2_u8, lower_seed, upper_seed)


    S_min_after = 15
    seed0 = seed0 & ((hsv2_u8[...,1] >= S_min_after).astype(np.uint8) * 255)

    # cleanup
    seed0 = cv2.medianBlur(seed0, 5)

    # horizontal emphasis: open with long horizontal kernel
    seed1 = cv2.morphologyEx(
        seed0, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3)),
        iterations=1
    )

    # connect horizontally
    seed1 = cv2.morphologyEx(
        seed1, cv2.MORPH_CLOSE,
        cv2.getStructuringElement(cv2.MORPH_RECT, (25, 5)),
        iterations=1
    )

    seed01 = (seed1 > 0)

    # -----------------------------
    # (5) EXTREME push of S/V inside seed
    # target-ish: S ~ 95-100, V ~ 80-100
    # -----------------------------
    S2 = S.copy()
    V2 = Vg.copy()

    # Push S upward HARD:
    #   - If S is tiny, multiplication does nothing, so we also ADD a lot
    S2[seed01] = np.clip(S2[seed01] * 2.8 + 80, 0, 255)

    # Push V upward HARD (but still capped):
    V2[seed01] = np.clip(V2[seed01] * 1.35 + 35, 0, 255)

    # Optional: clamp seed area to minimum "good" S/V
    # (forces your desired style)
    S2[seed01] = np.maximum(S2[seed01], 255)   # ~90% of 255
    V2[seed01] = np.maximum(V2[seed01], 230)   # ~75% of 255

    # -----------------------------
    # (6) Stronger hue pull toward blue center
    # -----------------------------
    H2 = H.copy()
    blue_center = 100.0
    pull = 0.55   # was 0.25, now harsher

    good = seed01  # since we already enforced S_min_after
    H2[good] = (1 - pull) * H2[good] + pull * blue_center
    H2 = np.clip(H2, 0, 179)

    hsv_out = np.stack([H2, S2, V2], axis=-1).astype(np.uint8)
    out_rgb = cv2.cvtColor(hsv_out, cv2.COLOR_HSV2RGB)

    if debug:
        nseed = int(seed01.sum())
        print(f"[RESCUE EXTREME] seed_pixels={nseed} gamma={gamma} clahe=4.0 pull={pull}")

    return out_rgb


New_Origignal_Img1111 = io.imread("Localization Test/NewTestCases/white car harsh right angle.jpg")
New_Origigna2_Img1111 = io.imread("Localization Test/NewTestCases/white car harshest angle.jpg")
New_Origigna3_Img1111 = io.imread("Localization Test/NewTestCases/white car slight angle .jpg")
New_Origigna4_Img1111 = io.imread("Localization Test/NewTestCases/white car straight.jpg")
New_Origigna5_Img1111 = io.imread("Localization Test/NewTestCases/white car very elevated angle.jpg")
New_Origigna6_Img1111 = io.imread("Localization Test/NewTestCases/white car very harsh left angle with alot of noise.jpg")

# New_Origigna7_Img1111 = io.imread("Localization Test/NewTestCases/angle no bright.jpeg")
New_Origigna8_Img1111 = io.imread("Localization Test/NewTestCases/blury gedan.jpeg")
New_Origigna9_Img1111 = io.imread("Localization Test/NewTestCases/bright with angle.jpeg")
New_Origigna12_Img1111 = io.imread("Localization Test/NewTestCases/very dirty and noisy.jpeg")
New_Origigna13_Img1111 = io.imread("Localization Test/NewTestCases/very light bright.jpeg")
New_Origigna14_Img1111 = io.imread("Localization Test/NewTestCases/veryyyy far.jpeg")
New_Origigna14_Img111411111 = io.imread("WhatsApp Image 2025-12-18 at 1.32.03 PM.jpeg")


# cropped_Plate1=debug_blue_crop(New_Origigna14_Img111111111,"Detected_New_Origignal_Img1111") 
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False)

# cropped_Plate1=debug_blue_crop(New_Origigna14_Img111111211,"Detected_New_Origignal_Img1111") 
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False)

# cropped_Plate1=debug_blue_crop(New_Origigna14_Img111411111,"Detected_New_Origignal_Img1111") 
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False)

# cropped_Plate1=debug_blue_crop(New_Origignal_Img1111,"Detected_New_Origignal_Img1111")
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False) 

# cropped_Plate2=debug_blue_crop(New_Origigna2_Img1111,"Detected_New_Origigna2_Img1111")    
# cropped_Plate2=debug_blue_crop(cropped_Plate2,"Detected_New_Origigna2_Img1111",False)  

# cropped_Plate3=debug_blue_crop(New_Origigna3_Img1111,"Detected_New_Origigna3_Img1111")    
# cropped_Plate3=debug_blue_crop(cropped_Plate3,"Detected_New_Origigna3_Img1111",False)    

# cropped_Plate4=debug_blue_crop(New_Origigna4_Img1111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate4=debug_blue_crop(cropped_Plate4,"Detected_New_Origigna4_Img1111",False) 
# cropped_Plate5=debug_blue_crop(New_Origigna5_Img1111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate5=debug_blue_crop(cropped_Plate5,"Detected_New_Origigna5_Img1111",False)     
# cropped_Plate6=debug_blue_crop(New_Origigna6_Img1111,"Detected_New_Origigna6_Img1111")
# cropped_Plate6=debug_blue_crop(cropped_Plate6,"Detected_New_Origigna6_Img1111",False)  

# cropped_Plate7=debug_blue_crop(Origignal_Img1,"Detected_New_Origignal_Img1111")    
# cropped_Plate7=debug_blue_crop(cropped_Plate7,"Detected_New_Origignal_Img1111",False)    
# cropped_Plate8=debug_blue_crop(Origignal_Img2,"Detected_New_Origigna2_Img1111")    
# cropped_Plate8=debug_blue_crop(cropped_Plate8,"Detected_New_Origigna2_Img1111",False)    
# cropped_Plate9=debug_blue_crop(Origignal_Img3,"Detected_New_Origigna3_Img1111")    
# cropped_Plate9=debug_blue_crop(cropped_Plate9,"Detected_New_Origigna3_Img1111",False)     
# cropped_Plate10=debug_blue_crop(Origignal_Img4,"Detected_New_Origigna4_Img1111")    
# cropped_Plate10=debug_blue_crop(cropped_Plate10,"Detected_New_Origigna4_Img1111",False)      
# cropped_Plate11=debug_blue_crop(Origignal_Img5,"Detected_New_Origigna5_Img1111")
# cropped_Plate11=debug_blue_crop(cropped_Plate11,"Detected_New_Origigna5_Img1111",False)


# cropped_Plate12=debug_blue_crop(Origignal_Img11,"Detected_New_Origigna6_Img1111") 
# cropped_Plate12=debug_blue_crop(cropped_Plate12,"Detected_New_Origigna6_Img1111",False)  
# cropped_Plate13=debug_blue_crop(Origignal_Img12,"Detected_New_Origignal_Img1111")    
# cropped_Plate13=debug_blue_crop(cropped_Plate13,"Detected_New_Origignal_Img1111",False)    
# cropped_Plate14=debug_blue_crop(Origignal_Img13,"Detected_New_Origigna2_Img1111")    
# cropped_Plate14=debug_blue_crop(cropped_Plate14,"Detected_New_Origigna2_Img1111",False)      
# cropped_Plate15=debug_blue_crop(Origignal_Img14,"Detected_New_Origigna3_Img1111")    
# cropped_Plate15=debug_blue_crop(cropped_Plate15,"Detected_New_Origigna3_Img1111",False)    
# cropped_Plate16=debug_blue_crop(Origignal_Img15,"Detected_New_Origigna4_Img1111")
# cropped_Plate16=debug_blue_crop(cropped_Plate16,"Detected_New_Origigna4_Img1111",False)

# cropped_Plate17=debug_blue_crop(Origignal_Img111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate17=debug_blue_crop(cropped_Plate17,"Detected_New_Origigna5_Img1111",False)     
# cropped_Plate18=debug_blue_crop(Origignal_Img112,"Detected_New_Origigna6_Img1111") 
# cropped_Plate18=debug_blue_crop(cropped_Plate18,"Detected_New_Origigna6_Img1111",False) 
# cropped_Plate19=debug_blue_crop(Origignal_Img113,"Detected_New_Origignal_Img1111")    
# cropped_Plate19=debug_blue_crop(cropped_Plate19,"Detected_New_Origignal_Img1111",False)    

# cropped_Plate22=debug_blue_crop(Origignal_Img1111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate22=debug_blue_crop(cropped_Plate22,"Detected_New_Origigna4_Img1111",False)   
# cropped_Plate23=debug_blue_crop(Origignal_Img1112,"Detected_New_Origigna5_Img1111")    
# cropped_Plate23=debug_blue_crop(cropped_Plate23,"Detected_New_Origigna5_Img1111",False)    
# cropped_Plate24=debug_blue_crop(Origignal_Img1113,"Detected_New_Origigna6_Img1111") 
# cropped_Plate24=debug_blue_crop(cropped_Plate24,"Detected_New_Origigna6_Img1111",False)   
# cropped_Plate25=debug_blue_crop(Origignal_Img1114,"Detected_New_Origignal_Img1111")    
# cropped_Plate25=debug_blue_crop(cropped_Plate25,"Detected_New_Origignal_Img1111",False)      
 

# cropped_Plate27=debug_blue_crop(Original_Img11111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate27=debug_blue_crop(cropped_Plate27,"Detected_New_Origigna4_Img1111",False)    
# cropped_Plate28=debug_blue_crop(Original_Img11112,"Detected_New_Origigna5_Img1111")    
# cropped_Plate28=debug_blue_crop(cropped_Plate28,"Detected_New_Origigna5_Img1111",False)     
# cropped_Plate29=debug_blue_crop(Original_Img11113,"Detected_New_Origigna6_Img1111") 
# cropped_Plate29=debug_blue_crop(cropped_Plate29,"Detected_New_Origigna6_Img1111",False)  
# cropped_Plate30=debug_blue_crop(Original_Img11114,"Detected_New_Origignal_Img1111")    
# cropped_Plate30=debug_blue_crop(cropped_Plate30,"Detected_New_Origignal_Img1111",False)    
# cropped_Plate31=debug_blue_crop(Original_Img11115,"Detected_New_Origigna2_Img1111") 
# cropped_Plate31=debug_blue_crop(cropped_Plate31,"Detected_New_Origigna2_Img1111",False)   

# # cropped_Plate32=debug_blue_crop(New_Origigna7_Img1111,"Detected_New_Origigna4_Img1111")    
# # cropped_Plate32=debug_blue_crop(cropped_Plate32,"Detected_New_Origigna4_Img1111",False)   
# cropped_Plate33=debug_blue_crop(New_Origigna8_Img1111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate33=debug_blue_crop(cropped_Plate33,"Detected_New_Origigna5_Img1111",False)      
# cropped_Plate34=debug_blue_crop(New_Origigna9_Img1111,"Detected_New_Origigna6_Img1111") 
# cropped_Plate34=debug_blue_crop(cropped_Plate34,"Detected_New_Origigna6_Img1111",False)   
# cropped_Plate37=debug_blue_crop(New_Origigna12_Img1111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate37=debug_blue_crop(cropped_Plate37,"Detected_New_Origigna4_Img1111",False)    
# cropped_Plate38=debug_blue_crop(New_Origigna13_Img1111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate38=debug_blue_crop(cropped_Plate38,"Detected_New_Origigna5_Img1111",False)    
# cropped_Plate39=debug_blue_crop(New_Origigna14_Img1111,"Detected_New_Origigna6_Img1111") 
# cropped_Plate39=debug_blue_crop(cropped_Plate39,"Detected_New_Origigna6_Img1111",False)










# import os
# from skimage import io

# BASE_PATH = "Localization Test/Vehicles"

# START_ID = 1
# END_ID   = 2087

# for i in range(START_ID, END_ID,1):
#     import matplotlib.pyplot as plt
#     plt.ioff()  # disable interactive rendering
#     fname = f"{i:04d}.jpg"        # 0001.jpg, 0002.jpg, ...
#     fpath = os.path.join(BASE_PATH, fname)

#     if not os.path.exists(fpath):
#         print(f"[SKIP] Missing: {fpath}")
#         continue

#     try:
#         img = io.imread(fpath)    # RGB (as you want)
#         title = f"Vehicle_{fname}"
#         print(f"[RUN] {title}")
#         img1=debug_blue_crop(img, title)
#         img2=debug_blue_crop(img1, title,False)

#     except Exception as e:
#         print(f"[ERROR] {fname}: {e}")




import os
from skimage import io
import matplotlib.pyplot as plt

BASE_PATH = "Localization Test/Vehicles"
OUT_DIR   = "Localization Test/FinalCrops"
os.makedirs(OUT_DIR, exist_ok=True)

IDS = [
    2087, 2084, 2083, 2082, 2081, 2080, 2079, 2078,
    2076, 2073, 2072, 2070, 2069, 2068, 2066, 2065,
    2063, 2062, 2060, 2058, 2057, 2056, 2055, 2054,
    2051, 2049, 2047, 2044, 2043, 2041,
    2039, 2038, 2037, 2036, 2035, 2034,
    2033, 2031, 2030, 2029, 2026, 2025,

    1, 2, 6, 10, 13, 14, 15, 16, 25, 33, 34, 35,
    39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    51, 52, 57, 58, 59, 60, 61, 62, 63, 64, 65,
    124
]

plt.ioff()

for i in IDS:
    fname = f"{i:04d}.jpg"
    fpath = os.path.join(BASE_PATH, fname)

    if not os.path.exists(fpath):
        print(f"[SKIP] Missing: {fname}")
        continue

    try:
        img = io.imread(fpath)
        title = f"Vehicle_{fname}"
        print(f"[RUN] {title}")

        img1 = debug_blue_crop(img, title, enable_perspective=True)
        img2 = debug_blue_crop(img1, title, enable_perspective=False)

        if img2 is not None:
            out_path = os.path.join(OUT_DIR, f"{i:04d}_final.png")
            io.imsave(out_path, img2)
            print(f"[SAVE] {out_path}")

    except Exception as e:
        print(f"[ERROR] {fname}: {e}")





In [ ]:
'''
import pytesseract
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import os
import easyocr
os.environ['TESSDATA_PREFIX'] = os.path.abspath("Dataset")
# os.environ.pop('TESSDATA_PREFIX', None)
def add_padding(img, pad=10):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=255
    )

def add_paddingblack(img, pad=200):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=0
    )

def recognize_arabic_digit(image_path, show_preprocessed=False):
    """
    Recognize a single Arabic digit from an image.
    
    Args:
        image_path (str): Path to image file.
        show_preprocessed (bool): If True, shows the binary preprocessed image.
    
    Returns:
        str: Detected Arabic digit or empty string if not recognized.
    """
    
    # 1. Load image
    # img = io.imread(image_path)
    # img=add_paddingblack(img)
    # img=add_padding(img)
    img=d_mask
    
    # # 2. Convert to grayscale if needed
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    
    # # 3. Resize image up to help OCR
    # gray = cv2.resize(gray, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
    
    # # 4. Smooth image to reduce noise
    # blur = cv2.GaussianBlur(gray, (3,3), 0)
    
    # # 5. Threshold to get binary image
    # _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # # 6. Invert if background is dark
    # if np.mean(thresh) < 127:
    #     thresh = cv2.bitwise_not(thresh)
    
    # Optional: show preprocessed image
    if show_preprocessed:
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.show()
    
    # 7. Configure Tesseract for single character + Arabic digits only
    # config = f'--oem 3 --psm 10 -c tessedit_char_whitelist={ARABIC_DIGITS}'
    
    # 8. Run OCR
    # text = pytesseract.image_to_string(thresh, lang='ara')

    reader = easyocr.Reader(['ar'])
    results = reader.readtext(img)
    texts = [text for bbox, text, prob in results]


    # 9. Clean result
    # text = text.strip()
    return texts

# Example usage:
digit = recognize_arabic_digit("letter_0.png", show_preprocessed=True)
print("Detected Arabic digit EASYOCR:", repr(digit))
# print("Detected Arabic digit:", repr(digit))

'''

In [ ]:
'''
MAAW STEPS 
-(ALL-IN-ONE CELL) — Step2 v4 (ANTI-TOP-CROP SAFETY)
Step 1 (v2): Robust header cut (blue CC + safe fallbacks)
Step 2 (v4): Robust text-band crop using:
  - black-hat + threshold
  - remove long HORIZONTAL lines only (avoid killing ١/ا/ل)
  - Y band from horizontal projection
  - X band from vertical projection (min->max span, not "largest segment")
  - refine Y again inside X band
  - strong TOP padding + border-touch expansion (protect dots/hamza)
Shows: original | after header cut | text-band crop  (for first MAX_SHOW samples)
Saves under: "maaw outputs/<stem>/step1_*.png" and "maaw outputs/<stem>/step2_*.png"
'''

import os, glob
import numpy as np
import matplotlib.pyplot as plt
import cv2

# ============================================================
# VISUALIZATION (missing)
# ============================================================
def show_images(images, titles=None, figsize_scale=4):
    n = len(images)
    if titles is None:
        titles = [f"({i+1})" for i in range(n)]
    plt.figure(figsize=(figsize_scale * n, figsize_scale))
    for i, (img, t) in enumerate(zip(images, titles)):
        ax = plt.subplot(1, n, i + 1)
        if img is None:
            ax.set_title(t)
            ax.axis("off")
            continue

        # convert float images safely for display
        if isinstance(img, np.ndarray) and img.dtype != np.uint8 and img.max() <= 1.01:
            img = (np.clip(img, 0, 1) * 255).astype(np.uint8)

        if img.ndim == 2:
            plt.imshow(img, cmap="gray")
        else:
            plt.imshow(img)
        ax.set_title(t)
        ax.axis("off")
    plt.show()

# ============================================================
# CONFIG
# ============================================================
INPUT_FOLDER = "Localization Test/FinalCrops"
OUTPUT_ROOT  = "maaw outputs"
MAX_SHOW     = 30    # set None to show all images
SAVE_DEBUG   = True

# --- NEW safety pads (small, but effective) ---


STEP2_BG_PAD_X_FRAC = 0.04     # final safety margin after Step2
STEP2_BG_PAD_Y_FRAC = 0.08


# ============================================================
# IO HELPERS
# ============================================================
def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def _to_uint8(img):
    if img is None:
        return None
    if img.dtype == np.uint8:
        return img
    if img.dtype in (np.float32, np.float64):
        x = img
        if x.max() <= 1.0 + 1e-6:
            x = x * 255.0
        return np.clip(x, 0, 255).astype(np.uint8)
    return np.clip(img, 0, 255).astype(np.uint8)

def bgr_to_rgb(bgr):
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def save_rgb(path, rgb):
    ensure_dir(os.path.dirname(path))
    x = _to_uint8(rgb)
    if x is None:
        return False
    if x.ndim == 3 and x.shape[2] == 3:
        x = cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
    ok = cv2.imwrite(path, x)
    if not ok:
        print("[WARN] Failed to save:", path)
    return ok

def load_plate_images(folder=INPUT_FOLDER, exts=("*.png","*.jpg","*.jpeg","*.bmp","*.tif","*.tiff")):
    if not os.path.isdir(folder):
        print(f"[ERROR] Folder not found: {folder}")
        print("Current working directory:", os.getcwd())
        return []
    paths = []
    for e in exts:
        paths += glob.glob(os.path.join(folder, e))
        paths += glob.glob(os.path.join(folder, e.upper()))
    paths = sorted(list(set(paths)))

    items = []
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_COLOR)
        if img is None:
            print("[WARN] Could not read:", p)
            continue
        items.append({
            "path": p,
            "name": os.path.basename(p),
            "stem": os.path.splitext(os.path.basename(p))[0],
            "bgr": img
        })
    print(f"[OK] Loaded {len(items)} images from: {folder}")
    return items



def _estimate_bg_color_bgr(bgr):
    """Robust-ish light background estimate (median of bright-ish pixels)."""
    H, W = bgr.shape[:2]
    sample = bgr.reshape(-1, 3)
    gray = (0.114*sample[:,0] + 0.587*sample[:,1] + 0.299*sample[:,2])
    keep = gray > np.percentile(gray, 55)  # keep brighter half (plate background)
    if keep.sum() < 200:
        keep = gray > 80
    if keep.sum() < 200:
        return (200, 200, 200)
    med = np.median(sample[keep], axis=0)
    return (int(med[0]), int(med[1]), int(med[2]))  # BGR

def _pad_const_bgr(bgr, top=0, bottom=0, left=0, right=0, color=(200,200,200)):
    if top==0 and bottom==0 and left==0 and right==0:
        return bgr
    return cv2.copyMakeBorder(
        bgr, int(top), int(bottom), int(left), int(right),
        borderType=cv2.BORDER_CONSTANT, value=color
    )


def _tighten_to_plate_background(bgr, min_area_frac=0.18, pad_x_frac=0.02, pad_y_frac=0.04):
    """
    Post-crop cleanup:
    Keep the biggest BRIGHT plate-background component and crop to it.
    This removes bumper/body/edge-bars that leak into Step2.
    """
    if bgr is None:
        return bgr
    H, W = bgr.shape[:2]
    if H < 20 or W < 20:
        return bgr

    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(gray, (5, 5), 0)

    # Otsu split: bright plate bg vs dark stuff (text/bumper/body)
    _, th = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Ensure "255" corresponds to BRIGHT region
    m255 = g[th == 255].mean() if np.any(th == 255) else 0
    m0   = g[th == 0].mean()   if np.any(th == 0)   else 0
    if m255 < m0:
        th = 255 - th

    # Connect the plate background into one component
    kx = max(7, int(0.06 * W))
    ky = max(7, int(0.20 * H))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (kx, ky)))
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN,  cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)))

    num, lab, stats, _ = cv2.connectedComponentsWithStats((th > 0).astype(np.uint8), 8)
    if num <= 1:
        return bgr

    # Biggest bright component (ignore background idx 0)
    idx = 1 + int(np.argmax(stats[1:, cv2.CC_STAT_AREA]))
    x, y, w, h, area = stats[idx]

    area_frac = area / float(H * W + 1e-6)
    if area_frac < min_area_frac:
        return bgr

    # Small safety padding around the bright plate region
    px = max(2, int(pad_x_frac * w))
    py = max(2, int(pad_y_frac * h))

    x0 = max(0, x - px)
    y0 = max(0, y - py)
    x1 = min(W, x + w + px)
    y1 = min(H, y + h + py)

    # Sanity: don't shrink too aggressively
    if (x1 - x0) < 0.35 * W or (y1 - y0) < 0.35 * H:
        return bgr

    return bgr[y0:y1, x0:x1].copy()




# ============================================================
# STEP 1 (v2): robust header cut
# ============================================================
def cut_header_step1_v2(img_bgr):
    H, W = img_bgr.shape[:2]
    top_lim = int(0.55 * H)

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    blue = cv2.inRange(hsv, (85, 60, 30), (140, 255, 255))
    blue[top_lim:, :] = 0

    k1 = max(3, int(0.02 * min(H, W)) | 1)
    open_k = cv2.getStructuringElement(cv2.MORPH_RECT, (k1, k1))
    blue_clean = cv2.morphologyEx(blue, cv2.MORPH_OPEN, open_k)

    close_k = cv2.getStructuringElement(
        cv2.MORPH_RECT,
        (max(9, int(0.08 * W)), max(3, int(0.03 * H)))
    )
    blue_clean = cv2.morphologyEx(blue_clean, cv2.MORPH_CLOSE, close_k)

    num, lab, stats, _ = cv2.connectedComponentsWithStats((blue_clean > 0).astype(np.uint8), 8)
    best = None
    for i in range(1, num):
        x, y, w, h, area = stats[i]
        if y > int(0.35 * H):
            continue
        if w < 0.55 * W:
            continue
        if h > 0.40 * H:
            continue
        if h < max(6, int(0.06 * H)):
            continue
        if area < 0.03 * W * H:
            continue
        if (best is None) or (area > best["area"]):
            best = {"x": x, "y": y, "w": w, "h": h, "area": area}

    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(gray, (5, 5), 0)
    sobel_y = cv2.Sobel(g, cv2.CV_32F, 0, 1, ksize=3)
    row_edge = np.abs(sobel_y)[:top_lim].mean(axis=1)

    k = max(5, int(0.03 * H) | 1)
    row_edge_s = np.convolve(row_edge, np.ones(k, np.float32) / k, mode="same")

    y0 = int(0.10 * H)
    y1 = int(0.48 * H)
    y0 = max(0, min(y0, top_lim - 2))
    y1 = max(y0 + 1, min(y1, top_lim - 1))

    cut_y_edge = int(np.argmax(row_edge_s[y0:y1]) + y0)
    med = np.median(row_edge_s[y0:y1]) + 1e-6
    peak = row_edge_s[cut_y_edge]
    if peak / med < 2.0:
        cut_y_edge = None

    cut_y_blue = None
    if best is not None:
        cut_y_blue = int(best["y"] + best["h"] + 2)

    cut_y = None
    method = "none"
    if cut_y_blue is not None:
        cut_y = cut_y_blue
        method = "blueCC"
        if cut_y / H > 0.48:
            if cut_y_edge is not None:
                cut_y = cut_y_edge
                method = "edge(replace_deep_blue)"
            else:
                cut_y = None
                method = "none(reject_deep)"
    elif cut_y_edge is not None:
        cut_y = cut_y_edge
        method = "edge"

    if cut_y is not None:
        body = img_bgr[cut_y:, :]
        g2 = cv2.cvtColor(body, cv2.COLOR_BGR2GRAY)
        dark_ratio = (g2 < 140).mean()
        if dark_ratio < 0.02:
            if cut_y_edge is not None and method.startswith("blue"):
                cut_y = cut_y_edge
                method = "edge(replace_bad_blue)"
            else:
                cut_y = None
                method = "none(reject_bad)"
    # --- SAFETY: don't cut too deep (keeps a small strip under the blue header) ---
    if cut_y is not None:
        backoff = max(2, int(0.02 * H))  # ~2% of height
        cut_y = max(0, cut_y - backoff)


    body_bgr = img_bgr if cut_y is None else img_bgr[cut_y:, :].copy()

    dbg = np.zeros((H, W), dtype=np.uint8)
    dbg[:top_lim, :] = blue_clean[:top_lim, :]
    dbg_rgb = cv2.cvtColor(dbg, cv2.COLOR_GRAY2RGB)

    if best is not None:
        x, y, w, h = best["x"], best["y"], best["w"], best["h"]
        cv2.rectangle(dbg_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(dbg_rgb, "blue header CC", (x + 5, y + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)

    if cut_y is not None:
        cv2.line(dbg_rgb, (0, cut_y), (W - 1, cut_y), (255, 0, 0), 2)
        cv2.putText(dbg_rgb, f"cut={cut_y} ({method})", (8, min(H - 8, cut_y + 18)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255,255,255), 2, cv2.LINE_AA)
    else:
        cv2.putText(dbg_rgb, "no cut", (8, 22),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

    return body_bgr, dbg_rgb

# ============================================================
# STEP 2 (v4): projection-first + border-touch expansion
# ============================================================
def _smooth_1d(x, k):
    k = int(k)
    if k < 3:
        return x
    if k % 2 == 0:
        k += 1
    ker = np.ones(k, np.float32) / k
    return np.convolve(x, ker, mode="same")

def _band_from_projection(proj, lo, hi, rel_thr=0.33):
    if hi <= lo + 2:
        return None
    p = proj[lo:hi]
    if p.size == 0:
        return None
    peak = float(p.max())
    if peak < 1e-9:
        return None
    thr = rel_thr * peak
    good = np.where(p >= thr)[0]
    if len(good) == 0:
        return None
    # take the biggest span (min->max) of the densest cluster:
    return (int(good.min()) + lo, int(good.max()) + lo)

def _touch_expand(mask, y0, y1, x0, x1, H, W,
                  touch_rows=3, touch_cols=3,
                  frac=0.001,
                  expand_y=0.08, expand_x=0.06):
    """
    If foreground touches crop borders, expand crop outward.
    """
    crop = mask[y0:y1+1, x0:x1+1]
    if crop.size == 0:
        return y0, y1, x0, x1

    # top
    if (crop[:touch_rows, :] > 0).mean() > frac:
        dy = max(2, int(expand_y * H))
        y0 = max(0, y0 - dy)

    # bottom
    crop = mask[y0:y1+1, x0:x1+1]
    if (crop[-touch_rows:, :] > 0).mean() > frac:
        dy = max(2, int(expand_y * H))
        y1 = min(H - 1, y1 + dy)

    # left
    crop = mask[y0:y1+1, x0:x1+1]
    if (crop[:, :touch_cols] > 0).mean() > frac:
        dx = max(2, int(expand_x * W))
        x0 = max(0, x0 - dx)

    # right
    crop = mask[y0:y1+1, x0:x1+1]
    if (crop[:, -touch_cols:] > 0).mean() > frac:
        dx = max(2, int(expand_x * W))
        x1 = min(W - 1, x1 + dx)

    return y0, y1, x0, x1


def _bbox_union_filtered(mask01):
    """
    mask01: uint8 mask (0/255 or 0/1), single channel
    Returns (x0,y0,x1,y1) in mask coords, or None
    Filters obvious plate frame lines (very long thin vertical/horizontal).
    """
    m = (mask01 > 0).astype(np.uint8)
    H, W = m.shape[:2]

    num, lab, stats, _ = cv2.connectedComponentsWithStats(m, 8)
    boxes = []

    # dynamic minimum area (works across scales)
    min_area = max(60, int(0.0008 * H * W))

    for i in range(1, num):
        x, y, w, h, area = stats[i]
        if area < min_area:
            continue

        # Filter obvious FRAME/BORDER lines:
        # - very tall & thin (left/right border)
        if (h > 0.92 * H) and (w < 0.10 * W):
            continue
        # - very wide & thin (bottom bar / separators)
        if (w > 0.92 * W) and (h < 0.22 * H):
            continue

        boxes.append((x, y, w, h, area))

    if not boxes:
        return None

    x0 = min(b[0] for b in boxes)
    y0 = min(b[1] for b in boxes)
    x1 = max(b[0] + b[2] - 1 for b in boxes)
    y1 = max(b[1] + b[3] - 1 for b in boxes)

    return x0, y0, x1, y1


def _remove_edge_vertical_bars(mask, edge_margin_frac=0.07, min_h_frac=0.55):
    """
    Remove long-ish vertical bars near left/right edges (ONLY for bbox estimation).
    Safe for digits like '١' because it's rarely at extreme edges.
    """
    m = (mask > 0).astype(np.uint8) * 255
    H, W = m.shape[:2]
    edge = max(2, int(edge_margin_frac * W))

    # detect vertical bars
    vk_h = max(10, int(min_h_frac * H))
    vk = cv2.getStructuringElement(cv2.MORPH_RECT, (3, vk_h))
    vbars = cv2.morphologyEx(m, cv2.MORPH_OPEN, vk)

    # only remove them near edges
    out = m.copy()
    if edge < W:
        out[:, :edge] = cv2.subtract(out[:, :edge], vbars[:, :edge])
        out[:, W-edge:] = cv2.subtract(out[:, W-edge:], vbars[:, W-edge:])

    return out







def auto_text_band_step2_v4(body_bgr):
    H, W = body_bgr.shape[:2]
    gray = cv2.cvtColor(body_bgr, cv2.COLOR_BGR2GRAY)

    # 1) blackhat (stroke enhancer)
    kx = max(21, int(0.22 * W))
    ky = max(9,  int(0.16 * H))
    bh_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kx, ky))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, bh_kernel)

    # 2) threshold (blackhat)
    _, bw = cv2.threshold(blackhat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if (bw > 0).mean() < 0.002:
        bw = cv2.adaptiveThreshold(blackhat, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 31, -2)

    # Extra mask ONLY for "touch detection" (captures thin ١/ا better than blackhat sometimes)
    bw_touch = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, 31, 7)
    bw_touch = cv2.morphologyEx(bw_touch, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_RECT,(3,3)))
    touch_mask = cv2.bitwise_or(bw, bw_touch)

    # 3) remove long HORIZONTAL lines only
    hk_w = max(25, int(0.40 * W))
    hk = cv2.getStructuringElement(cv2.MORPH_RECT, (hk_w, 3))
    horiz = cv2.morphologyEx(bw, cv2.MORPH_OPEN, hk)
    bw2 = cv2.subtract(bw, horiz)
    bw2 = cv2.morphologyEx(bw2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)))

    # Build a stronger mask for PROJECTIONS/BBOX so we don't miss a whole block (digits or letters)
    bw_proj = cv2.bitwise_or(bw2, bw_touch)

    # 4) Y band from row projection (USE bw2 only to avoid bumper/body noise)
    row = (bw2 > 0).mean(axis=1).astype(np.float32)
    row_s = _smooth_1d(row, max(9, int(0.08 * H)))

    a = int(0.03 * H)

    # Prefer upper ~82% of the image first (text is almost always above bumper)
    b_pref = int(0.82 * H)
    yseg = _band_from_projection(row_s, a, b_pref, rel_thr=0.33)

    # If not found, fall back to the full range
    if yseg is None:
        b_full = int(0.97 * H)
        yseg = _band_from_projection(row_s, a, b_full, rel_thr=0.33)

    if yseg is None:
        y0, y1 = int(0.10 * H), int(0.90 * H)
    else:
        y0, y1 = yseg

    # 5) Robust bbox using BOTH masks + edge-bar removal + X recovery
    roi_proj  = bw_proj[y0:y1+1, :].copy()
    roi_touch = bw_touch[y0:y1+1, :].copy()

    # remove edge vertical bars only for bbox estimation
    roi_proj  = _remove_edge_vertical_bars(roi_proj,  edge_margin_frac=0.07, min_h_frac=0.55)
    roi_touch = _remove_edge_vertical_bars(roi_touch, edge_margin_frac=0.07, min_h_frac=0.55)

    # bbox from each, then union (this fixes 0010 when blackhat misses digits)
    bb1 = _bbox_union_filtered(roi_proj)
    bb2 = _bbox_union_filtered(roi_touch)

    if (bb1 is None) and (bb2 is None):
        x0, x1 = 0, W - 1
    else:
        if bb1 is None:
            rx0, ry0, rx1, ry1 = bb2
        elif bb2 is None:
            rx0, ry0, rx1, ry1 = bb1
        else:
            rx0 = min(bb1[0], bb2[0])
            ry0 = min(bb1[1], bb2[1])
            rx1 = max(bb1[2], bb2[2])
            ry1 = max(bb1[3], bb2[3])

        # refine BOTH X and Y
        y0_old = y0
        x0, x1 = rx0, rx1
        y0 = y0_old + ry0
        y1 = y0_old + ry1

        # ---- X recovery (fixes 0015 right cut) using column projection on roi_touch ----
        col = (roi_touch > 0).mean(axis=0).astype(np.float32)
        col_s = _smooth_1d(col, max(7, int(0.06 * W)))
        xseg = _band_from_projection(col_s, 0, W, rel_thr=0.22)
        if xseg is not None:
            px0, px1 = xseg
            x0 = min(x0, px0)
            x1 = max(x1, px1)


    # 7) padding (TOP-heavy, but keep bottom tight to avoid bumper)
    band_h = (y1 - y0 + 1)
    band_w = (x1 - x0 + 1)

    pad_top = max(int(0.40 * band_h), int(0.12 * H))   # keep strong top safety
    pad_bot = max(int(0.10 * band_h), int(0.04 * H))   # MUCH smaller bottom padding
    pad_x   = max(int(0.06 * band_w), int(0.03 * W))

    y0 = max(0, y0 - pad_top)
    y1 = min(H - 1, y1 + pad_bot)
    x0 = max(0, x0 - pad_x)
    x1 = min(W - 1, x1 + pad_x)

    # 8) border-touch expansion (use stronger touch_mask)
    y0, y1, x0, x1 = _touch_expand(
        touch_mask, y0, y1, x0, x1, H, W,
        touch_rows=6, touch_cols=6,
        frac=0.0004,
        expand_y=0.07,   # was 0.10 (too aggressive downward when bumper is foreground)
        expand_x=0.06    # was 0.08
    )


    # final sanity (avoid tiny crops, but don't blow up to include bumper)
    min_h = int(0.28 * H)  # was 0.40*H (too big)
    cur_h = (y1 - y0 + 1)
    if cur_h < min_h:
        need = min_h - cur_h
        up = int(0.65 * need)
        down = need - up
        y0 = max(0, y0 - up)
        y1 = min(H - 1, y1 + down)


    band = body_bgr[y0:y1+1, x0:x1+1].copy()

    # NEW: tighten to the bright plate rectangle (removes bumper/body/edge-bars)
    band = _tighten_to_plate_background(band, min_area_frac=0.18, pad_x_frac=0.02, pad_y_frac=0.04)


    # FINAL SAFE BG padding (guarantees no top/bottom touching even when crop hits image edge)
    bg = _estimate_bg_color_bgr(band)
    bh, bw_ = band.shape[:2]
    px = max(2, int(STEP2_BG_PAD_X_FRAC * bw_))
    py = max(2, int(STEP2_BG_PAD_Y_FRAC * bh))
    band = _pad_const_bgr(band, top=py, bottom=py, left=px, right=px, color=bg)

    dbg = cv2.cvtColor(bw2, cv2.COLOR_GRAY2RGB)
    cv2.rectangle(dbg, (x0, y0), (x1, y1), (0, 255, 0), 2)
    cv2.putText(dbg, "Step2 v4 (fixed X-run + safe pad)", (8, 22),
                cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255,255,255), 2, cv2.LINE_AA)

    return band, dbg







# ============================================================
# STEP 3 (v1): segment characters (digits + letters) + split groups
# Output chars are 150x150, black ink on white background.
# ============================================================

# === Plate constraints (set DIGIT_MAX=4 if you insist, but your sample 0002 shows 5 digits) ===
DIGIT_MIN  = 1
DIGIT_MAX  = 5   # <-- change to 4 only if you're 100% sure your dataset never has 5-digit plates
LETTER_MIN = 1
LETTER_MAX = 3




def _remove_center_separator_bar(mask255, min_h_frac=0.55, max_w_frac=0.14,
                                 x_lo=0.22, x_hi=0.78):
    """
    Remove the main digit/letter separator bar (tall vertical stroke near the middle).
    Safe vs digit '١' because '١' is not usually centered; also separator is typically taller.
    """
    m = (mask255 > 0).astype(np.uint8) * 255
    H, W = m.shape[:2]
    if H < 10 or W < 10:
        return mask255

    vk_h = max(12, int(min_h_frac * H))
    vk_w = max(1, int(0.02 * W))  # catch thick separators too
    vk = cv2.getStructuringElement(cv2.MORPH_RECT, (vk_w, vk_h))
    vbar = cv2.morphologyEx(m, cv2.MORPH_OPEN, vk)
    vbar = cv2.dilate(vbar, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 9)), iterations=1)

    num, lab, stats, _ = cv2.connectedComponentsWithStats((vbar > 0).astype(np.uint8), 8)
    out = m.copy()

    for i in range(1, num):
        x, y, w, h, area = stats[i]
        cx = (x + w/2.0) / (W + 1e-6)

        if not (x_lo <= cx <= x_hi):
            continue
        if h < min_h_frac * H:
            continue
        if w > max_w_frac * W:
            continue

        out[lab == i] = 0

    return out





def _pad_to_square_u8(img, pad_value=255):
    h, w = img.shape[:2]
    if h == w:
        return img
    if h > w:
        pad = (h - w) // 2
        return cv2.copyMakeBorder(img, 0, 0, pad, h - w - pad, cv2.BORDER_CONSTANT, value=pad_value)
    else:
        pad = (w - h) // 2
        return cv2.copyMakeBorder(img, pad, w - h - pad, 0, 0, cv2.BORDER_CONSTANT, value=pad_value)

def _resize_150(img):
    return cv2.resize(img, (150, 150), interpolation=cv2.INTER_NEAREST)

def _remove_long_lines(mask255):
    """
    Step3 cleanup (safer + stronger):
    - Remove long/broken horizontal frame/baseline lines (multi-scale).
    - Detect tall vertical bars (borders + separator), even if broken.
    - Remove vertical bars near content edges (padding-safe).
    - Remove separator using the strongest valley in column projection (if found).
    - Remove dense top/bottom reflection rows.
    """
    H, W = mask255.shape[:2]
    m = mask255.copy()

    # ----------------------------
    # 1) Horizontal lines (multi-scale)
    # ----------------------------
    horiz_all = np.zeros_like(m)
    widths = [
        max(25, int(0.90 * W)),
        max(25, int(0.70 * W)),
        max(25, int(0.50 * W)),
        max(25, int(0.35 * W)),
        max(25, int(0.25 * W)),
        max(25, int(0.18 * W)),
    ]
    for hk_w in widths:
        for hk_h in (2, 3):
            hk = cv2.getStructuringElement(cv2.MORPH_RECT, (hk_w, hk_h))
            horiz_all = cv2.bitwise_or(horiz_all, cv2.morphologyEx(m, cv2.MORPH_OPEN, hk))
    m = cv2.subtract(m, horiz_all)

    # ----------------------------
    # 2) Vertical bars (separator/borders) - stronger
    # ----------------------------
    vk_h = max(14, int(0.72 * H))
    v_all = np.zeros_like(m)
    for vk_w in (1, 3, 5, 7):
        vk = cv2.getStructuringElement(cv2.MORPH_RECT, (vk_w, vk_h))
        v = cv2.morphologyEx(m, cv2.MORPH_OPEN, vk)
        # thicken a bit to catch broken bars
        v = cv2.dilate(v, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 7)), iterations=1)
        v_all = cv2.bitwise_or(v_all, v)

    out = m.copy()

    # Content span from projection (padding-safe)
    col = (out > 0).mean(axis=0).astype(np.float32)
    col_s = _smooth_1d(col, max(9, int(0.08 * W)))
    peak = float(col_s.max()) if col_s.size else 0.0

    if peak > 1e-6:
        sig_thr = 0.16 * peak
        sig = np.where(col_s >= sig_thr)[0]
        if len(sig) >= 2:
            L, R = int(sig[0]), int(sig[-1])
        else:
            L, R = 0, W - 1
    else:
        L, R = 0, W - 1

    # 2.a) Remove vertical bars near *content edges*
    edge_margin = max(2, int(0.035 * W))
    left_edge  = min(W, L + edge_margin)
    right_edge = max(0, R - edge_margin)

    out[:, :left_edge]    = cv2.subtract(out[:, :left_edge], v_all[:, :left_edge])
    out[:, right_edge:]   = cv2.subtract(out[:, right_edge:], v_all[:, right_edge:])

    # 2.b) Remove separator by strongest low-density valley between L..R
    if peak > 1e-6 and R > L + 12:
        valley = col_s[L:R+1]
        low_thr = 0.10 * peak
        low = (valley <= low_thr).astype(np.uint8)

        # longest low-run
        best_len = 0
        best_a = best_b = None
        a = None
        for i, v in enumerate(low):
            if v == 1 and a is None:
                a = i
            if (v == 0 or i == len(low)-1) and a is not None:
                b = i if (v == 1 and i == len(low)-1) else i-1
                run_len = b - a + 1
                if run_len > best_len:
                    best_len = run_len
                    best_a, best_b = a, b
                a = None

        if best_a is not None and best_len >= int(0.022 * W):
            gap0 = L + best_a
            gap1 = L + best_b
            margin = max(2, int(0.02 * W))
            g0 = max(0, gap0 - margin)
            g1 = min(W-1, gap1 + margin)
            out[:, g0:g1+1] = cv2.subtract(out[:, g0:g1+1], v_all[:, g0:g1+1])

    # ----------------------------
    # 3) Remove dense reflection/frame rows near top/bottom
    # ----------------------------
    row_den = (out > 0).mean(axis=1)
    band = max(1, int(0.018 * H))

    for r in np.where(row_den > 0.38)[0]:
        if r < int(0.22 * H) or r > int(0.78 * H):
            r0 = max(0, r - band)
            r1 = min(H, r + band + 1)
            out[r0:r1, :] = 0

    return out






def _remove_border_frame_components(mask255, border_frac=0.02):
    """
    Remove connected components that touch the crop border and look like frame/border/separator remnants.
    Safe because Step2 already adds background padding.
    """
    m = (mask255 > 0).astype(np.uint8)
    H, W = m.shape[:2]
    b = max(2, int(border_frac * min(H, W)))

    num, lab, stats, _ = cv2.connectedComponentsWithStats(m, 8)
    out = mask255.copy()

    for i in range(1, num):
        x, y, w, h, area = stats[i]
        if area <= 0:
            continue

        fill = area / float(w * h + 1e-6)
        touch = (x <= b) or (y <= b) or (x + w >= W - b) or (y + h >= H - b)

        if not touch:
            continue

        aspect = max(w / float(h + 1e-6), h / float(w + 1e-6))

        # Frame-like heuristics (thin/long, low fill, border-touching)
        if (
            aspect > 6.0 or
            (fill < 0.22 and (w > 0.18 * W or h > 0.30 * H)) or
            (h > 0.70 * H and w < 0.25 * W) or
            (w > 0.70 * W and h < 0.25 * H)
        ):
            out[lab == i] = 0

    return out




def _binarize_band_to_fgmask(band_bgr):
    """
    Returns fg mask (uint8 0/255) where characters are foreground (255).

    Fixes:
    - Dark gating becomes OTSU(gray) + percentile fallback (less “missing digits”)
    - Removes center separator bar explicitly (fixes extra digit / wrong split)
    """
    gray = cv2.cvtColor(band_bgr, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    g0 = clahe.apply(gray)
    g  = cv2.bilateralFilter(g0, 7, 40, 40)

    H, W = g.shape[:2]

    # adaptive base
    bs = max(21, (int(min(H, W) * 0.18) | 1))
    bw_gray = cv2.adaptiveThreshold(
        g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, bs, 7
    )

    # blackhat helps thin strokes
    kx = max(17, int(0.16 * W))
    ky = max(9,  int(0.22 * H))
    bh = cv2.morphologyEx(g, cv2.MORPH_BLACKHAT,
                          cv2.getStructuringElement(cv2.MORPH_RECT, (kx, ky)))
    _, bw_bh = cv2.threshold(bh, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    bw = cv2.bitwise_or(bw_gray, bw_bh)

    # -------------------------
    # DARK GATING (improved)
    # -------------------------
    # Otsu on grayscale gives a robust dark threshold even when ink is weak.
    t_otsu, _ = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    t_pct = np.percentile(g, 72)  # safer than 62 for low-contrast plates
    t_dark = min(float(t_otsu), float(t_pct)) + 5.0

    dark = (g < t_dark).astype(np.uint8) * 255
    bw = cv2.bitwise_and(bw, dark)

    # remove long borders/lines
    bw = _remove_long_lines(bw)

    # remove the center separator explicitly (KEY FIX)
    bw = _remove_center_separator_bar(bw)

    # keep dots
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN,
                          cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)))
    bw = cv2.morphologyEx(bw, cv2.MORPH_CLOSE,
                          cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)))

    return bw


def _get_components(mask255):
    """
    Connected components on 255-foreground mask.
    Returns list of boxes: (x,y,w,h,area)

    Fix:
    - compute content span L/R ONLY inside central 5%..95% columns
      so edge bars don't become "content".
    """
    m = (mask255 > 0).astype(np.uint8)
    H, W = m.shape[:2]

    col = m.mean(axis=0).astype(np.float32)
    col_s = _smooth_1d(col, max(9, int(0.08 * W)))
    peak = float(col_s.max()) if col_s.size else 0.0

    if peak > 1e-6:
        sig_thr = 0.14 * peak

        s0 = int(0.05 * W)
        s1 = int(0.95 * W)
        core = col_s[s0:s1]
        sig = np.where(core >= sig_thr)[0]
        sig = sig + s0

        if len(sig) >= 2:
            L, R = int(sig[0]), int(sig[-1])
        else:
            L, R = 0, W - 1
    else:
        L, R = 0, W - 1

    pad_guard = max(2, int(0.05 * W))
    Lg = max(0, L - pad_guard)
    Rg = min(W - 1, R + pad_guard)

    num, lab, stats, _ = cv2.connectedComponentsWithStats(m, 8)
    boxes = []
    min_area = max(30, int(0.0006 * H * W))

    for i in range(1, num):
        x, y, w, h, area = stats[i]
        if area < min_area:
            continue

        cx = x + w / 2.0
        if cx < Lg or cx > Rg:
            continue

        fill = area / float(w * h + 1e-6)
        aspect = max(w / float(h + 1e-6), h / float(w + 1e-6))

        # strong bar rejection (but keep digit '١' by requiring low fill OR central position)
        if (h > 0.70 * H) and (w < 0.10 * W) and (aspect > 6.0) and (fill < 0.80):
            continue

        if (w * h) > 0.16 * H * W and fill > 0.62:
            continue

        if (w > 0.55 * W) and (h < 0.14 * H):
            continue

        boxes.append((x, y, w, h, area))

    return boxes





def _merge_dots_and_fragments(boxes, H, W):
    """
    More robust dot/hamza merge:
    - Uses 75th percentile for "typical character" so bars/noise don't shrink medians.
    - Treats small height/width as dot/fragment, not just area.
    """
    if not boxes:
        return boxes

    boxes = list(boxes)

    areas = np.array([b[4] for b in boxes], dtype=np.float32)
    hs    = np.array([b[3] for b in boxes], dtype=np.float32)
    ws    = np.array([b[2] for b in boxes], dtype=np.float32)

    ref_area = float(np.percentile(areas, 75)) if len(areas) else 0.0
    ref_h    = float(np.percentile(hs, 75)) if len(hs) else 0.0
    ref_w    = float(np.percentile(ws, 75)) if len(ws) else 0.0

    dot_area_thr = max(25.0, 0.12 * ref_area)
    dot_h_thr    = max(5.0,  0.32 * ref_h)
    dot_w_thr    = max(4.0,  0.30 * ref_w)

    big, small = [], []
    for b in boxes:
        x, y, w, h, a = b
        if (a <= dot_area_thr) or (h <= dot_h_thr) or (w <= dot_w_thr):
            small.append(b)
        else:
            big.append(b)

    if not big:
        return boxes

    merged = big[:]

    for (sx, sy, sw, sh, sa) in small:
        scx = sx + sw / 2.0
        scy = sy + sh / 2.0

        best_j = None
        best_score = 1e18

        for j, (bx, by, bw, bh, ba) in enumerate(merged):
            bcx = bx + bw / 2.0
            bcy = by + bh / 2.0

            dx = abs(scx - bcx)
            dy = abs(scy - bcy)

            if dx > 1.05 * bw:
                continue
            if dy > 1.45 * bh:
                continue

            overlap = max(0, min(sx+sw, bx+bw) - max(sx, bx))
            if overlap < 0.10 * min(sw, bw) and dx > 0.60 * bw:
                continue

            score = dx + 1.15 * dy
            if score < best_score:
                best_score = score
                best_j = j

        if best_j is None:
            continue

        bx, by, bw, bh, ba = merged[best_j]
        x0 = min(bx, sx)
        y0 = min(by, sy)
        x1 = max(bx + bw, sx + sw)
        y1 = max(by + bh, sy + sh)
        merged[best_j] = (x0, y0, x1 - x0, y1 - y0, ba + sa)

    return merged



def _split_digits_letters_by_gap(boxes, W,
                                 min_digits=DIGIT_MIN, max_digits=DIGIT_MAX,
                                 min_letters=LETTER_MIN, max_letters=LETTER_MAX):
    """
    Choose split by largest x-gap BUT only accept splits that satisfy constraints.
    If no valid split exists, DO NOT return an invalid split; return ([],[]) and let caller use separator split.
    """
    if len(boxes) <= 1:
        return [], []

    boxes = sorted(boxes, key=lambda b: b[0] + b[2]/2.0)
    xs = [b[0] + b[2]/2.0 for b in boxes]
    gaps = [xs[i+1] - xs[i] for i in range(len(xs)-1)]
    if not gaps:
        return [], []

    cand = sorted(range(len(gaps)), key=lambda i: gaps[i], reverse=True)

    def ok(n_digits, n_letters):
        return (min_digits <= n_digits <= max_digits) and (min_letters <= n_letters <= max_letters)

    for i in cand:
        left = boxes[:i+1]
        right = boxes[i+1:]
        # digits are LEFT, letters are RIGHT
        if ok(len(left), len(right)):
            return left, right

    # no valid split
    return [], []



def _cleanup_char_mask_crop(mask255_crop):
    """
    Remove isolated specks inside a single character crop while keeping dots/hamza near the main glyph.
    Input: 255=fg mask.
    Output: 255=fg mask (cleaned).
    """
    m = (mask255_crop > 0).astype(np.uint8)
    H, W = m.shape[:2]
    if H < 5 or W < 5:
        return mask255_crop

    num, lab, stats, _ = cv2.connectedComponentsWithStats(m, 8)
    if num <= 2:
        return mask255_crop

    # largest component (ignore bg=0)
    areas = stats[1:, cv2.CC_STAT_AREA].astype(np.int32)
    li = 1 + int(np.argmax(areas))
    lx, ly, lw, lh, la = stats[li]
    l_area = float(la)

    keep = np.zeros_like(m, dtype=np.uint8)
    keep[lab == li] = 1

    for i in range(1, num):
        if i == li:
            continue
        x, y, w, h, a = stats[i]
        a = float(a)
        if a < 12:
            continue

        # keep big parts
        if a >= max(18.0, 0.06 * l_area):
            keep[lab == i] = 1
            continue

        # keep dot-like parts close to the main glyph
        overlap = max(0, min(x+w, lx+lw) - max(x, lx))
        if overlap >= 0.30 * min(w, lw):
            cy = y + h/2.0
            main_cy = ly + lh/2.0
            if abs(cy - main_cy) <= 1.20 * lh:
                keep[lab == i] = 1

    return (keep * 255).astype(np.uint8)





def _crop_char_from_mask(mask255, x, y, w, h, pad_frac=0.18):
    """
    mask255: 255=fg
    Return 150x150 uint8, WHITE on BLACK (after the caller inverts if needed).
    """
    H, W = mask255.shape[:2]
    pad = int(pad_frac * max(w, h))
    x0 = max(0, x - pad)
    y0 = max(0, y - pad)
    x1 = min(W, x + w + pad)
    y1 = min(H, y + h + pad)

    crop = mask255[y0:y1, x0:x1].copy()
    crop = _cleanup_char_mask_crop(crop)  # <-- key

    # black-on-white inside the crop function (same behavior you used)
    crop = 255 - crop

    crop = _pad_to_square_u8(crop, pad_value=255)
    crop = _resize_150(crop)
    return crop





def _split_wide_components(boxes, mask255, widen_factor=1.22, min_w_frac=0.10, valley_rel=0.55):
    """
    Split abnormally-wide components (often 2 chars stuck together) using column projection valley.

    Improvements:
    - widen_factor is lower (more likely to trigger on cases like 0013, 0033)
    - valley condition is more permissive (valley_rel)
    """
    if not boxes:
        return boxes

    H, W = mask255.shape[:2]
    ws = np.array([b[2] for b in boxes], dtype=np.float32)
    med_w = float(np.median(ws)) if len(ws) else 0.0
    if med_w <= 1:
        return boxes

    out = []
    for (x, y, w, h, area) in boxes:
        if (w > widen_factor * med_w) and (w > min_w_frac * W):
            crop = (mask255[y:y+h, x:x+w] > 0).astype(np.uint8)
            if crop.size == 0:
                out.append((x, y, w, h, area))
                continue

            proj = crop.mean(axis=0).astype(np.float32)
            proj_s = _smooth_1d(proj, max(7, int(0.10 * w)))
            pmax = float(proj_s.max()) if proj_s.size else 0.0
            if pmax < 1e-6:
                out.append((x, y, w, h, area))
                continue

            a = int(0.15 * w)
            b = int(0.85 * w)
            if b <= a + 2:
                out.append((x, y, w, h, area))
                continue

            valley_i = int(np.argmin(proj_s[a:b]) + a)
            if proj_s[valley_i] > valley_rel * pmax:
                out.append((x, y, w, h, area))
                continue

            left = crop[:, :valley_i]
            right = crop[:, valley_i:]

            def tight_bbox(binary):
                ys, xs = np.where(binary > 0)
                if len(xs) == 0:
                    return None
                return xs.min(), ys.min(), xs.max(), ys.max()

            bbL = tight_bbox(left)
            bbR = tight_bbox(right)
            if bbL is None or bbR is None:
                out.append((x, y, w, h, area))
                continue

            lx0, ly0, lx1, ly1 = bbL
            rx0, ry0, rx1, ry1 = bbR
            rx0 += valley_i
            rx1 += valley_i

            out.append((x + lx0, y + ly0, (lx1 - lx0 + 1), (ly1 - ly0 + 1), int(left.sum())))
            out.append((x + rx0, y + ry0, (rx1 - rx0 + 1), (ry1 - ry0 + 1), int(right.sum())))
        else:
            out.append((x, y, w, h, area))

    return out



def _remove_separator_in_largest_gap(boxes, H, W):
    """
    After components are found, remove a tall-thin box that sits inside the largest gap between boxes.
    This fixes cases like 0043 where the separator survives earlier removal.
    """
    if len(boxes) < 3:
        return boxes

    boxes_s = sorted(boxes, key=lambda b: b[0] + b[2]/2.0)
    xs = [b[0] + b[2]/2.0 for b in boxes_s]
    gaps = [xs[i+1] - xs[i] for i in range(len(xs)-1)]
    if not gaps:
        return boxes

    i = int(np.argmax(gaps))
    left = boxes_s[i]
    right = boxes_s[i+1]

    gap_x0 = left[0] + left[2]
    gap_x1 = right[0]

    kept = []
    for (x, y, w, h, area) in boxes_s:
        cx = x + w/2.0
        # candidate separator: tall & thin and located in the big gap interval
        if (gap_x0 <= cx <= gap_x1) and (h > 0.55 * H) and (w < 0.10 * W):
            continue
        kept.append((x, y, w, h, area))
    return kept



def _find_separator_x(mask255):
    """
    Find the digit/letter separator x using:
    1) vertical-bar detection (morph open)
    2) fallback: deepest valley in column projection
    Returns int x or None.
    """
    m = (mask255 > 0).astype(np.uint8) * 255
    H, W = m.shape[:2]
    if H < 10 or W < 10:
        return None

    # (1) vertical bar detection
    vk_h = max(12, int(0.65 * H))
    vk_w = max(1, int(0.015 * W))  # ~1.5% of width
    vk = cv2.getStructuringElement(cv2.MORPH_RECT, (vk_w, vk_h))
    vbar = cv2.morphologyEx(m, cv2.MORPH_OPEN, vk)
    vbar = cv2.dilate(vbar, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 7)), iterations=1)

    num, lab, stats, _ = cv2.connectedComponentsWithStats((vbar > 0).astype(np.uint8), 8)
    best = None
    for i in range(1, num):
        x, y, w, h, area = stats[i]
        cx = x + w / 2.0
        if cx < 0.20 * W or cx > 0.80 * W:
            continue
        if h < 0.45 * H:
            continue
        if w > 0.10 * W:
            continue
        score = h * area
        if best is None or score > best[0]:
            best = (score, int(cx))

    if best is not None:
        return best[1]

    # (2) fallback: valley in column projection
    col = (m > 0).mean(axis=0).astype(np.float32)
    col_s = _smooth_1d(col, max(9, int(0.08 * W)))
    peak = float(col_s.max()) if col_s.size else 0.0
    if peak < 1e-6:
        return None

    a = int(0.20 * W)
    b = int(0.80 * W)
    seg = col_s[a:b]
    if seg.size < 5:
        return None

    x_min = int(np.argmin(seg) + a)
    if col_s[x_min] < 0.18 * peak:
        # ensure ink exists on both sides
        if col_s[:x_min].mean() > 0.08 * peak and col_s[x_min:].mean() > 0.08 * peak:
            return x_min

    return None


def _split_digits_letters_by_separator(boxes, mask255,
                                       min_digits=DIGIT_MIN, max_digits=DIGIT_MAX,
                                       min_letters=LETTER_MIN, max_letters=LETTER_MAX):
    """
    Split by detected separator x. Drop any box that straddles separator.
    If separator not found, fallback to _split_digits_letters_by_gap (which now returns [] if invalid).
    """
    if not boxes:
        return [], []

    H, W = mask255.shape[:2]
    split_x = _find_separator_x(mask255)

    boxes_s = sorted(boxes, key=lambda b: b[0] + b[2]/2.0)

    # Fallback to gap split (ONLY if valid)
    if split_x is None:
        d, l = _split_digits_letters_by_gap(boxes_s, W, min_digits, max_digits, min_letters, max_letters)
        return d, l

    digits, letters = [], []
    for (x, y, w, h, area) in boxes_s:
        cx = x + w / 2.0

        # drop separator remnants: box crosses separator column
        if x <= split_x <= (x + w):
            continue

        if cx < split_x:
            digits.append((x, y, w, h, area))
        else:
            letters.append((x, y, w, h, area))

    def ok(nd, nl):
        return (min_digits <= nd <= max_digits) and (min_letters <= nl <= max_letters)

    # If counts are too large, first remove "most bar-like" / tiny fragments per side.
    def prune_side(side_boxes):
        if not side_boxes:
            return side_boxes
        # prefer keeping real chars: sort by (height, bbox area) not raw fg area (keeps '١')
        side_boxes = sorted(side_boxes, key=lambda b: (b[3], b[2]*b[3]), reverse=True)
        return side_boxes

    # prune if absurdly large
    if len(digits) > max_digits:
        digits = prune_side(digits)[:max_digits]
        digits = sorted(digits, key=lambda b: b[0] + b[2]/2.0)

    if len(letters) > max_letters:
        letters = prune_side(letters)[:max_letters]
        letters = sorted(letters, key=lambda b: b[0] + b[2]/2.0)

    # If still invalid, try moving nearest-to-separator boxes
    digits = sorted(digits, key=lambda b: b[0] + b[2]/2.0)
    letters = sorted(letters, key=lambda b: b[0] + b[2]/2.0)

    while len(digits) > max_digits and len(letters) < max_letters:
        letters.insert(0, digits.pop())  # move rightmost digit -> letters

    while len(letters) > max_letters and len(digits) < max_digits:
        digits.append(letters.pop(0))    # move leftmost letter -> digits

    if not ok(len(digits), len(letters)):
        # final fallback: try valid gap split; otherwise return what we have
        d2, l2 = _split_digits_letters_by_gap(boxes_s, W, min_digits, max_digits, min_letters, max_letters)
        if d2 and l2:
            return d2, l2

    return digits, letters






def segment_step3_v1(textband_bgr):
    """
    Input: Step2 output (BGR).
    Output:
      digits_imgs (150x150 u8)  : WHITE on BLACK (L->R)
      letters_imgs (150x150 u8) : WHITE on BLACK (R->L)
      dbg_band_rgb (RGB)        : boxes overlay on Step2 band (for saving)
      dbg_mask_rgb (RGB)        : boxes overlay on B/W mask (for printing)
    """
    H, W = textband_bgr.shape[:2]

    fg2 = _binarize_band_to_fgmask(textband_bgr)  # already includes _remove_long_lines

    
    # ---- OPTIONAL: break tiny horizontal bridges between adjacent chars ----
    # (safe for thin vertical strokes like ١ because kernel is (2,1))
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
    fg_sep = cv2.dilate(cv2.erode(fg2, k, iterations=1), k, iterations=1)

    # choose the version that gives a more realistic component count
    n_base = len(_get_components(fg2))
    n_sep  = len(_get_components(fg_sep))

    def score(n):
        if n < 2:  return 10**9
        if n > 10: return 10**8 + n
        return abs(n - 6)  # target around 5~7 total components

    if score(n_sep) < score(n_base):
        fg2 = fg_sep



    # NEW: remove border-touching frame remnants that still survive
    fg2 = _remove_border_frame_components(fg2, border_frac=0.02)

    # light cleanup (keep dots)
    fg2 = cv2.morphologyEx(fg2, cv2.MORPH_OPEN,  cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)))
    fg2 = cv2.morphologyEx(fg2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)))


    boxes = _get_components(fg2)
    boxes = _merge_dots_and_fragments(boxes, H, W)

    # kill separator remnants if any survived
    boxes = _remove_separator_in_largest_gap(boxes, H, W)

    digit_boxes, letter_boxes = _split_digits_letters_by_separator(boxes, fg2)

    # NOW split wide blobs PER SIDE (fixes 0013, 0033)
    digit_boxes  = _split_wide_components(digit_boxes,  fg2, widen_factor=1.18, min_w_frac=0.09, valley_rel=0.60)
    letter_boxes = _split_wide_components(letter_boxes, fg2, widen_factor=1.22, min_w_frac=0.10, valley_rel=0.60)

    # optional: merge dots again after splitting (helps letters like ق/ن/ي)
    digit_boxes  = _merge_dots_and_fragments(digit_boxes,  H, W)
    letter_boxes = _merge_dots_and_fragments(letter_boxes, H, W)


    '''
    # height-based cleanup (remove tiny fragments that survive)
    if boxes:
        hs = np.array([b[3] for b in boxes], dtype=np.float32)
        med_h = float(np.median(hs))
        keep = []
        for b in boxes:
            x, y, w, h, a = b
            if h < 0.35 * med_h:
                continue
            keep.append(b)
        if len(keep) >= 2:
            boxes = keep

    # split digits/letters by gap + constraints
    digit_boxes, letter_boxes = _split_digits_letters_by_separator(boxes, fg2)
    '''

    
    # ---- crops ----
    # _crop_char_from_mask returns BLACK on WHITE, so invert -> WHITE on BLACK
    digits_imgs  = [_crop_char_from_mask(fg2, *b[:4]) for b in sorted(digit_boxes,  key=lambda b: b[0])]
    digits_imgs  = [255 - im for im in digits_imgs]  # -> WHITE on BLACK

    letters_imgs = [_crop_char_from_mask(fg2, *b[:4]) for b in sorted(letter_boxes, key=lambda b: -b[0])]  # R->L
    letters_imgs = [255 - im for im in letters_imgs]  # -> WHITE on BLACK

    # ---- debug overlays ----
    dbg_band = cv2.cvtColor(textband_bgr, cv2.COLOR_BGR2RGB)

    # mask visualization: WHITE fg on BLACK
    dbg_mask = cv2.cvtColor(fg2, cv2.COLOR_GRAY2RGB)

    for b in digit_boxes:
        x, y, w, h, _ = b
        cv2.rectangle(dbg_band, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.rectangle(dbg_mask, (x, y), (x+w, y+h), (0, 255, 0), 2)

    for b in letter_boxes:
        x, y, w, h, _ = b
        cv2.rectangle(dbg_band, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.rectangle(dbg_mask, (x, y), (x+w, y+h), (255, 0, 0), 2)

    return digits_imgs, letters_imgs, dbg_band, dbg_mask





# ============================================================
# RUN (Step1 + Step2)
# ============================================================
plates = load_plate_images(INPUT_FOLDER)
ensure_dir(OUTPUT_ROOT)

for i, item in enumerate(plates, start=1):
    bgr  = item["bgr"]
    stem = item["stem"]
    name = f"[{i}] {item['name']}"

    body_bgr, dbg1_rgb = cut_header_step1_v2(bgr)


    band_bgr, dbg2_rgb = auto_text_band_step2_v4(body_bgr)


    orig_rgb = bgr_to_rgb(bgr)
    body_rgb = bgr_to_rgb(body_bgr)
    band_rgb = bgr_to_rgb(band_bgr)

    # --------------------------
    # STEP 3 (segment + split)
    # --------------------------
    digits_150, letters_150, dbg3_band_rgb, dbg3_mask_rgb = segment_step3_v1(band_bgr)

    if (MAX_SHOW is None) or (i <= MAX_SHOW):

        # -------- LINE 1 --------
        show_images(
            [orig_rgb, body_rgb, band_rgb, dbg3_mask_rgb],
            [f"{name} | original",
            f"{name} | after header cut",
            f"{name} | Step2 band",
            f"{name} | Step3 MASK boxes (digits=green, letters=red)"]
        )

        # -------- LINE 2 --------
        line2_imgs   = digits_150 + letters_150
        line2_titles = (
            [f"{name} | digit {k+1}" for k in range(len(digits_150))] +
            [f"{name} | letter {k+1} (R->L)" for k in range(len(letters_150))]
        )
        if len(line2_imgs) > 0:
            show_images(line2_imgs, line2_titles)



    '''#no need for save now
    out_dir = ensure_dir(os.path.join(OUTPUT_ROOT, stem))


    
    # Step1 saves
    save_rgb(os.path.join(out_dir, "step1_01_original.png"), orig_rgb)
    save_rgb(os.path.join(out_dir, "step1_02_after_header_cut.png"), body_rgb)
    if SAVE_DEBUG:
        save_rgb(os.path.join(out_dir, "step1_03_header_debug.png"), dbg1_rgb)

    # Step2 saves
    save_rgb(os.path.join(out_dir, "step2_01_text_band_v4.png"), band_rgb)

    # Step3 saves
    save_rgb(os.path.join(out_dir, "step3_01_boxes_overlay.png"), dbg3_band_rgb)


    # Save digits / letters as 150x150 grayscale PNGs
    for k, ch in enumerate(digits_150, start=1):
        cv2.imwrite(os.path.join(out_dir, f"step3_digit_{k:02d}.png"), ch)

    for k, ch in enumerate(letters_150, start=1):
        cv2.imwrite(os.path.join(out_dir, f"step3_letter_{k:02d}.png"), ch)



    if SAVE_DEBUG and (dbg2_rgb is not None):
        save_rgb(os.path.join(out_dir, "step2_02_textband_debug_v4.png"), dbg2_rgb)

    '''#no need for save now

print(f"[OK] Done. Saved under: {OUTPUT_ROOT}/<stem>/")

'''
MAAW STEPS
'''


In [ ]:
'''
MAAW ANGER
'''

# Robust Egyptian License Plate Arabic Digit + Letter Segmentation (ONE CELL)
# Studied reference patterns in: :contentReference[oaicite:0]{index=0}

import os, glob
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.color import rgb2gray, rgba2rgb, gray2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.measure import label, regionprops

import cv2

# ============================================================
# Helpers
# ============================================================
def _to_u8(x):
    if x.dtype == np.uint8: return x
    x = np.clip(x, 0, 1) if x.max() <= 1.01 else np.clip(x, 0, 255)
    if x.max() <= 1.01: x = (x * 255.0)
    return x.astype(np.uint8)

def _ensure_rgb(img):
    if img.ndim == 2:
        return gray2rgb(img)
    if img.shape[-1] == 4:
        img = rgba2rgb(img)
        img = _to_u8(img)
    if img.dtype != np.uint8:
        img = _to_u8(img)
    return img

def _read_img(path):
    img = imread(path)
    img = _ensure_rgb(img)
    return img

def _find_input_dir(preferred="maaw final crops"):
    # Prefer exact folder name, else fuzzy
    if os.path.isdir(preferred):
        return preferred
    for root, dirs, _files in os.walk("."):
        for d in dirs:
            if d.strip().lower() == preferred.strip().lower():
                return os.path.join(root, d)
    # fuzzy
    target = preferred.lower().replace(" ", "")
    for root, dirs, _files in os.walk("."):
        for d in dirs:
            if target in d.lower().replace(" ", ""):
                return os.path.join(root, d)
    raise FileNotFoundError(f"Could not find folder named '{preferred}' (or close match).")

def _blue_header_cut(img_rgb):
    """
    Find the blue header band by row-wise blue-pixel density, then cut below it.
    Robust to partial crops and varying header heights.
    """
    H, W = img_rgb.shape[:2]
    topH = max(10, int(0.55 * H))  # search mostly in upper part
    r = img_rgb[:topH, :, 0].astype(np.int16)
    g = img_rgb[:topH, :, 1].astype(np.int16)
    b = img_rgb[:topH, :, 2].astype(np.int16)

    # Blue score: blue much larger than red/green (works for "EGYPT" header blues)
    blue_mask = (b > 70) & ((b - np.maximum(r, g)) > 25)

    row_density = blue_mask.mean(axis=1)  # fraction of blue pixels per row
    if row_density.max() < 0.05:
        return img_rgb, 0  # no obvious blue header

    # Smooth densities
    k = max(5, int(0.02 * topH))
    kernel = np.ones(k, dtype=np.float32) / k
    sm = np.convolve(row_density, kernel, mode="same")

    peak = sm.max()
    thr = max(0.06, 0.45 * peak)
    rows = np.where(sm >= thr)[0]
    if len(rows) == 0:
        return img_rgb, 0

    # Take the last strong-blue row as header bottom; add a small margin
    y_cut = int(min(H - 1, rows.max() + max(3, int(0.01 * H))))
    out = img_rgb[y_cut:, :, :]
    return out, y_cut

def _enhance_for_text(gray_u8):
    """
    Stroke-preserving enhancement:
    - CLAHE for contrast (handles shadows)
    - BLACKHAT to highlight dark strokes on bright background
    - Combine for robust thresholding of thin strokes (١, dots, hamza)
    """
    clahe = cv2.createCLAHE(clipLimit=2.2, tileGridSize=(8, 8)).apply(gray_u8)

    # Blackhat kernel tuned for Arabic strokes (a bit wider horizontally)
    kh = max(9, (gray_u8.shape[0] // 20) | 1)
    kw = max(13, (gray_u8.shape[1] // 18) | 1)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (kw, kh))
    blackhat = cv2.morphologyEx(clahe, cv2.MORPH_BLACKHAT, k)

    # Combine: characters become bright; background stays moderate
    enhanced = cv2.addWeighted(clahe, 0.9, blackhat, 1.6, 0)
    enhanced = cv2.GaussianBlur(enhanced, (3, 3), 0)
    return enhanced, clahe, blackhat

def _auto_text_band(img_rgb_cut):
    """
    Auto-crop digits+letters band using blackhat projection AFTER suppressing long horizontal lines.
    This fixes cases where the strongest signal is the bumper/border line.
    """
    H, W = img_rgb_cut.shape[:2]
    gray = _to_u8(rgb2gray(img_rgb_cut))
    enh_full, clahe_full, bh_full = _enhance_for_text(gray)

    # --- suppress long horizontal lines in blackhat ---
    # wide horizontal opening extracts long lines; subtract them out
    kw = max(31, (W // 3) | 1)
    kh = 1
    hk = cv2.getStructuringElement(cv2.MORPH_RECT, (kw, kh))
    horiz = cv2.morphologyEx(bh_full, cv2.MORPH_OPEN, hk, iterations=1)
    bh = cv2.subtract(bh_full, horiz)

    row_energy = bh.mean(axis=1).astype(np.float32)

    # smooth
    k = max(9, int(0.06 * H))
    kernel = np.ones(k, dtype=np.float32) / k
    sm = np.convolve(row_energy, kernel, mode="same")

    peak = float(sm.max())
    if peak < 1.5:
        # fallback: prefer middle-lower but avoid extreme bottom (bumper)
        y0 = int(0.18 * H)
        y1 = int(0.88 * H)
        band = img_rgb_cut[y0:y1, :]
        return band, enh_full[y0:y1, :], gray[y0:y1, :], (y0, y1)

    y_peak = int(np.argmax(sm))

    # build candidates around the peak and choose the one that yields plausible component count
    cand_half = max(10, int(0.18 * H))
    candidates = []
    for scale in [0.70, 0.55, 0.45]:
        thr = scale * peak
        above = np.where(sm >= thr)[0]
        if len(above) == 0:
            y0 = max(0, y_peak - cand_half)
            y1 = min(H, y_peak + cand_half)
        else:
            # choose run containing peak if possible
            runs = []
            start = above[0]; prev = above[0]
            for y in above[1:]:
                if y == prev + 1:
                    prev = y
                else:
                    runs.append((start, prev))
                    start = y; prev = y
            runs.append((start, prev))
            chosen = None
            for a, b in runs:
                if a <= y_peak <= b:
                    chosen = (a, b); break
            if chosen is None:
                chosen = max(runs, key=lambda t: (t[1]-t[0]))
            y0, y1 = chosen
            margin = max(8, int(0.10 * (y1 - y0 + 1)))
            y0 = max(0, y0 - margin)
            y1 = min(H, y1 + margin)

        # avoid bottom bumper zone aggressively
        y1 = min(y1, int(0.92 * H))
        if (y1 - y0) < int(0.22 * H):
            pad = int(0.12 * H)
            y0 = max(0, y0 - pad)
            y1 = min(int(0.92 * H), y1 + pad)

        # quick plausibility check: how many CCs after light binarize?
        band_gray = gray[y0:y1, :]
        band_enh, band_clahe, band_bh = _enhance_for_text(_to_u8(band_gray))
        bw = _binarize_stroke_preserving(band_enh, clahe_u8=band_clahe)
        _closed, clean = _morph_clean(bw)
        boxes, _ = _boxes_from_mask(clean)
        # prefer 2..9 components (digits+letters+noise)
        # --- improved scoring: penalize "single strong line" bands ---
        fg_sum = int(clean.sum())
        row_max = int(clean.sum(axis=1).max()) if fg_sum > 0 else 0
        line_ratio = (row_max / max(1, fg_sum))  # if ~0.3-0.8 => dominated by 1 line

        ncc = len(boxes)

        score = 0
        # prefer plausible CC count
        if 3 <= ncc <= 8: score += 80
        score += min(50, ncc * 8)

        # penalize bumper/border-dominated masks
        if line_ratio > 0.18:
            score -= 120 * (line_ratio / 0.18)

        # prefer mid-lower band height (not ultra-thin)
        score -= abs((y1 - y0) - int(0.45 * H)) * 0.05


        candidates.append((score, y0, y1))

    # choose best
    candidates.sort(reverse=True, key=lambda t: t[0])
    _, y0, y1 = candidates[0]

    band = img_rgb_cut[y0:y1, :, :]
    return band, enh_full[y0:y1, :], gray[y0:y1, :], (y0, y1)


def _binarize_stroke_preserving(enh_u8, clahe_u8=None, blackhat_u8=None):
    """
    Robust binarization for Egyptian plates in your samples:
    - Tests BOTH polarities:
        A) BLACKHAT Otsu   (dark ink on bright plate)
        B) TOPHAT  Otsu    (bright ink on dark/dirty plate or glare inversions)
        C) Adaptive on inverted CLAHE (dark ink)
        D) Adaptive on CLAHE          (bright ink)
    - Chooses best by a "not empty / not flood-filled / reasonable CCs / not blocky" score.
    Returns boolean mask (True = foreground strokes).
    """
    if clahe_u8 is None:
        clahe_u8 = enh_u8
    if blackhat_u8 is None:
        blackhat_u8 = enh_u8

    H, W = enh_u8.shape[:2]

    # ---- TOPHAT (bright strokes on dark) using same scale as your blackhat ----
    kh = max(9, (H // 20) | 1)
    kw = max(13, (W // 18) | 1)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (kw, kh))
    tophat_u8 = cv2.morphologyEx(clahe_u8, cv2.MORPH_TOPHAT, k)

    # ---- Cand A: Otsu on BLACKHAT ----
    bh_f = blackhat_u8.astype(np.float32) / 255.0
    try:
        t_bh = threshold_otsu(bh_f)
    except Exception:
        t_bh = float(np.median(bh_f))
    bw_bh = (bh_f > t_bh)

    # ---- Cand B: Otsu on TOPHAT ----
    th_f = tophat_u8.astype(np.float32) / 255.0
    try:
        t_th = threshold_otsu(th_f)
    except Exception:
        t_th = float(np.median(th_f))
    bw_th = (th_f > t_th)

    # ---- Cand C: Adaptive on inverted CLAHE (dark ink -> white) ----
    inv = 255 - clahe_u8
    block = max(21, (min(H, W) // 10) | 1)
    C = 6
    bw_ad_dark = cv2.adaptiveThreshold(inv, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, block, C) > 0

    # ---- Cand D: Adaptive on CLAHE (bright ink) ----
    bw_ad_bright = cv2.adaptiveThreshold(clahe_u8, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, block, C) > 0

    # ---- Score: avoid flood-filled rectangles + prefer reasonable CC count ----
    def _score(m):
        if m.sum() == 0:
            return -1e18

        fg = m.mean()
        # flood-fill / empty penalties
        if fg > 0.25:
            return -1e18 + (0.25 - fg) * 1e7
        if fg < 0.002:
            return -1e18 + fg * 1e7
        


        # Extra flood-fill penalty: if ONE component dominates, it's usually background fill
        regs2 = regionprops(label(m))
        if len(regs2) > 0:
            max_area = max(r.area for r in regs2)
            if max_area > 0.28 * H * W:
                return -1e18






        regs = regionprops(label(m))
        good = 0
        blocky = 0
        for r in regs:
            y0, x0, y1, x1 = r.bbox
            h = y1 - y0
            w = x1 - x0
            area = r.area

            if area < max(10, int(0.00008 * H * W)):
                continue

            # plausible character-ish
            if h >= 0.14 * H and (0.004 * W <= w <= 0.70 * W):
                good += 1

            # very filled rectangle look
            if r.extent > 0.78 and getattr(r, "solidity", 1.0) > 0.92 and area > 0.01 * H * W:
                blocky += 1

        # prefer ~4..8 components, penalize blocky blobs
        return (-abs(good - 6) * 35) + (good * 12) - (blocky * 120)

    cands = [
        (bw_bh, "bh"),
        (bw_th, "th"),
        (bw_ad_dark, "ad_dark"),
        (bw_ad_bright, "ad_bright"),
    ]
    best = max(cands, key=lambda t: _score(t[0]))[0]
    return best



def _remove_stripe_artifacts(mask_bool):
    """
    Stronger removal of:
    - wide-thin horizontal bars (bumper/frame) EVEN if not touching borders
    - tall-thin vertical frame strips touching left/right
    - central separator-like tall thin bars
    While trying NOT to kill Arabic digit '١' by only removing vertical strips
    when they are edge-attached or separator-central.
    """
    H, W = mask_bool.shape[:2]
    out = mask_bool.copy()
    lab = label(out)

    for rp in regionprops(lab):
        y0, x0, y1, x1 = rp.bbox
        h = y1 - y0
        w = x1 - x0
        area = rp.area
        extent = rp.extent
        solidity = getattr(rp, "solidity", 1.0)

        touches_top = (y0 <= 1)
        touches_bot = (y1 >= H - 2)
        touches_left = (x0 <= 1)
        touches_right = (x1 >= W - 2)

        cx = 0.5 * (x0 + x1)

        # --- horizontal stripe anywhere (bumper/frame/baseline) ---
        if (h <= max(3, int(0.22 * H)) and
            w >= int(0.55 * W) and
            area >= int(0.0015 * H * W) and
            extent > 0.10):
            out[lab == rp.label] = 0
            continue

        # --- vertical edge frame strips (safe: only if edge-attached) ---
        if ((touches_left or touches_right) and
            w <= max(3, int(0.06 * W)) and
            h >= int(0.55 * H) and
            area >= int(0.0015 * H * W) and
            extent > 0.15):
            out[lab == rp.label] = 0
            continue

        # --- central separator-like bar (safe: only near center region) ---
        if (w <= max(3, int(0.05 * W)) and
            h >= int(0.70 * H) and
            (0.35 * W <= cx <= 0.65 * W) and
            solidity > 0.70 and
            extent > 0.18 and
            area >= int(0.0010 * H * W)):
            out[lab == rp.label] = 0
            continue

    # Kill any extremely dense rows/cols (borders)
    row_frac = out.mean(axis=1)
    for r in np.where(row_frac > 0.60)[0]:
        out[r, :] = 0

    col_frac = out.mean(axis=0)
    for c in np.where(col_frac > 0.60)[0]:
        out[:, c] = 0

    return out







def _morph_clean(bw):
    """
    Safer cleaning:
    - NEVER uses closing result as the final segmentation mask.
    - Closing is ONLY for display ("Binary after closing").
    - Removes long horizontal AND long vertical line artifacts before CC boxing.
    """
    H, W = bw.shape[:2]
    bw_u8 = (bw.astype(np.uint8) * 255)

    # ---- DISPLAY ONLY ----
    k_disp = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
    bw_closed_vis = cv2.morphologyEx(bw_u8, cv2.MORPH_CLOSE, k_disp, iterations=1)

    def _remove_long_lines_u8(mask_u8):
        # remove long horizontal bars (try 2 widths)
        hk1 = cv2.getStructuringElement(cv2.MORPH_RECT, (max(25, W // 2), 1))
        hk2 = cv2.getStructuringElement(cv2.MORPH_RECT, (max(25, W // 3), 1))
        horiz1 = cv2.morphologyEx(mask_u8, cv2.MORPH_OPEN, hk1, iterations=1)
        horiz2 = cv2.morphologyEx(mask_u8, cv2.MORPH_OPEN, hk2, iterations=1)
        mask_u8 = cv2.subtract(mask_u8, cv2.max(horiz1, horiz2))

        # remove long vertical edge strips (try 2 heights)
        vk1 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, max(25, H // 2)))
        vk2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, max(25, H // 3)))
        vert1 = cv2.morphologyEx(mask_u8, cv2.MORPH_OPEN, vk1, iterations=1)
        vert2 = cv2.morphologyEx(mask_u8, cv2.MORPH_OPEN, vk2, iterations=1)
        mask_u8 = cv2.subtract(mask_u8, cv2.max(vert1, vert2))

        return mask_u8

    def _remove_border_frames(mask_bool):
        # stronger border-attached removal (keeps interior dots)
        lab = label(mask_bool)
        out = mask_bool.copy()

        for rp in regionprops(lab):
            y0, x0, y1, x1 = rp.bbox
            h = y1 - y0
            w = x1 - x0
            area = rp.area

            touches_top = (y0 <= 1)
            touches_bot = (y1 >= H - 1)
            touches_left = (x0 <= 1)
            touches_right = (x1 >= W - 1)
            touches = touches_top or touches_bot or touches_left or touches_right
            if not touches:
                continue

            span_w = w / max(1, W)
            span_h = h / max(1, H)
            extent = rp.extent
            solidity = getattr(rp, "solidity", 1.0)

            # side strips
            if (touches_left or touches_right) and span_h > 0.50 and span_w < 0.12 and area > 0.0025 * H * W:
                out[lab == rp.label] = 0
                continue

            # top/bottom strips
            if (touches_top or touches_bot) and span_w > 0.55 and span_h < 0.14 and area > 0.0025 * H * W:
                out[lab == rp.label] = 0
                continue

            # big border-attached blocks
            if area > 0.18 * H * W and extent > 0.60 and solidity > 0.80:
                out[lab == rp.label] = 0
                continue

        return out

    # ---- build FINAL clean mask (NO closing) ----
    u8 = _remove_long_lines_u8(bw_u8.copy())
    m = (u8 > 0)

    m = _remove_border_frames(m)

    # remove only tiny speckles (keep dots)
    min_obj = max(3, int(0.00001 * H * W))
    m = remove_small_objects(m, min_size=min_obj)

    # fill only very small holes
    max_hole = max(8, int(0.00010 * H * W))
    m = remove_small_holes(m, area_threshold=max_hole)

    # final stripe killer (wide horizontals + edge strips + separator-ish)
    m = _remove_stripe_artifacts(m)

    return bw_closed_vis, m







def _boxes_from_mask(mask):
    H, W = mask.shape[:2]
    lab = label(mask)
    regs = regionprops(lab)

    boxes = []
    for rp in regs:
        y0, x0, y1, x1 = rp.bbox
        h = y1 - y0
        w = x1 - x0
        area = rp.area

        # Basic geometric filtering (keeps dots; we'll merge later)
        if area < max(10, int(0.00010 * H * W)):
            continue
        if h < max(6, int(0.08 * H)) and w < max(6, int(0.08 * W)):
            # tiny tiny speck -> drop
            continue
        if h > 0.98 * H and w > 0.70 * W:
            continue
        boxes.append([x0, y0, x1, y1, area])
    return boxes, lab

def _nms_merge_boxes(boxes, x_overlap_thr=0.6, y_overlap_thr=0.6):
    if not boxes:
        return []
    boxes = sorted(boxes, key=lambda b: (b[0], b[1]))
    merged = []
    for b in boxes:
        x0,y0,x1,y1,_ = b
        placed = False
        for m in merged:
            mx0,my0,mx1,my1,_ = m
            # overlap ratios
            ix0, iy0 = max(x0, mx0), max(y0, my0)
            ix1, iy1 = min(x1, mx1), min(y1, my1)
            inter_w = max(0, ix1 - ix0)
            inter_h = max(0, iy1 - iy0)
            if inter_w == 0 or inter_h == 0:
                continue
            w0 = x1 - x0; mw = mx1 - mx0
            h0 = y1 - y0; mh = my1 - my0
            ox = inter_w / max(1, min(w0, mw))
            oy = inter_h / max(1, min(h0, mh))
            if ox >= x_overlap_thr and oy >= y_overlap_thr:
                # merge
                m[0] = min(mx0, x0)
                m[1] = min(my0, y0)
                m[2] = max(mx1, x1)
                m[3] = max(my1, y1)
                m[4] = m[4] + b[4]
                placed = True
                break
        if not placed:
            merged.append([x0,y0,x1,y1,b[4]])
    return merged

def _merge_dots_into_main(boxes, H, W):
    """
    Merge small components (dots/hamza) into their nearest main character by x-overlap and proximity.
    """
    if not boxes:
        return []

    # Separate likely main vs dot by area and height
    areas = np.array([b[4] for b in boxes], dtype=np.float32)
    med_area = float(np.median(areas)) if len(areas) else 0.0

    main = []
    dots = []
    for b in boxes:
        x0,y0,x1,y1,area = b
        h = y1-y0
        w = x1-x0
        # dot: small and relatively short
        if (area < 0.35 * med_area and h < 0.35 * H) or (area < 0.0015 * H * W and h < 0.30 * H):
            dots.append(b)
        else:
            main.append(b)

    if not main:
        # if everything is tiny (rare), just return biggest few
        boxes = sorted(boxes, key=lambda b: b[4], reverse=True)[:7]
        return sorted(boxes, key=lambda b: b[0])

    # Merge each dot into the best main
    for d in dots:
        dx0,dy0,dx1,dy1,da = d
        dcx = 0.5*(dx0+dx1)
        dcy = 0.5*(dy0+dy1)

        best_i = None
        best_score = 1e18
        for i, m in enumerate(main):
            mx0,my0,mx1,my1,ma = m
            # require x overlap OR close x distance (dots above character)
            x_overlap = max(0, min(dx1,mx1) - max(dx0,mx0))
            x_span = max(1, min(dx1-dx0, mx1-mx0))
            ox = x_overlap / x_span
            if ox < 0.20 and abs(dcx - 0.5*(mx0+mx1)) > 0.20 * W:
                continue

            # distance: prefer dot near/above the main bbox
            mcx = 0.5*(mx0+mx1)
            mcy = 0.5*(my0+my1)
            # if dot is far below main, ignore
            if dcy > my1 + 0.20*H:
                continue

            dist = abs(dcx - mcx) + 0.7*abs(dcy - mcy)
            # bonus if dot is above main
            if dcy < my0:
                dist *= 0.7
            # bonus if x overlaps
            dist *= (0.75 if ox >= 0.25 else 1.0)

            if dist < best_score:
                best_score = dist
                best_i = i

        if best_i is not None:
            mx0,my0,mx1,my1,ma = main[best_i]
            main[best_i] = [min(mx0,dx0), min(my0,dy0), max(mx1,dx1), max(my1,dy1), ma+da]
        else:
            # If a "dot" can't be associated with any main character, it's usually noise in your samples.
            # Keep it ONLY if it's not too far from the center region (very conservative).
            if abs(dcx - 0.5 * W) < 0.40 * W:
                # still risky, but less likely to be a corner speckle
                main.append(d)
            # otherwise: drop it


    # Merge overlaps again
    main = _nms_merge_boxes(main, 0.55, 0.55)
    main = sorted(main, key=lambda b: b[0])
    return main




def _split_wide_boxes_by_vproj(clean_mask, boxes, max_total=7):
    """
    If a detected box is too wide (often multiple chars merged),
    split it by finding low-foreground vertical gaps in the ROI.
    """
    H, W = clean_mask.shape[:2]
    if not boxes:
        return boxes

    # median width to detect "too wide"
    widths = [b[2]-b[0] for b in boxes]
    med_w = np.median(widths) if len(widths) else 0

    out = []
    for b in boxes:
        x0, y0, x1, y1, area = b
        bw = x1 - x0
        bh = y1 - y0

        # decide if this looks like multiple merged chars
        too_wide = (bw > 1.55 * med_w) or (bw > 0.28 * W)
        if (not too_wide) or (bh < 0.12 * H):
            out.append(b)
            continue

        roi = clean_mask[y0:y1, x0:x1].astype(np.uint8)
        col = roi.sum(axis=0).astype(np.float32)

        if col.max() <= 0:
            out.append(b)
            continue

        # smooth projection
        k = max(5, int(0.06 * len(col))) | 1
        ker = np.ones(k, np.float32) / k
        sm = np.convolve(col, ker, mode="same")

        # valley threshold = "gap"
        thr = 0.12 * sm.max()
        is_gap = sm <= thr

        # find contiguous gap runs that are wide enough
        gaps = []
        i = 0
        min_gap_w = max(2, int(0.03 * (x1 - x0)))
        while i < len(is_gap):
            if not is_gap[i]:
                i += 1
                continue
            j = i
            while j < len(is_gap) and is_gap[j]:
                j += 1
            if (j - i) >= min_gap_w:
                gaps.append((i, j))
            i = j

        if not gaps:
            out.append(b)
            continue

        # pick up to 3 best split points (plates max 7 chars total)
        # choose the widest gaps first
        gaps = sorted(gaps, key=lambda t: (t[1]-t[0]), reverse=True)[:3]
        split_cols = sorted([int((a+b)//2) for a, b in gaps])

        # build segments
        cuts = [0] + split_cols + [x1 - x0]
        pieces = []
        for a, c in zip(cuts[:-1], cuts[1:]):
            if (c - a) < max(4, int(0.06 * (x1 - x0))):
                continue
            sub = roi[:, a:c]
            ys, xs = np.where(sub > 0)
            if len(xs) == 0:
                continue
            sx0, sx1 = xs.min(), xs.max()+1
            sy0, sy1 = ys.min(), ys.max()+1
            px0 = x0 + a + sx0
            px1 = x0 + a + sx1
            py0 = y0 + sy0
            py1 = y0 + sy1
            parea = int((sy1 - sy0) * (sx1 - sx0))
            pieces.append([px0, py0, px1, py1, parea])

        if len(pieces) >= 2:
            out.extend(pieces)
        else:
            out.append(b)

    out = sorted(out, key=lambda b: b[0])
    # keep reasonable max (4 digits + 3 letters)
    if len(out) > max_total:
        out = out[:max_total]
    return out


def _choose_split_by_largest_gap(boxes, min_d=1, max_d=4, min_l=1, max_l=3):
    """
    Choose split by combining:
    - largest x-gap
    - preference that split lies near ~55% of width (digits left, letters right)
    """
    n = len(boxes)
    if n < 2:
        return 1 if n == 1 else 0

    rights = [b[2] for b in boxes]
    lefts  = [b[0] for b in boxes]
    W = max(1, boxes[-1][2] - boxes[0][0])
    x0_all = boxes[0][0]

    best = None
    best_score = -1e18
    for i in range(1, n):
        nd, nl = i, n-i
        if not (min_d <= nd <= max_d and min_l <= nl <= max_l):
            continue

        gap = lefts[i] - rights[i-1]
        gap_norm = gap / max(1, W)

        split_x = 0.5 * (rights[i-1] + lefts[i])
        split_norm = (split_x - x0_all) / max(1, W)

        # prefer split around ~0.55
        penalty = abs(split_norm - 0.55)

        score = (gap_norm * 100.0) - (penalty * 35.0)
        if score > best_score:
            best_score = score
            best = i

    if best is None:
        # fallback: nearest to 0.55 of width
        target = x0_all + 0.55 * W
        best = min(range(1, n), key=lambda i: abs(0.5*(rights[i-1]+lefts[i]) - target))
    return best


def _draw_boxes_rgb(img_rgb, boxes, color=(255,0,0), thickness=2):
    out = img_rgb.copy()
    if out.dtype != np.uint8:
        out = _to_u8(out)
    bgr = out[..., ::-1].copy()  # to BGR for cv2
    for b in boxes:
        x0,y0,x1,y1 = map(int, b[:4])
        cv2.rectangle(bgr, (x0,y0), (x1-1,y1-1), color[::-1], thickness)
    return bgr[..., ::-1]

def _tight_char_crop(mask_crop):
    ys, xs = np.where(mask_crop > 0)
    if len(xs) == 0:
        return mask_crop
    y0,y1 = ys.min(), ys.max()+1
    x0,x1 = xs.min(), xs.max()+1
    return mask_crop[y0:y1, x0:x1]

def _pad_to_square(img_u8, pad_value=255):
    h,w = img_u8.shape[:2]
    if h == w:
        return img_u8
    if h > w:
        pad = (h-w)//2
        return cv2.copyMakeBorder(img_u8, 0, 0, pad, h-w-pad, cv2.BORDER_CONSTANT, value=pad_value)
    else:
        pad = (w-h)//2
        return cv2.copyMakeBorder(img_u8, pad, w-h-pad, 0, 0, cv2.BORDER_CONSTANT, value=pad_value)

def _crop_char_150(band_gray_u8, clean_mask, box, pad=3):
    H,W = clean_mask.shape[:2]
    x0,y0,x1,y1 = map(int, box[:4])
    x0 = max(0, x0-pad); y0 = max(0, y0-pad)
    x1 = min(W, x1+pad); y1 = min(H, y1+pad)

    m = clean_mask[y0:y1, x0:x1].astype(np.uint8)
    m = _tight_char_crop(m)

    # Make "ink" black on white background (good for digits/letters + dots)
    # WHITE ink on BLACK background (as you requested)
    out = np.zeros(m.shape, dtype=np.uint8)
    out[m > 0] = 255

    out = _pad_to_square(out, pad_value=0)

    out150 = cv2.resize(out, (150, 150), interpolation=cv2.INTER_AREA)
    return out150




def _trim_band_by_mask(band_rgb, gray_u8, enh_u8, bw_bool, bw_closed_u8, clean_bool):
    """
    Uses the final clean mask to:
    - crop away empty top/bottom area
    - wipe thick border rows near edges (prevents L/┐ shaped crops)
    Returns trimmed versions of all inputs.
    """
    H, W = clean_bool.shape[:2]
    row = clean_bool.mean(axis=1)
    if row.max() <= 0:
        return band_rgb, gray_u8, enh_u8, bw_bool, bw_closed_u8, clean_bool

    thr = max(0.01, 0.12 * row.max())
    ys = np.where(row >= thr)[0]
    if len(ys) == 0:
        return band_rgb, gray_u8, enh_u8, bw_bool, bw_closed_u8, clean_bool

    y0, y1 = int(ys.min()), int(ys.max() + 1)
    margin = max(2, int(0.08 * (y1 - y0)))
    y0 = max(0, y0 - margin)
    y1 = min(H, y1 + margin)

    # wipe thick border rows near edges
    edge_zone = max(2, int(0.08 * (y1 - y0)))
    for i in range(edge_zone):
        if clean_bool[y0 + i].mean() > 0.85:
            clean_bool[y0 + i, :] = 0
        if clean_bool[y1 - 1 - i].mean() > 0.85:
            clean_bool[y1 - 1 - i, :] = 0

    # crop everything
    band_rgb2 = band_rgb[y0:y1, :, :]
    gray2 = gray_u8[y0:y1, :]
    enh2 = enh_u8[y0:y1, :]
    bw2 = bw_bool[y0:y1, :]
    bwc2 = bw_closed_u8[y0:y1, :]
    clean2 = clean_bool[y0:y1, :]

    return band_rgb2, gray2, enh2, bw2, bwc2, clean2

def _remove_border_artifacts(mask_bool):
    """
    Removes thick plate frame/border strips that touch the band borders.
    This fixes the 'L-shape' / big rectangle crops you see in the PDF.
    """
    H, W = mask_bool.shape[:2]
    lab = label(mask_bool)
    out = mask_bool.copy()

    for rp in regionprops(lab):
        y0, x0, y1, x1 = rp.bbox
        h = y1 - y0
        w = x1 - x0
        area = rp.area

        touches_top = (y0 <= 1)
        touches_bot = (y1 >= H - 1)
        touches_left = (x0 <= 1)
        touches_right = (x1 >= W - 1)
        touches = touches_top or touches_bot or touches_left or touches_right

        if not touches:
            continue

        span_w = w / max(1, W)
        span_h = h / max(1, H)
        extent = rp.extent
        solidity = getattr(rp, "solidity", 1.0)

        # 1) Thick vertical side frame strips
        if (touches_left or touches_right) and span_h > 0.55 and span_w < 0.09 and area > 0.003 * H * W:
            out[lab == rp.label] = 0
            continue

        # 2) Thick horizontal top/bottom frame strips
        if (touches_top or touches_bot) and span_w > 0.55 and span_h < 0.10 and area > 0.003 * H * W:
            out[lab == rp.label] = 0
            continue

        # 3) Big border-attached filled blocks (common flood-fill failure leftovers)
        if area > 0.18 * H * W and (span_w > 0.70 or span_h > 0.70) and extent > 0.65 and solidity > 0.85:
            out[lab == rp.label] = 0
            continue

    return out



# ============================================================
# Main processing
# ============================================================

def _split_by_separator_valley(clean_bool, boxes, center_range=(0.35, 0.75)):
    """
    If the vertical separator exists, it appears as a LOW valley in the column projection.
    This is more reliable than 'largest x-gap' when characters are missing/merged.
    Returns (split_i, x_sep) or None.
    """
    if len(boxes) < 2:
        return None

    H, W = clean_bool.shape[:2]
    col = clean_bool.sum(axis=0).astype(np.float32)

    k = max(9, (W // 30) | 1)
    sm = np.convolve(col, np.ones(k, np.float32) / k, mode="same")
    mx = float(sm.max())
    if mx <= 0:
        return None

    lo = int(center_range[0] * W)
    hi = int(center_range[1] * W)
    if hi <= lo + 5:
        return None

    valley = lo + int(np.argmin(sm[lo:hi]))

    # Must be "empty enough" compared to max ink column
    if sm[valley] > 0.18 * mx:
        return None

    thr = 0.25 * mx
    i = valley
    while i > 0 and sm[i] < thr:
        i -= 1
    j = valley
    while j < W - 1 and sm[j] < thr:
        j += 1

    if (j - i) < max(2, int(0.01 * W)):
        return None

    x_sep = int((i + j) // 2)
    centers = [0.5 * (b[0] + b[2]) for b in boxes]
    split_i = sum(c < x_sep for c in centers)

    if not (1 <= split_i <= len(boxes) - 1):
        return None

    return split_i, x_sep

def _erase_separator_band_by_projection(clean_bool, center_range=(0.30, 0.80)):
    """
    Detect the vertical separator as a LOW valley in column projection,
    then erase a narrow band around it so it cannot become a 'character'.
    Returns: (new_mask, (x0,x1) or None, x_sep or None)
    """
    H, W = clean_bool.shape[:2]
    col = clean_bool.mean(axis=0).astype(np.float32)

    # smooth
    k = max(9, (W // 25) | 1)
    sm = np.convolve(col, np.ones(k, np.float32) / k, mode="same")

    mx = float(sm.max())
    if mx < 1e-6:
        return clean_bool, None, None

    lo = int(center_range[0] * W)
    hi = int(center_range[1] * W)
    if hi <= lo + 6:
        return clean_bool, None, None

    valley = lo + int(np.argmin(sm[lo:hi]))
    v = float(sm[valley])

    # Must be a strong valley compared to peaks
    left_peak = float(sm[:valley].max()) if valley > 3 else 0.0
    right_peak = float(sm[valley+1:].max()) if valley < W-4 else 0.0

    if not (left_peak > 0.35 * mx and right_peak > 0.35 * mx and v < 0.12 * mx):
        return clean_bool, None, None

    # expand band around valley while it's still "low ink"
    thr = 0.22 * mx
    i = valley
    while i > 0 and sm[i] < thr:
        i -= 1
    j = valley
    while j < W - 1 and sm[j] < thr:
        j += 1

    x0 = max(0, i - 1)
    x1 = min(W, j + 2)

    # sanity: separator band shouldn't be huge
    if (x1 - x0) > int(0.10 * W):
        return clean_bool, None, None
    if (x1 - x0) < 2:
        return clean_bool, None, None

    out = clean_bool.copy()
    out[:, x0:x1] = 0
    x_sep = int((x0 + x1) // 2)
    return out, (x0, x1), x_sep







def process_plate(img_rgb):
    """
    Returns:
      digits_150: list of 150x150 uint8 (digits left->right)
      letters_150: list of 150x150 uint8 (letters right->left)
      debug: dict of intermediate visuals
    """
    img_rgb = _ensure_rgb(img_rgb)

    # 1) "Blue header cut"
    cut_rgb, y_cut = _blue_header_cut(img_rgb)

    # 2) Auto text-band crop
    band_rgb, enh_band_u8, gray_band_u8, (yb0, yb1) = _auto_text_band(cut_rgb)

    # Enhancement components for display (we want both enhanced gray + enhancement output)
    gray_u8 = gray_band_u8
    enh_u8, clahe_u8, blackhat_u8 = _enhance_for_text(gray_u8)

    # 3) Binarize (TRY BOTH polarities and pick the one that yields plausible components)
    bw = _binarize_stroke_preserving(enh_u8, clahe_u8=clahe_u8, blackhat_u8=blackhat_u8)
    bw_closed_vis, clean = _morph_clean(bw)




    # --- NEW: trim band using the final clean mask (removes border lines & extra band height) ---
    band_rgb, gray_u8, enh_u8, bw, bw_closed_vis, clean = _trim_band_by_mask(
        band_rgb, gray_u8, enh_u8, bw, bw_closed_vis, clean
    )


    # --- NEW: remove border/frame artifacts that break crops ---
    clean = _remove_border_artifacts(clean)
    #bw_closed = (clean.astype(np.uint8) * 255)



    # --- NEW: erase the vertical separator band (prevents 'thin bar' crops) ---
    clean, sep_band, x_sep = _erase_separator_band_by_projection(clean)
    #bw_closed = (clean.astype(np.uint8) * 255)




    # ---- fallback to break merged blobs (light erosion) if CC count is too small/too large ----
    def _count_reasonable_cc(m):
        regs = regionprops(label(m))
        Hm, Wm = m.shape[:2]
        n = 0
        for r in regs:
            y0,x0,y1,x1 = r.bbox
            h = y1-y0; w = x1-x0
            if r.area < max(12, int(0.00008*Hm*Wm)):
                continue
            if h < 0.16*Hm:
                continue
            if w < 0.004*Wm or w > 0.65*Wm:
                continue
            n += 1
        return n

    ncc = _count_reasonable_cc(clean)
    if ncc < 3 or ncc > 12:
        cu8 = clean.astype(np.uint8) * 255
        ek = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        er = cv2.erode(cu8, ek, iterations=1) > 0

        # keep dots: OR the erosion result with original small components
        # (so we don't lose dot-only regions)
        lab0 = label(clean)
        dot_keep = np.zeros_like(clean, dtype=bool)

        # keep small components (likely dots/hamza) from original mask
        for rp in regionprops(lab0):
            if rp.area <= max(25, int(0.00012 * clean.shape[0] * clean.shape[1])):
                dot_keep[lab0 == rp.label] = True

        alt = er | dot_keep


        if 3 <= _count_reasonable_cc(alt) <= 12:
            clean = alt




    # 5) Boxes (IMPORTANT FIX)
    # Use a small dilation ONLY for grouping fragments into single character boxes.
    # Cropping still uses the original 'clean' mask (so dots/shape stay correct).
    Hm, Wm = clean.shape[:2]
    group_k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    group_mask = (cv2.dilate(clean.astype(np.uint8) * 255, group_k, iterations=1) > 0)

    boxes_raw, lab = _boxes_from_mask(group_mask)
    boxes_raw = _nms_merge_boxes(boxes_raw, 0.55, 0.55)
    boxes = _merge_dots_into_main(boxes_raw, Hm, Wm)


    # Extra filtering on final boxes (avoid remaining junk)
    H, W = clean.shape[:2]
    boxes_f = []
    for b in boxes:
        x0,y0,x1,y1,area = b
        h = y1-y0; w = x1-x0
        if area < max(18, int(0.00014 * H * W)):
            continue

        # characters can be shorter if band crop is tall
        if h < 0.16 * H:
            continue

        # allow ultra-thin "١"
        if w < 0.005 * W or w > 0.60 * W:
            continue

        boxes_f.append(b)

    # If we became too strict, fall back to earlier merged boxes
    if len(boxes_f) < 2 and len(boxes) >= 2:
        boxes_f = boxes

    boxes_f = sorted(boxes_f, key=lambda b: b[0])
    # If characters merged into big blobs, split wide boxes using vertical projection gaps
    boxes_f = _split_wide_boxes_by_vproj(clean, boxes_f, max_total=7)
    boxes_f = sorted(boxes_f, key=lambda b: b[0])

    # NEW: drop any remaining stripe-like boxes (baseline/frame leftovers)
    Hm, Wm = clean.shape[:2]

    def _is_stripe_box(b):
        x0, y0, x1, y1, area = b
        w = x1 - x0
        h = y1 - y0

        # horizontal stripe
        if h <= max(2, int(0.18 * Hm)) and w >= int(0.55 * Wm):
            return True

        # vertical stripe that behaves like a separator (touches TB) and is very thin
        touches_top = (y0 <= 1)
        touches_bot = (y1 >= Hm - 1)
        if (touches_top and touches_bot) and (w <= max(2, int(0.030 * Wm))) and (h >= int(0.80 * Hm)):
            return True

        return False

    boxes_f = [b for b in boxes_f if not _is_stripe_box(b)]
    boxes_f = sorted(boxes_f, key=lambda b: b[0])



    # Remove tall separator-like bars (touch top+bottom of band, narrow, central-ish)
    Hm, Wm = clean.shape[:2]
    def _is_global_sep(b):
        x0,y0,x1,y1,area = b
        w = x1-x0; h = y1-y0
        touches_tb = (y0 <= 1) and (y1 >= Hm-1)
        central = (0.30*Wm <= 0.5*(x0+x1) <= 0.70*Wm)
        very_tall = h > 0.92*Hm
        narrow = w < 0.035*Wm
        return touches_tb and central and very_tall and narrow

    boxes_f = [b for b in boxes_f if not _is_global_sep(b)]
    boxes_f = sorted(boxes_f, key=lambda b: b[0])



    # 6) Split digits vs letters
    # If we detected separator projection x_sep, use it (more stable than largest gap).
    if 'x_sep' in locals() and x_sep is not None:
        centers = [0.5 * (b[0] + b[2]) for b in boxes_f]
        split_i = int(np.clip(np.sum(np.array(centers) < x_sep), 1, max(1, len(boxes_f)-1)))
    else:
        split_i = _choose_split_by_largest_gap(boxes_f, 1, 4, 1, 3)



    digit_boxes = boxes_f[:split_i]
    letter_boxes = boxes_f[split_i:]


    # ---- remove separator line if it exists near the split ----
    if len(boxes_f) >= 3:
        rights = [b[2] for b in boxes_f]
        lefts  = [b[0] for b in boxes_f]
        split_x = 0.5 * (rights[split_i-1] + lefts[split_i])
        Hm, Wm = clean.shape[:2]

        def is_separator(b):
            x0,y0,x1,y1,area = b
            w = x1-x0; h = y1-y0
            cx = 0.5*(x0+x1)
            return (h > 0.80*Hm) and (w < 0.02*Wm) and (abs(cx - split_x) < 0.10*Wm)

        digit_boxes = [b for b in digit_boxes if not is_separator(b)]
        letter_boxes = [b for b in letter_boxes if not is_separator(b)]


    # Enforce max counts safely
    digit_boxes = sorted(digit_boxes, key=lambda b: b[0])[:4]
    letter_boxes = sorted(letter_boxes, key=lambda b: b[0])[-3:]  # keep rightmost 3 as letters

    # If split produced empty side, fix by simple half split
    if len(digit_boxes) == 0 and len(boxes_f) > 0:
        digit_boxes = boxes_f[:1]
        letter_boxes = boxes_f[1:]
    if len(letter_boxes) == 0 and len(boxes_f) > 1:
        letter_boxes = boxes_f[-1:]
        digit_boxes = boxes_f[:-1]

    # Order outputs:
    digit_boxes = sorted(digit_boxes, key=lambda b: b[0])                  # left->right
    letter_boxes_lr = sorted(letter_boxes, key=lambda b: b[0])             # left->right for drawing
    letter_boxes = sorted(letter_boxes, key=lambda b: b[0], reverse=True)  # right->left for output

    # 7) Crop 150x150 characters
    digits_150 = [_crop_char_150(gray_u8, clean, b, pad=3) for b in digit_boxes]
    letters_150 = [_crop_char_150(gray_u8, clean, b, pad=3) for b in letter_boxes]

    # 8) Overlays
    overlay_all = _draw_boxes_rgb(band_rgb, boxes_f, color=(255, 255, 0), thickness=2)
    overlay_split = band_rgb.copy()
    overlay_split = _draw_boxes_rgb(overlay_split, digit_boxes, color=(0, 255, 0), thickness=2)   # digits = green
    overlay_split = _draw_boxes_rgb(overlay_split, letter_boxes_lr, color=(255, 0, 0), thickness=2)  # letters = red

    debug = dict(
        original=img_rgb,
        cut=cut_rgb,
        band=band_rgb,
        enhancement=blackhat_u8,        # "enhancement output" (blackhat strokes)
        enh_gray=enh_u8,                # enhanced grayscale used for binarization
        binary=bw.astype(np.uint8)*255,
        binary_closed=bw_closed_vis,
        clean_mask=clean.astype(np.uint8)*255,
        overlay_all=overlay_all,
        overlay_split=overlay_split,
        digit_boxes=digit_boxes,
        letter_boxes=letter_boxes
    )

    return digits_150, letters_150, debug

# ============================================================
# Run on all images in "maaw final crops"
# ============================================================
input_dir = _find_input_dir("maaw final crops")
exts = ("*.png","*.jpg","*.jpeg","*.bmp","*.webp")
paths = []
for e in exts:
    paths += glob.glob(os.path.join(input_dir, e))
paths = sorted(paths)

print(f"Found {len(paths)} images in: {input_dir}")

for p in paths:
    img = _read_img(p)
    digits, letters, dbg = process_plate(img)

    # -------- Line 1 (4 images) --------
    show_images(
        [dbg["original"], dbg["cut"], dbg["band"], dbg["enhancement"]],
        ["Original", "After blue header cut", "Auto text-band crop", "Enhancement (blackhat)"]
    )

    # -------- Line 2 (4 images) --------
    show_images(
        [dbg["enh_gray"], dbg["binary"], dbg["binary_closed"], dbg["clean_mask"]],
        ["Enhanced grayscale", "Binary", "Binary after closing", "Final clean mask"]
    )

    # -------- Line 3 (2 overlays + crops in reading order) --------
    # Order: digits left->right, then letters right->left
    crops = digits + letters
    crop_titles = ([f"Digit {i+1}" for i in range(len(digits))] +
                   [f"Letter {i+1}" for i in range(len(letters))])

    show_images(
        [dbg["overlay_all"], dbg["overlay_split"]] + crops,
        ["Overlay: all boxes", "Overlay: digits (green) vs letters (red)"] + crop_titles
    )

    print(f"\n{os.path.basename(p)}  ->  digits={len(digits)}  letters={len(letters)}")




'''
MAAW ANGER
'''     

In [ ]:
"""
MAAW LABUBU — Option A (Rewrite the pipeline correctly)
======================================================
Key redesign vs your previous pipeline:
- NO early digits/letters split using the vertical separator (unreliable in your samples).
- Segment ALL characters together first (stroke-preserving).
- Then split into [digits | letters] using a robust 1D clustering rule: "largest x-gap" with plate constraints.
- Text band crop is NOT a fixed ratio anymore: we locate the densest text rows using black-hat + horizontal projection.
- Thresholding is stroke-preserving (black-hat + Sauvola/adaptive hybrid), tuned for thin Arabic strokes (١,٧, etc).
- Output visualization is ALWAYS in 3 lines per image:
  Line 1: Original | Below-blue (rough) | Text-band (auto) | Enhancement
  Line 2: Gray | Binary | Clean mask | Boxes overlay
  Line 3: Digits boxes | Letters boxes | Digit 1..N | Letter 1..M
"""

# =========================
# Imports
# =========================
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.filters import threshold_sauvola


# =========================
# MANDATORY visualization function (EXACT)
# =========================
def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


# =========================
# Utilities
# =========================
def bgr_to_rgb(img_bgr):
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

def safe_u8(x):
    x = np.clip(x, 0, 255)
    return x.astype(np.uint8)

def tight_crop_from_mask(img, mask_u8):
    ys, xs = np.where(mask_u8 > 0)
    if len(xs) == 0:
        return img
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    return img[y0:y1+1, x0:x1+1]

def pad_to_square(img, pad_value=0):
    h, w = img.shape[:2]
    if h == w:
        return img
    if h > w:
        p = (h - w) // 2
        return cv2.copyMakeBorder(img, 0, 0, p, h - w - p, cv2.BORDER_CONSTANT, value=pad_value)
    else:
        p = (w - h) // 2
        return cv2.copyMakeBorder(img, p, w - h - p, 0, 0, cv2.BORDER_CONSTANT, value=pad_value)

def rotate_keep_bounds(img, angle_deg, border_value=0):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle_deg, 1.0)
    cos = np.abs(M[0,0]); sin = np.abs(M[0,1])
    new_w = int((h*sin) + (w*cos))
    new_h = int((h*cos) + (w*sin))
    M[0,2] += (new_w/2) - (w/2)
    M[1,2] += (new_h/2) - (h/2)
    return cv2.warpAffine(img, M, (new_w, new_h),
                          flags=cv2.INTER_LINEAR,
                          borderMode=cv2.BORDER_CONSTANT,
                          borderValue=border_value)

def deskew_bin_char(bin_u8):
    # Deskew only if we have enough pixels
    ys, xs = np.where(bin_u8 > 0)
    if len(xs) < 40:
        return bin_u8
    pts = np.stack([xs, ys], axis=1).astype(np.float32)
    rect = cv2.minAreaRect(pts)
    angle = rect[-1]  # [-90,0)
    if angle < -45:
        angle += 90
    # modest correction only (avoid messing up noisy blobs)
    if abs(angle) <= 18:
        rot = rotate_keep_bounds(bin_u8, angle, border_value=0)
        rot = (rot > 0).astype(np.uint8) * 255
        return rot
    return bin_u8


# =========================
# 1) Remove blue header (robust, but permissive)
# =========================
def remove_blue_header(plate_bgr):
    """
    Egyptian plates: blue header at top (EGYPT / مصر).
    We detect it in HSV and crop BELOW it.
    If uncertain, we fall back to a mild geometric crop.
    """
    H, W = plate_bgr.shape[:2]
    hsv = cv2.cvtColor(plate_bgr, cv2.COLOR_BGR2HSV)

    # Blue range (broad for washed-out and saturated)
    lower = np.array([80, 30, 30], dtype=np.uint8)
    upper = np.array([145, 255, 255], dtype=np.uint8)
    blue = cv2.inRange(hsv, lower, upper)

    # Only analyze top half
    top = blue[:int(0.45 * H), :]
    top = cv2.medianBlur(top, 5)
    top = cv2.morphologyEx(top, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5)))

    cnts, _ = cv2.findContours(top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    y_cut = None

    if cnts:
        cnt = max(cnts, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        area = w*h
        # header tends to be wide and not tiny
        if w > 0.45*W and area > 0.02*(H*W):
            # Never allow blue crop to exceed safe limit
            y_cut = y + h
            # HARD SAFETY: never cut more than top 18%
            y_cut = min(y + h, int(0.18 * H))




    if y_cut is None:
        # fallback (very mild): keep lower ~70%
        y_cut = int(0.22 * H)

    # also remove a bit of bottom to avoid bumper
    cropped = plate_bgr[y_cut:int(0.98*H), :]
    return cropped


# =========================
# 2) Auto-crop the true text band using projection (NO fixed ratio)
# =========================
def auto_text_band(bgr_below_blue):
    """
    Find the densest text rows using black-hat on grayscale + row projection.
    This removes bumper edges and keeps the white band containing digits+letters.
    """
    H, W = bgr_below_blue.shape[:2]
    gray = cv2.cvtColor(bgr_below_blue, cv2.COLOR_BGR2GRAY)

    # Contrast normalize
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
    g = clahe.apply(gray)

    # Black-hat emphasizes dark text on bright background
    k = max(9, (min(H, W)//25) | 1)  # odd-ish
    bh = cv2.morphologyEx(g, cv2.MORPH_BLACKHAT, cv2.getStructuringElement(cv2.MORPH_RECT, (k, k)))

    # Normalize and binarize blackhat to locate rows containing strokes
    bh_blur = cv2.GaussianBlur(bh, (3,3), 0)
    _, bh_bin = cv2.threshold(bh_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Row projection
    row_sum = (bh_bin > 0).sum(axis=1).astype(np.float32)
    row_sum = cv2.GaussianBlur(row_sum.reshape(-1,1), (1, 21), 0).ravel()

    # Find main band by thresholding projection
    thr = 0.25 * row_sum.max()
    rows = np.where(row_sum > thr)[0]

    if len(rows) < 10:
        # fallback: central region (still safer than fixed ratio)
        y0 = int(0.10*H)
        y1 = int(0.90*H)
    else:
        y0 = max(0, int(rows.min() - 0.06 * H))
        y1 = min(H, int(rows.max() + 0.06 * H))


    band = bgr_below_blue[y0:y1, :]


    # After band extraction
    H2 = band.shape[0]
    # Adaptive vertical padding — protect dots
    top_pad = int(0.02 * H2)
    bot_pad = int(0.03 * H2)

    band = band[top_pad:H2 - bot_pad, :]




    # Return also an "enhancement" visualization for Line 1
    enh_vis = bh  # blackhat view
    return band, g, enh_vis


# =========================
# 3) Stroke-preserving binarization (black-hat + Sauvola/adaptive hybrid)
# =========================
def stroke_binarize(band_bgr):
    gray = cv2.cvtColor(band_bgr, cv2.COLOR_BGR2GRAY)

    # CLAHE (preserves thin Arabic strokes)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    g = clahe.apply(gray)

    # Gaussian blur stabilizes thresholding
    g_blur = cv2.GaussianBlur(g, (3,3), 0)

    # --- Primary: OTSU ---
    _, bin1 = cv2.threshold(
        g_blur, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # --- Backup: Adaptive ---
    bin2 = cv2.adaptiveThreshold(
        g, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        31, 7
    )

    # Choose by foreground ratio (CRITICAL)
    def fg_ratio(b): return (b > 0).mean()
    r1, r2 = fg_ratio(bin1), fg_ratio(bin2)

    if 0.04 <= r1 <= 0.35:
        chosen = bin1
    else:
        chosen = bin2

    # Black-hat ONLY for visualization (Line 1)
    k = max(9, (min(g.shape)//22) | 1)
    bh = cv2.morphologyEx(
        g, cv2.MORPH_BLACKHAT,
        cv2.getStructuringElement(cv2.MORPH_RECT, (k, k))
    )

    return g, chosen, bh




# =========================
# 4) Clean mask (Arabic-aware: keep thin strokes, remove bumper noise)
# =========================
def clean_mask(bin_u8):
    """
    Arabic-aware cleaning:
    - Preserve dots (ن، ق، ي)
    - Preserve internal holes (و، ص، ه)
    - Remove only true noise
    """
    H, W = bin_u8.shape

    # Binary → bool
    m = (bin_u8 > 0)

    # --- 1) REMOVE ONLY VERY TINY SPECKLES ---
    # Dots are small but NOT microscopic
    # Preserve Arabic dots explicitly
    # Allow Arabic dots (very small but valid)
    min_area = max(2, int(0.000015 * H * W))

    m = remove_small_objects(m, min_size=min_area)

    # --- 2) VERY LIGHT CLOSE (ONLY reconnect broken strokes) ---
    m8 = (m.astype(np.uint8) * 255)
    m8 = cv2.morphologyEx(
        m8,
        cv2.MORPH_CLOSE,
        cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)),
        iterations=1
    )

    # --- 3) DO NOT globally fill holes ---
    # Instead: fill only MICRO pinholes caused by thresholding
    m = (m8 > 0)
    m = remove_small_holes(
        m,
        area_threshold=max(8, int(0.00005 * H * W))
    )

    return (m.astype(np.uint8) * 255)



# =========================
# 5) Connected components + split merged blobs
# =========================
def split_merged_by_valleys(mask_u8, box, max_splits=2):
    """
    Split a too-wide component using vertical projection valleys.
    Returns a list of sub-boxes in original coordinates.
    """
    x, y, w, h = box
    crop = mask_u8[y:y+h, x:x+w]
    if w < 10:
        return [box]

    proj = (crop > 0).sum(axis=0).astype(np.float32)
    proj = cv2.GaussianBlur(proj.reshape(1,-1), (max(5, (w//10)|1), 1), 0).ravel()

    # valley threshold relative to max stroke density
    thr = 0.22 * proj.max()
    valleys = np.where(proj < thr)[0]

    cuts = []
    if len(valleys) > 0:
        groups = np.split(valleys, np.where(np.diff(valleys) > 3)[0] + 1)
        for g in groups:
            c = int(np.median(g))
            if 0.15*w < c < 0.85*w:
                cuts.append(c)

    cuts = sorted(cuts)
    if len(cuts) == 0:
        return [box]

    # limit number of splits
    cuts = cuts[:max_splits]

    xs = [0] + cuts + [w]
    out = []
    for i in range(len(xs)-1):
        xx0, xx1 = xs[i], xs[i+1]
        ww = xx1 - xx0
        if ww < max(6, int(0.06*w)):  # too thin
            continue
        out.append((x+xx0, y, ww, h))

    return out if len(out) >= 2 else [box]


def extract_all_char_boxes(clean_u8):
    """
    Extract candidate character boxes from the whole band (digits+letters together).
    Apply Arabic-aware filters and split merged boxes.
    """
    H, W = clean_u8.shape
    num, labels, stats, _ = cv2.connectedComponentsWithStats((clean_u8 > 0).astype(np.uint8), connectivity=8)

    boxes = []
    for i in range(1, num):
        x, y, w, h, area = stats[i]

        # reject tiny
        # Never kill potential dots / hamza
        if area < max(6, int(0.00004 * H * W)):
            continue



        # reject flat blobs EXCEPT tall thin strokes (Arabic ١)
        if h < 0.18 * H:
            aspect = h / max(w, 1)
            if aspect < 3.0:   # allow tall thin digits like ١
                continue


        # reject huge regions
        if h > 0.98 * H or w > 0.80 * W:
            continue


        # Reject tall-thin vertical straps (plate divider / reflection)
        aspect = h / max(w, 1)
        # Kill plate dividers / emboss lines
        if aspect > 3.8 and w < 0.10 * W and area < 0.12 * H * H:
            continue





        # reject border-touching long lines (common bumper edges)
        if (y <= 1 or y+h >= H-1) and (w > 0.35*W) and (h < 0.45*H):
            continue


        # Reject left/right frame borders
        edge_margin = int(0.04 * W)
        if (x < edge_margin or x + w > W - edge_margin) and h > 0.50 * H:
            continue






        boxes.append((x, y, w, h))

    # Sort left->right
    boxes = sorted(boxes, key=lambda b: b[0])

    # Split wide boxes (merged characters)
    out = []
    for b in boxes:
        x,y,w,h = b
        if w > 0.22 * W:
            out.extend(split_merged_by_valleys(clean_u8, b, max_splits=2))
        else:
            out.append(b)

    out = sorted(out, key=lambda b: b[0])


    # Remove near-duplicate overlapping boxes
    filtered = []
    for b in out:
        x,y,w,h = b
        keep = True
        for bb in filtered:
            xx,yy,ww,hh = bb
            iou_w = min(x+w, xx+ww) - max(x, xx)
            iou_h = min(y+h, yy+hh) - max(y, yy)
            if iou_w > 0 and iou_h > 0:
                inter = iou_w * iou_h
                if inter > 0.6 * min(w*h, ww*hh):
                    keep = False
                    break
        if keep:
            filtered.append(b)

    out = filtered


    ##

    

    # --- Merge small dot boxes with nearest base character ---
    merged = []
    used = [False] * len(out)

    for i, b in enumerate(out):
        if used[i]:
            continue
        x, y, w, h = b
        area = w * h

        # candidate dot
        if area < 0.06 * H * H:
            cx = x + w / 2
            cy = y + h / 2
            best_j, best_d = None, 1e9

            for j, bb in enumerate(out):
                if i == j or used[j]:
                    continue
                x2, y2, w2, h2 = bb
                if h2 > h * 2:  # likely base letter
                    cx2 = x2 + w2 / 2
                    cy2 = y2 + h2 / 2
                    d = abs(cx - cx2) + abs(cy - cy2)
                    if d < best_d:
                        best_d = d
                        best_j = j

            # Prefer vertical proximity (dots are ABOVE base in Arabic)
            # Arabic dots: above base, close vertically, limited horizontal drift
            if (
                best_j is not None
                and cy < cy2                       # dot ABOVE base
                and abs(cx - cx2) < 0.12 * W       # tight horizontal lock
                and abs(cy - cy2) < 0.18 * H       # tighter vertical range
            ):
                # merge dot into base box
                bx, by, bw, bh = out[best_j]
                nx = min(x, bx)
                ny = min(y, by)
                nw = max(x + w, bx + bw) - nx
                nh = max(y + h, by + bh) - ny
                out[best_j] = (nx, ny, nw, nh)
                used[i] = True
                continue

        merged.append(b)
        used[i] = True

    out = merged



    # Keep at most 7 chars by area (since plates are 1..4 digits + 1..3 letters)
    if len(out) > 7:
        out = sorted(out, key=lambda b: b[2]*b[3], reverse=True)[:7]
        out = sorted(out, key=lambda b: b[0])



    return out


# =========================
# 6) Split all characters into digits vs letters WITHOUT the separator
# =========================
def split_digits_letters_by_gap(boxes, W):
    """
    Robust split using the largest gap in x-centers with plate constraints:
    - digits count: 1..4 (LEFT)
    - letters count: 1..3 (RIGHT)
    We pick the split that maximizes inter-group gap while satisfying constraints.
    """
    if len(boxes) < 2:
        return boxes, []

    centers = np.array([b[0] + b[2]/2 for b in boxes], dtype=np.float32)

    # Candidate splits i: left = [0..i], right = [i+1..]
    best = None
    for i in range(len(boxes)-1):
        left_n = i+1
        right_n = len(boxes) - left_n
        if not (1 <= left_n <= 4):  # digits constraint
            continue
        if not (1 <= right_n <= 3):  # letters constraint
            continue

        gap = centers[i+1] - centers[i]
        # prefer splits near middle-ish, but gap is main driver
        score = gap

        if best is None or score > best[0]:
            best = (score, i)

    if best is None:
        # fallback: force a feasible split by best gap ignoring constraints
        gaps = centers[1:] - centers[:-1]
        i = int(np.argmax(gaps))
        return boxes[:i+1], boxes[i+1:]

    i = best[1]
    
    left = boxes[:i+1]
    right = boxes[i+1:]

    # Digits usually TALLER than letters
    lh = np.mean([b[3] for b in left])
    rh = np.mean([b[3] for b in right])

    # Prefer taller group as digits
    if lh < rh:
        left, right = right, left


    return left, right



# =========================
# 7) Extract character images from boxes
# =========================
def extract_chars_from_boxes(clean_u8, boxes):
    """
    Returns list of normalized (150x150) binary chars.
    """
    H, W = clean_u8.shape
    chars = []
    for (x,y,w,h) in boxes:
        pad = max(1, int(0.08*h))
        x0 = max(0, x-pad); y0 = max(0, y-pad)
        x1 = min(W, x+w+pad); y1 = min(H, y+h+pad)

        c = clean_u8[y0:y1, x0:x1]
        c = tight_crop_from_mask(c, c)
        c = deskew_bin_char(c)
        c = tight_crop_from_mask(c, c)

        c = pad_to_square(c, 0)
        c = cv2.resize(c, (150,150), interpolation=cv2.INTER_AREA)

        chars.append(c)
    return chars


# =========================
# 8) Drawing helpers
# =========================
def draw_boxes_on_rgb(bgr, boxes, color=(0,255,0), thickness=2):
    rgb = bgr_to_rgb(bgr.copy())
    for (x,y,w,h) in boxes:
        cv2.rectangle(rgb, (x,y), (x+w, y+h), color, thickness)
    return rgb


# =========================
# MAIN LOOP
# =========================
BASE_DIR = "Localization Test/FinalCrops"
paths = sorted(glob.glob(os.path.join(BASE_DIR, "*.*")))
if len(paths) == 0:
    raise FileNotFoundError(f"No images found in: {BASE_DIR}")

for idx, p in enumerate(paths, 1):
    img_bgr = cv2.imread(p)
    if img_bgr is None:
        continue

    # ---- Stage A: remove blue header
    below_blue = remove_blue_header(img_bgr)

    # ---- Stage B: auto-crop true text band using projection
    band_bgr, norm_gray_vis, enh_vis = auto_text_band(below_blue)

    # ---- Stage C: binarize (stroke-preserving)
    gray, bin_u8, bh = stroke_binarize(band_bgr)

    # ---- Stage D: clean
    clean_u8 = clean_mask(bin_u8)

    # ---- Stage E: CC -> all boxes
    all_boxes = extract_all_char_boxes(clean_u8)

    # ---- Stage F: split to digits/letters by largest x-gap under constraints
    d_boxes, l_boxes = split_digits_letters_by_gap(all_boxes, clean_u8.shape[1])

    # ---- Letters should be displayed right->left (Arabic reading),
    # but we keep their boxes in x-order for extraction, then reverse for display.
    digits_chars = extract_chars_from_boxes(clean_u8, d_boxes)
    letters_chars = extract_chars_from_boxes(clean_u8, l_boxes)

    # reverse letters display order (right->left)
    if len(l_boxes) > 1:
        letters_chars = letters_chars[::-1]
        l_boxes_disp = l_boxes[::-1]
    else:
        l_boxes_disp = l_boxes

    # ---- Overlays
    overlay_all = draw_boxes_on_rgb(band_bgr, all_boxes, color=(0,255,0), thickness=2)
    overlay_digits = draw_boxes_on_rgb(band_bgr, d_boxes, color=(0,255,0), thickness=2)
    overlay_letters = draw_boxes_on_rgb(band_bgr, l_boxes, color=(0,255,0), thickness=2)

    # =========================
    # LINE 1: Original | Below-blue | Text-band | Enhancement
    # =========================
    # Enhancement view: blackhat from the band (text emphasis)
    show_images(
        [
            bgr_to_rgb(img_bgr),
            bgr_to_rgb(below_blue),
            bgr_to_rgb(band_bgr),
            bh
        ],
        [
            f"[{idx}] Original ({os.path.basename(p)})",
            "Below Blue Header",
            "Auto Text Band",
            "Enhancement (Black-hat)"
        ]
    )

    # =========================
    # LINE 2: Gray | Binary | Clean | Boxes (ALL)
    # =========================
    show_images(
        [
            gray,
            bin_u8,
            clean_u8,
            overlay_all
        ],
        [
            "Band Gray (CLAHE)",
            "Binary (stroke-preserving)",
            "Clean Mask",
            "Boxes (ALL chars)"
        ]
    )

    # =========================
    # LINE 3: Digits boxes | Letters boxes | Digits... | Letters...
    # =========================
    line3_images = [overlay_digits, overlay_letters] + digits_chars + letters_chars
    line3_titles = (
        ["Digits Boxes (LEFT group)", "Letters Boxes (RIGHT group)"]
        + [f"Digit {i+1}" for i in range(len(digits_chars))]
        + [f"Letter {i+1}" for i in range(len(letters_chars))]
    )

    # If nothing detected, still show overlays only (avoid empty show_images)
    if len(line3_images) == 0:
        line3_images = [overlay_all]
        line3_titles = ["No characters detected"]

    show_images(line3_images, line3_titles)



'''
MAAW LABUBU
'''

In [ ]:
"""
MAAW LABUBU — Option A (Rewrite the pipeline correctly)
======================================================
Key redesign vs your previous pipeline:
- NO early digits/letters split using the vertical separator (unreliable in your samples).
- Segment ALL characters together first (stroke-preserving).
- Then split into [digits | letters] using a robust 1D clustering rule: "largest x-gap" with plate constraints.
- Text band crop is NOT a fixed ratio anymore: we locate the densest text rows using black-hat + horizontal projection.
- Thresholding is stroke-preserving (black-hat + Sauvola/adaptive hybrid), tuned for thin Arabic strokes (١,٧, etc).
- Output visualization is ALWAYS in 3 lines per image:
  Line 1: Original | Below-blue (rough) | Text-band (auto) | Enhancement
  Line 2: Gray | Binary | Clean mask | Boxes overlay
  Line 3: Digits boxes | Letters boxes | Digit 1..N | Letter 1..M
"""

# =========================
# Imports
# =========================
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.filters import threshold_sauvola


# =========================
# MANDATORY visualization function (EXACT)
# =========================
def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


# =========================
# Utilities
# =========================
def bgr_to_rgb(img_bgr):
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

def safe_u8(x):
    x = np.clip(x, 0, 255)
    return x.astype(np.uint8)

def tight_crop_from_mask(img, mask_u8):
    ys, xs = np.where(mask_u8 > 0)
    if len(xs) == 0:
        return img
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    return img[y0:y1+1, x0:x1+1]

def pad_to_square(img, pad_value=0):
    h, w = img.shape[:2]
    if h == w:
        return img
    if h > w:
        p = (h - w) // 2
        return cv2.copyMakeBorder(img, 0, 0, p, h - w - p, cv2.BORDER_CONSTANT, value=pad_value)
    else:
        p = (w - h) // 2
        return cv2.copyMakeBorder(img, p, w - h - p, 0, 0, cv2.BORDER_CONSTANT, value=pad_value)

def rotate_keep_bounds(img, angle_deg, border_value=0):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle_deg, 1.0)
    cos = np.abs(M[0,0]); sin = np.abs(M[0,1])
    new_w = int((h*sin) + (w*cos))
    new_h = int((h*cos) + (w*sin))
    M[0,2] += (new_w/2) - (w/2)
    M[1,2] += (new_h/2) - (h/2)
    return cv2.warpAffine(img, M, (new_w, new_h),
                          flags=cv2.INTER_LINEAR,
                          borderMode=cv2.BORDER_CONSTANT,
                          borderValue=border_value)

def deskew_bin_char(bin_u8):
    # Deskew only if we have enough pixels
    ys, xs = np.where(bin_u8 > 0)
    if len(xs) < 40:
        return bin_u8
    pts = np.stack([xs, ys], axis=1).astype(np.float32)
    rect = cv2.minAreaRect(pts)
    angle = rect[-1]  # [-90,0)
    if angle < -45:
        angle += 90
    # modest correction only (avoid messing up noisy blobs)
    if abs(angle) <= 18:
        rot = rotate_keep_bounds(bin_u8, angle, border_value=0)
        rot = (rot > 0).astype(np.uint8) * 255
        return rot
    return bin_u8


# =========================
# 1) Remove blue header (robust, but permissive)
# =========================
def remove_blue_header(plate_bgr):
    """
    Egyptian plates: blue header at top (EGYPT / مصر).
    We detect it in HSV and crop BELOW it.
    If uncertain, we fall back to a mild geometric crop.
    """
    H, W = plate_bgr.shape[:2]
    hsv = cv2.cvtColor(plate_bgr, cv2.COLOR_BGR2HSV)

    # Blue range (broad for washed-out and saturated)
    lower = np.array([80, 30, 30], dtype=np.uint8)
    upper = np.array([145, 255, 255], dtype=np.uint8)
    blue = cv2.inRange(hsv, lower, upper)

    # Only analyze top half
    top = blue[:int(0.45 * H), :]
    top = cv2.medianBlur(top, 5)
    top = cv2.morphologyEx(top, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5)))

    cnts, _ = cv2.findContours(top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    y_cut = None

    if cnts:
        cnt = max(cnts, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        area = w*h
        # header tends to be wide and not tiny
        if w > 0.45*W and area > 0.02*(H*W):
            # Never allow blue crop to exceed safe limit
            y_cut = y + h
            # HARD SAFETY: never cut more than top 18%
            y_cut = min(y + h, int(0.18 * H))




    if y_cut is None:
        # fallback (very mild): keep lower ~70%
        y_cut = int(0.22 * H)

    # also remove a bit of bottom to avoid bumper
    cropped = plate_bgr[y_cut:int(0.98*H), :]
    return cropped


# =========================
# 2) Auto-crop the true text band using projection (NO fixed ratio)
# =========================
def auto_text_band(bgr_below_blue):
    """
    Find the densest text rows using black-hat on grayscale + row projection.
    This removes bumper edges and keeps the white band containing digits+letters.
    """
    H, W = bgr_below_blue.shape[:2]
    gray = cv2.cvtColor(bgr_below_blue, cv2.COLOR_BGR2GRAY)

    # Contrast normalize
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
    g = clahe.apply(gray)

    # Black-hat emphasizes dark text on bright background
    k = max(9, (min(H, W)//25) | 1)  # odd-ish
    bh = cv2.morphologyEx(g, cv2.MORPH_BLACKHAT, cv2.getStructuringElement(cv2.MORPH_RECT, (k, k)))

    # Normalize and binarize blackhat to locate rows containing strokes
    bh_blur = cv2.GaussianBlur(bh, (3,3), 0)
    _, bh_bin = cv2.threshold(bh_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Row projection
    row_sum = (bh_bin > 0).sum(axis=1).astype(np.float32)
    row_sum = cv2.GaussianBlur(row_sum.reshape(-1,1), (1, 21), 0).ravel()

    # Find main band by thresholding projection
    thr = 0.25 * row_sum.max()
    rows = np.where(row_sum > thr)[0]

    if len(rows) < 10:
        # fallback: central region (still safer than fixed ratio)
        y0 = int(0.10*H)
        y1 = int(0.90*H)
    else:
        y0 = max(0, int(rows.min() - 0.06 * H))
        y1 = min(H, int(rows.max() + 0.06 * H))


    band = bgr_below_blue[y0:y1, :]


    # After band extraction
    H2 = band.shape[0]
    # Adaptive vertical padding — protect dots
    top_pad = int(0.02 * H2)
    bot_pad = int(0.03 * H2)

    band = band[top_pad:H2 - bot_pad, :]




    # Return also an "enhancement" visualization for Line 1
    enh_vis = bh  # blackhat view
    return band, g, enh_vis


# =========================
# 3) Stroke-preserving binarization (black-hat + Sauvola/adaptive hybrid)
# =========================
def stroke_binarize(band_bgr):
    gray = cv2.cvtColor(band_bgr, cv2.COLOR_BGR2GRAY)

    # CLAHE (preserves thin Arabic strokes)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    g = clahe.apply(gray)

    # Gaussian blur stabilizes thresholding
    g_blur = cv2.GaussianBlur(g, (3,3), 0)

    # --- Primary: OTSU ---
    _, bin1 = cv2.threshold(
        g_blur, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # --- Backup: Adaptive ---
    bin2 = cv2.adaptiveThreshold(
        g, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        31, 7
    )

    # Choose by foreground ratio (CRITICAL)
    def fg_ratio(b): return (b > 0).mean()
    r1, r2 = fg_ratio(bin1), fg_ratio(bin2)

    if 0.04 <= r1 <= 0.35:
        chosen = bin1
    else:
        chosen = bin2

    # Black-hat ONLY for visualization (Line 1)
    k = max(9, (min(g.shape)//22) | 1)
    bh = cv2.morphologyEx(
        g, cv2.MORPH_BLACKHAT,
        cv2.getStructuringElement(cv2.MORPH_RECT, (k, k))
    )

    return g, chosen, bh




# =========================
# 4) Clean mask (Arabic-aware: keep thin strokes, remove bumper noise)
# =========================
def clean_mask(bin_u8):
    """
    Arabic-aware cleaning:
    - Preserve dots (ن، ق، ي)
    - Preserve internal holes (و، ص، ه)
    - Remove only true noise
    """
    H, W = bin_u8.shape

    # Binary → bool
    m = (bin_u8 > 0)

    # --- 1) REMOVE ONLY VERY TINY SPECKLES ---
    # Dots are small but NOT microscopic
    # Preserve Arabic dots explicitly
    # Allow Arabic dots (very small but valid)
    min_area = max(2, int(0.000015 * H * W))

    m = remove_small_objects(m, min_size=min_area)

    # --- 2) VERY LIGHT CLOSE (ONLY reconnect broken strokes) ---
    m8 = (m.astype(np.uint8) * 255)
    m8 = cv2.morphologyEx(
        m8,
        cv2.MORPH_CLOSE,
        cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)),
        iterations=1
    )

    # --- 3) DO NOT globally fill holes ---
    # Instead: fill only MICRO pinholes caused by thresholding
    m = (m8 > 0)
    m = remove_small_holes(
        m,
        area_threshold=max(8, int(0.00005 * H * W))
    )

    return (m.astype(np.uint8) * 255)



# =========================
# 5) Connected components + split merged blobs
# =========================
def split_merged_by_valleys(mask_u8, box, max_splits=2):
    """
    Split a too-wide component using vertical projection valleys.
    Returns a list of sub-boxes in original coordinates.
    """
    x, y, w, h = box
    crop = mask_u8[y:y+h, x:x+w]
    if w < 10:
        return [box]

    proj = (crop > 0).sum(axis=0).astype(np.float32)
    proj = cv2.GaussianBlur(proj.reshape(1,-1), (max(5, (w//10)|1), 1), 0).ravel()

    # valley threshold relative to max stroke density
    thr = 0.22 * proj.max()
    valleys = np.where(proj < thr)[0]

    cuts = []
    if len(valleys) > 0:
        groups = np.split(valleys, np.where(np.diff(valleys) > 3)[0] + 1)
        for g in groups:
            c = int(np.median(g))
            if 0.15*w < c < 0.85*w:
                cuts.append(c)

    cuts = sorted(cuts)
    if len(cuts) == 0:
        return [box]

    # limit number of splits
    cuts = cuts[:max_splits]

    xs = [0] + cuts + [w]
    out = []
    for i in range(len(xs)-1):
        xx0, xx1 = xs[i], xs[i+1]
        ww = xx1 - xx0
        if ww < max(6, int(0.06*w)):  # too thin
            continue
        out.append((x+xx0, y, ww, h))

    return out if len(out) >= 2 else [box]


def extract_all_char_boxes(clean_u8):
    """
    Extract candidate character boxes from the whole band (digits+letters together).
    Apply Arabic-aware filters and split merged boxes.
    """
    H, W = clean_u8.shape
    num, labels, stats, _ = cv2.connectedComponentsWithStats((clean_u8 > 0).astype(np.uint8), connectivity=8)

    boxes = []
    for i in range(1, num):
        x, y, w, h, area = stats[i]

        # reject tiny
        # Never kill potential dots / hamza
        if area < max(6, int(0.00004 * H * W)):
            continue



        # reject flat blobs EXCEPT tall thin strokes (Arabic ١)
        if h < 0.18 * H:
            aspect = h / max(w, 1)
            if aspect < 3.0:   # allow tall thin digits like ١
                continue


        # reject huge regions
        if h > 0.98 * H or w > 0.80 * W:
            continue


        # Reject tall-thin vertical straps (plate divider / reflection)
        aspect = h / max(w, 1)
        # Kill plate dividers / emboss lines
        if aspect > 3.8 and w < 0.10 * W and area < 0.12 * H * H:
            continue





        # reject border-touching long lines (common bumper edges)
        if (y <= 1 or y+h >= H-1) and (w > 0.35*W) and (h < 0.45*H):
            continue


        # Reject left/right frame borders
        edge_margin = int(0.04 * W)
        if (x < edge_margin or x + w > W - edge_margin) and h > 0.50 * H:
            continue






        boxes.append((x, y, w, h))

    # Sort left->right
    boxes = sorted(boxes, key=lambda b: b[0])

    # Split wide boxes (merged characters)
    out = []
    for b in boxes:
        x,y,w,h = b
        if w > 0.22 * W:
            out.extend(split_merged_by_valleys(clean_u8, b, max_splits=2))
        else:
            out.append(b)

    out = sorted(out, key=lambda b: b[0])


    # Remove near-duplicate overlapping boxes
    filtered = []
    for b in out:
        x,y,w,h = b
        keep = True
        for bb in filtered:
            xx,yy,ww,hh = bb
            iou_w = min(x+w, xx+ww) - max(x, xx)
            iou_h = min(y+h, yy+hh) - max(y, yy)
            if iou_w > 0 and iou_h > 0:
                inter = iou_w * iou_h
                if inter > 0.6 * min(w*h, ww*hh):
                    keep = False
                    break
        if keep:
            filtered.append(b)

    out = filtered


    ##

    

    # --- Merge small dot boxes with nearest base character ---
    merged = []
    used = [False] * len(out)

    for i, b in enumerate(out):
        if used[i]:
            continue
        x, y, w, h = b
        area = w * h

        # candidate dot
        if area < 0.06 * H * H:
            cx = x + w / 2
            cy = y + h / 2
            best_j, best_d = None, 1e9

            for j, bb in enumerate(out):
                if i == j or used[j]:
                    continue
                x2, y2, w2, h2 = bb
                if h2 > h * 2:  # likely base letter
                    cx2 = x2 + w2 / 2
                    cy2 = y2 + h2 / 2
                    d = abs(cx - cx2) + abs(cy - cy2)
                    if d < best_d:
                        best_d = d
                        best_j = j

            # Prefer vertical proximity (dots are ABOVE base in Arabic)
            # Arabic dots: above base, close vertically, limited horizontal drift
            if (
                best_j is not None
                and cy < cy2                       # dot ABOVE base
                and abs(cx - cx2) < 0.12 * W       # tight horizontal lock
                and abs(cy - cy2) < 0.18 * H       # tighter vertical range
            ):
                # merge dot into base box
                bx, by, bw, bh = out[best_j]
                nx = min(x, bx)
                ny = min(y, by)
                nw = max(x + w, bx + bw) - nx
                nh = max(y + h, by + bh) - ny
                out[best_j] = (nx, ny, nw, nh)
                used[i] = True
                continue

        merged.append(b)
        used[i] = True

    out = merged



    # Keep at most 7 chars by area (since plates are 1..4 digits + 1..3 letters)
    if len(out) > 7:
        out = sorted(out, key=lambda b: b[2]*b[3], reverse=True)[:7]
        out = sorted(out, key=lambda b: b[0])



    return out


# =========================
# 6) Split all characters into digits vs letters WITHOUT the separator
# =========================
def split_digits_letters_by_gap(boxes, W):
    """
    Robust split using the largest gap in x-centers with plate constraints:
    - digits count: 1..4 (LEFT)
    - letters count: 1..3 (RIGHT)
    We pick the split that maximizes inter-group gap while satisfying constraints.
    """
    if len(boxes) < 2:
        return boxes, []

    centers = np.array([b[0] + b[2]/2 for b in boxes], dtype=np.float32)

    # Candidate splits i: left = [0..i], right = [i+1..]
    best = None
    for i in range(len(boxes)-1):
        left_n = i+1
        right_n = len(boxes) - left_n
        if not (1 <= left_n <= 4):  # digits constraint
            continue
        if not (1 <= right_n <= 3):  # letters constraint
            continue

        gap = centers[i+1] - centers[i]
        # prefer splits near middle-ish, but gap is main driver
        score = gap

        if best is None or score > best[0]:
            best = (score, i)

    if best is None:
        # fallback: force a feasible split by best gap ignoring constraints
        gaps = centers[1:] - centers[:-1]
        i = int(np.argmax(gaps))
        return boxes[:i+1], boxes[i+1:]

    i = best[1]
    
    left = boxes[:i+1]
    right = boxes[i+1:]

    # Digits usually TALLER than letters
    lh = np.mean([b[3] for b in left])
    rh = np.mean([b[3] for b in right])

    # Prefer taller group as digits
    if lh < rh:
        left, right = right, left


    return left, right



# =========================
# 7) Extract character images from boxes
# =========================
def extract_chars_from_boxes(clean_u8, boxes):
    """
    Returns list of normalized (150x150) binary chars.
    """
    H, W = clean_u8.shape
    chars = []
    for (x,y,w,h) in boxes:
        pad = max(1, int(0.08*h))
        x0 = max(0, x-pad); y0 = max(0, y-pad)
        x1 = min(W, x+w+pad); y1 = min(H, y+h+pad)

        c = clean_u8[y0:y1, x0:x1]
        c = tight_crop_from_mask(c, c)
        c = deskew_bin_char(c)
        c = tight_crop_from_mask(c, c)

        c = pad_to_square(c, 0)
        c = cv2.resize(c, (150,150), interpolation=cv2.INTER_AREA)

        chars.append(c)
    return chars


# =========================
# 8) Drawing helpers
# =========================
def draw_boxes_on_rgb(bgr, boxes, color=(0,255,0), thickness=2):
    rgb = bgr_to_rgb(bgr.copy())
    for (x,y,w,h) in boxes:
        cv2.rectangle(rgb, (x,y), (x+w, y+h), color, thickness)
    return rgb


# =========================
# MAIN LOOP
# =========================
BASE_DIR = "Localization Test/FinalCrops"
paths = sorted(glob.glob(os.path.join(BASE_DIR, "*.*")))
if len(paths) == 0:
    raise FileNotFoundError(f"No images found in: {BASE_DIR}")

for idx, p in enumerate(paths, 1):
    img_bgr = cv2.imread(p)
    if img_bgr is None:
        continue

    # ---- Stage A: remove blue header
    below_blue = remove_blue_header(img_bgr)

    # ---- Stage B: auto-crop true text band using projection
    band_bgr, norm_gray_vis, enh_vis = auto_text_band(below_blue)

    # ---- Stage C: binarize (stroke-preserving)
    gray, bin_u8, bh = stroke_binarize(band_bgr)

    # ---- Stage D: clean
    clean_u8 = clean_mask(bin_u8)

    # ---- Stage E: CC -> all boxes
    all_boxes = extract_all_char_boxes(clean_u8)

    # ---- Stage F: split to digits/letters by largest x-gap under constraints
    d_boxes, l_boxes = split_digits_letters_by_gap(all_boxes, clean_u8.shape[1])

    # ---- Letters should be displayed right->left (Arabic reading),
    # but we keep their boxes in x-order for extraction, then reverse for display.
    digits_chars = extract_chars_from_boxes(clean_u8, d_boxes)
    letters_chars = extract_chars_from_boxes(clean_u8, l_boxes)

    # reverse letters display order (right->left)
    if len(l_boxes) > 1:
        letters_chars = letters_chars[::-1]
        l_boxes_disp = l_boxes[::-1]
    else:
        l_boxes_disp = l_boxes

    # ---- Overlays
    overlay_all = draw_boxes_on_rgb(band_bgr, all_boxes, color=(0,255,0), thickness=2)
    overlay_digits = draw_boxes_on_rgb(band_bgr, d_boxes, color=(0,255,0), thickness=2)
    overlay_letters = draw_boxes_on_rgb(band_bgr, l_boxes, color=(0,255,0), thickness=2)

    # =========================
    # LINE 1: Original | Below-blue | Text-band | Enhancement
    # =========================
    # Enhancement view: blackhat from the band (text emphasis)
    show_images(
        [
            bgr_to_rgb(img_bgr),
            bgr_to_rgb(below_blue),
            bgr_to_rgb(band_bgr),
            bh
        ],
        [
            f"[{idx}] Original ({os.path.basename(p)})",
            "Below Blue Header",
            "Auto Text Band",
            "Enhancement (Black-hat)"
        ]
    )

    # =========================
    # LINE 2: Gray | Binary | Clean | Boxes (ALL)
    # =========================
    show_images(
        [
            gray,
            bin_u8,
            clean_u8,
            overlay_all
        ],
        [
            "Band Gray (CLAHE)",
            "Binary (stroke-preserving)",
            "Clean Mask",
            "Boxes (ALL chars)"
        ]
    )

    # =========================
    # LINE 3: Digits boxes | Letters boxes | Digits... | Letters...
    # =========================
    line3_images = [overlay_digits, overlay_letters] + digits_chars + letters_chars
    line3_titles = (
        ["Digits Boxes (LEFT group)", "Letters Boxes (RIGHT group)"]
        + [f"Digit {i+1}" for i in range(len(digits_chars))]
        + [f"Letter {i+1}" for i in range(len(letters_chars))]
    )

    # If nothing detected, still show overlays only (avoid empty show_images)
    if len(line3_images) == 0:
        line3_images = [overlay_all]
        line3_titles = ["No characters detected"]

    show_images(line3_images, line3_titles)



'''
MAAW LABUBU
'''

In [ ]:
'''
MAAW PLATE CROP
'''


import cv2
import numpy as np

def extract_clean_plate(img_bgr):
    """
    Input: BGR image
    Output: CLEAN tightly-cropped plate image (BGR)
    """

    # ---------- 1. PREPROCESS ----------
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Reduce noise but keep edges
    gray = cv2.bilateralFilter(gray, 11, 17, 17)

    # ---------- 2. EDGE DETECTION ----------
    edges = cv2.Canny(gray, 80, 200)

    # ---------- 3. MORPHOLOGY (JOIN PLATE EDGES) ----------
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 5))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    closed = cv2.dilate(closed, None, iterations=2)
    closed = cv2.erode(closed, None, iterations=1)

    # ---------- 4. FIND CONTOURS ----------
    cnts, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts) == 0:
        return None

    H, W = gray.shape

    best_cnt = None
    best_score = 0

    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        area = w * h
        aspect = w / float(h)

        # Egyptian plate geometry constraints
        if area < 0.01 * H * W:
            continue
        if not (2.0 < aspect < 6.0):
            continue

        score = area  # prioritize largest valid rectangle
        if score > best_score:
            best_score = score
            best_cnt = (x, y, w, h)

    if best_cnt is None:
        return None

    x, y, w, h = best_cnt

    # ---------- 5. SAFE TIGHT CROP ----------
    pad = int(0.03 * max(w, h))
    x0 = max(0, x - pad)
    y0 = max(0, y - pad)
    x1 = min(W, x + w + pad)
    y1 = min(H, y + h + pad)

    plate = img_bgr[y0:y1, x0:x1]

    return plate



import matplotlib.pyplot as plt

plates = []
titles = []




'''
sample_images = [
    globals()[f"cropped_Plate{i}"]
    for i in range(1, 40)
    if f"cropped_Plate{i}" in globals()
]
'''

imim1 = io.imread('maaw/image1.png')
imim2 = io.imread('maaw/image2.png')
imim3 = io.imread('maaw/image3.png')
imim4 = io.imread('maaw/image4.png')
imim5 = io.imread('maaw/image5.png')


sample_images = [imim1, imim2, imim3, imim4, imim5]




'''
for i, img in enumerate(sample_images):  # sample_images = list of your inputs
    plate = extract_clean_plate(img)
    if plate is not None:
        plates.append(plate)
        titles.append(f"Plate {i+1}")

show_images(plates, titles)
'''
for i, img in enumerate(sample_images):
    plate = extract_clean_plate(img)

    if plate is None:
        continue

    show_images(
        images=[img, plate],
        titles=[f"Input {i+1}", f"Plate {i+1}"]
    )




'''
MAAW PLATE CROP
'''

In [ ]:
'''
FORTNITE SEGMENT
'''


"""
FORT SEGMENT (Robust)
- Shows BIG boxes (digits / letters) first
- Splits into individual characters
- Produces tight-cropped characters (no internal frames/bars)
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt


# ============================================================
# VISUALIZATION
# ============================================================
def show_images(images, titles=None, figsize_scale=4):
    n = len(images)
    if titles is None:
        titles = [f"({i})" for i in range(n)]
    plt.figure(figsize=(figsize_scale * n, figsize_scale))
    for i, (img, t) in enumerate(zip(images, titles)):
        plt.subplot(1, n, i + 1)
        if img.ndim == 2:
            plt.imshow(img, cmap="gray")
        else:
            plt.imshow(img)
        plt.title(t)
        plt.axis("off")
    plt.show()


# ============================================================
# 1) PLATE ROI EXTRACTION (ROBUST)
# ============================================================
def extract_plate_roi(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 9, 60, 60)

    edges = cv2.Canny(gray, 80, 200)
    edges = cv2.dilate(edges, cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)))

    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return img_bgr

    cnt = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)

    pad = int(0.03 * max(w, h))
    H, W = img_bgr.shape[:2]

    x0 = max(0, x - pad)
    y0 = max(0, y - pad)
    x1 = min(W, x + w + pad)
    y1 = min(H, y + h + pad)

    return img_bgr[y0:y1, x0:x1]


# ============================================================
# 2) CHARACTER BAND
# ============================================================
def crop_character_band(plate):
    H = plate.shape[0]
    return plate[int(0.35 * H):int(0.93 * H), :]


# ============================================================
# 3) BINARIZATION (ARABIC SAFE) + (optional) light border cleanup
# ============================================================
def binarize_clean(region):
    gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(2.5, (8, 8))
    gray = clahe.apply(gray)
    gray = cv2.medianBlur(gray, 5)

    _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Gentle close: connect strokes but don't destroy holes too much
    bw = cv2.morphologyEx(
        bw, cv2.MORPH_CLOSE,
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)),
        iterations=1
    )

    # Gentle open: remove pepper noise
    bw = cv2.morphologyEx(
        bw, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)),
        iterations=1
    )

    return bw


def clean_digit(ch):
    bin_ch = (ch > 0).astype(np.uint8)
    num, labels, stats, _ = cv2.connectedComponentsWithStats(bin_ch, 8)

    if num <= 1:
        return ch

    # Keep ONLY the largest component
    areas = stats[1:, cv2.CC_STAT_AREA]
    best = 1 + np.argmax(areas)

    out = np.zeros_like(ch)
    out[labels == best] = 255

    # Optional light cleanup to remove tiny edge noise
    out = cv2.morphologyEx(
        out,
        cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)),
        iterations=1
    )

    return out




def clean_letter(ch):
    bin_ch = (ch > 0).astype(np.uint8)
    num, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_ch, 8)

    if num <= 1:
        return ch

    H, W = ch.shape

    comps = []
    for i in range(1, num):
        x, y, w, h, area = stats[i]
        cx, cy = centroids[i]

        touches_border = (
            x == 0 or y == 0 or
            x + w == W or y + h == H
        )

        comps.append({
            "i": i,
            "x": x, "y": y, "w": w, "h": h,
            "cx": cx, "cy": cy,
            "area": area,
            "touch": touches_border
        })

    # === MAIN BODY ===
    body = max(
        [c for c in comps if not c["touch"]],
        key=lambda c: c["area"],
        default=max(comps, key=lambda c: c["area"])
    )

    out = np.zeros_like(ch)
    out[labels == body["i"]] = 255

    body_bottom = body["y"] + body["h"]

    # === ATTACH DOTS + TAILS ===
    for c in comps:
        if c["i"] == body["i"]:
            continue

        dx = abs(c["cx"] - body["cx"])
        dy = c["cy"] - body_bottom

        # ----- DOTS -----
        if c["area"] < 0.4 * body["area"]:
            if dx < 0.7 * W and abs(c["cy"] - body["cy"]) < 0.9 * H:
                out[labels == c["i"]] = 255

        # ----- TAIL (ج) -----
        # below body, vertically aligned, thin stroke
        if (
            dy > 0 and
            dy < 0.6 * H and
            dx < 0.25 * W and
            c["area"] > 0.12 * body["area"]
        ):
            out[labels == c["i"]] = 255

    return out






# ============================================================
# 4) CONNECTED COMPONENTS (for initial rough detection)
# ============================================================
def get_components(bw, min_area=40):
    num, lab, stats, _ = cv2.connectedComponentsWithStats((bw > 0).astype(np.uint8), 8)
    comps = []
    H, W = bw.shape

    for i in range(1, num):
        x, y, w, h, area = stats[i]
        if area < min_area:
            continue
        # avoid taking the whole BW as a component
        if w > 0.98 * W or h > 0.98 * H:
            continue
        comps.append([x, y, x + w, y + h, area])
    return comps


# ============================================================
# 5) MERGE DOTS (ج ، ق ، ف) into nearest body
# ============================================================
def attach_dots(boxes):
    big, dots = [], []
    for b in boxes:
        x0, y0, x1, y1, a = b
        if a < 600 and (x1 - x0) < 45 and (y1 - y0) < 45:
            dots.append(b)
        else:
            big.append(b)

    for dx0, dy0, dx1, dy1, _ in dots:
        dcx = (dx0 + dx1) / 2
        dcy = (dy0 + dy1) / 2

        best = None
        best_d = 1e18
        for i, (x0, y0, x1, y1, a) in enumerate(big):
            bx = (x0 + x1) / 2
            by = (y0 + y1) / 2
            d = (dcx - bx) ** 2 + 0.7 * (dcy - by) ** 2
            if d < best_d and abs(dcx - bx) < 50:
                best_d = d
                best = i

        if best is not None:
            big[best][0] = min(big[best][0], dx0)
            big[best][1] = min(big[best][1], dy0)
            big[best][2] = max(big[best][2], dx1)
            big[best][3] = max(big[best][3], dy1)

    return big


# ============================================================
# 6) CLASSIFY by POSITION (Egyptian plate: digits left, letters right)
#    This is much more stable across your samples.
# ============================================================
def split_digits_letters_by_position(boxes, bw_width):
    mid = bw_width / 2.0
    digits, letters = [], []

    for x0, y0, x1, y1, a in boxes:
        cx = (x0 + x1) / 2.0
        if cx < mid:
            digits.append((x0, y0, x1, y1))
        else:
            letters.append((x0, y0, x1, y1))

    # sort for display / later
    digits = sorted(digits, key=lambda b: b[0])                 # L->R
    letters = sorted(letters, key=lambda b: b[0], reverse=True) # R->L
    return digits, letters


# ============================================================
# 7) UNION BOX helper
# ============================================================
def union_box(boxes):
    if not boxes:
        return None
    x0 = min(b[0] for b in boxes)
    y0 = min(b[1] for b in boxes)
    x1 = max(b[2] for b in boxes)
    y1 = max(b[3] for b in boxes)
    return (x0, y0, x1, y1)


# ============================================================
# 8) SPLIT a GROUP BOX into characters using vertical projection
#    - Works for both digits and letters
#    - Includes forced-cut fallback for "سى" type merges
# ============================================================
def split_group_into_chars(bw, group_box, min_width_px=10, valley_ratio=0.18, force_cut=True):
    x0, y0, x1, y1 = group_box
    crop = bw[y0:y1, x0:x1]
    H, W = crop.shape

    if W < 25:
        return [group_box]

    proj = np.sum(crop > 0, axis=0).astype(np.float32)

    win = max(5, W // 18)
    proj_s = np.convolve(proj, np.ones(win) / win, mode="same")

    thresh = valley_ratio * (np.percentile(proj_s, 90))
    gaps = proj_s < thresh

    segments = []
    start = None
    for i in range(W):
        if not gaps[i] and start is None:
            start = i
        if gaps[i] and start is not None:
            if i - start >= min_width_px:
                segments.append((start, i))
            start = None
    if start is not None and (W - start) >= min_width_px:
        segments.append((start, W))


    
    # Normal case: multiple segments found
    if len(segments) >= 2:
        return [(x0 + s, y0, x0 + e, y1) for (s, e) in segments]
    



    # Fallback: force a cut for wide merged group (like سى)
    if force_cut and (W > 1.25 * H):
        L = int(0.35 * W)
        R = int(0.65 * W)
        cut = L + int(np.argmin(proj_s[L:R]))

        if 10 < cut < W - 10:
            left = (x0, y0, x0 + cut, y1)
            right = (x0 + cut, y0, x1, y1)

            lp = np.sum(crop[:, :cut] > 0)
            rp = np.sum(crop[:, cut:] > 0)
            total = np.sum(crop > 0) + 1e-9

            if lp > 0.12 * total and rp > 0.12 * total:
                return [left, right]

    return [group_box]


# ============================================================
# 9) CHARACTER CLEANING:
#    Keep the largest component that DOES NOT touch the crop border.
#    This removes the "frame bars" and random border blobs.
# ============================================================
def best_component_smart(ch):
    bin_ch = (ch > 0).astype(np.uint8)
    num, labels, stats, _ = cv2.connectedComponentsWithStats(bin_ch, 8)

    if num <= 1:
        return ch

    H, W = ch.shape

    components = []
    for lab in range(1, num):
        x, y, w, h, area = stats[lab]
        cx = x + w / 2
        cy = y + h / 2

        # border-touch check (frame / bars)
        touches_border = (
            x == 0 or y == 0 or
            x + w == W or y + h == H
        )

        components.append({
            "lab": lab,
            "area": area,
            "cx": cx,
            "cy": cy,
            "touches_border": touches_border
        })

    # 1) find MAIN body (largest non-border component)
    bodies = [c for c in components if not c["touches_border"]]
    if not bodies:
        bodies = components

    main = max(bodies, key=lambda c: c["area"])

    out = np.zeros_like(ch)
    out[labels == main["lab"]] = 255

    # 2) attach DOTS near main body
    for c in components:
        if c["lab"] == main["lab"]:
            continue

        # dot heuristic
        if c["area"] < 0.35 * main["area"]:
            dx = abs(c["cx"] - main["cx"])
            dy = abs(c["cy"] - main["cy"])

            if dx < 0.6 * W and dy < 0.8 * H:
                out[labels == c["lab"]] = 255

    return out




def tight_crop(ch):
    ys, xs = np.where(ch > 0)
    if len(xs) == 0:
        return ch
    return ch[ys.min():ys.max() + 1, xs.min():xs.max() + 1]


def normalize_to_square(ch, out_size=64, pad=4):
    ch = tight_crop(ch)
    h, w = ch.shape
    if h == 0 or w == 0:
        return np.zeros((out_size, out_size), dtype=np.uint8)

    scale = (out_size - 2 * pad) / max(h, w)
    nh, nw = max(1, int(h * scale)), max(1, int(w * scale))
    ch_rs = cv2.resize(ch, (nw, nh), interpolation=cv2.INTER_AREA)

    canvas = np.zeros((out_size, out_size), dtype=np.uint8)
    y0 = (out_size - nh) // 2
    x0 = (out_size - nw) // 2
    canvas[y0:y0 + nh, x0:x0 + nw] = ch_rs
    return canvas


def extract_chars_tight_and_norm(bw, boxes, kind, pad_box=4, out_size=64):
    tight_list = []
    norm_list = []
    H, W = bw.shape

    for (x0, y0, x1, y1) in boxes:
        x0p = max(0, x0 - pad_box)
        y0p = max(0, y0 - pad_box)
        x1p = min(W, x1 + pad_box)
        y1p = min(H, y1 + pad_box)

        ch = bw[y0p:y1p, x0p:x1p].copy()

        if kind == "digit":
            ch2 = clean_digit(ch)
        else:
            ch2 = clean_letter(ch)

        ch2 = tight_crop(ch2)

        tight_list.append(ch2)
        norm_list.append(normalize_to_square(ch2, out_size=out_size))

    return tight_list, norm_list



# ============================================================
# 10) MAIN PIPELINE
# ============================================================
def split_iteratively(bw, boxes, max_iters=3):
    out = boxes[:]
    for _ in range(max_iters):
        new = []
        changed = False
        for b in out:
            parts = split_group_into_chars(bw, b, force_cut=False)
            if len(parts) > 1:
                changed = True
                new.extend(parts)
            else:
                new.append(b)
        out = new
        if not changed:
            break
    return out





def segment_egypt_plate_chars(img_bgr, debug=True):
    plate = extract_plate_roi(img_bgr)
    band = crop_character_band(plate)
    bw = binarize_clean(band)

    comps = get_components(bw, min_area=40)
    #comps = attach_dots(comps)
    digits_tmp, letters_tmp = split_digits_letters_by_position(comps, bw.shape[1])

    letters_tmp = attach_dots([
        [b[0], b[1], b[2], b[3], (b[2]-b[0])*(b[3]-b[1])]
        for b in letters_tmp
    ])


    # rebuild comps
    comps = [(x0,y0,x1,y1,(x1-x0)*(y1-y0)) for (x0,y0,x1,y1) in digits_tmp] + letters_tmp







    digits_raw, letters_raw = split_digits_letters_by_position(comps, bw.shape[1])

    # BIG BOXES (what you want to show first)
    big_digit_box = union_box(digits_raw)
    big_letter_box = union_box(letters_raw)

    # If one side is empty due to weird thresholding, fallback to using position-based halves
    if big_digit_box is None:
        big_digit_box = (0, 0, bw.shape[1] // 2, bw.shape[0])
    if big_letter_box is None:
        big_letter_box = (bw.shape[1] // 2, 0, bw.shape[1], bw.shape[0])

    # Split each BIG box into individual characters
    digits_boxes = split_iteratively(bw, [big_digit_box])
    letters_boxes = split_group_into_chars(bw, big_letter_box, min_width_px=10, valley_ratio=0.18, force_cut=True)

    
    # Sort for reading
    digits_boxes = sorted(digits_boxes, key=lambda b: b[0])                 # L->R
    letters_boxes = sorted(letters_boxes, key=lambda b: b[0], reverse=True) # R->L

    # Extract tight + normalized
    digits_tight, digits_norm = extract_chars_tight_and_norm(
        bw, digits_boxes, kind="digit", pad_box=5
    )

    letters_tight, letters_norm = extract_chars_tight_and_norm(
        bw, letters_boxes, kind="letter", pad_box=5
    )



    if debug:
        # View 1: pipeline overview + BIG boxes
        vis_groups = cv2.cvtColor(bw, cv2.COLOR_GRAY2RGB)

        x0, y0, x1, y1 = big_digit_box
        cv2.rectangle(vis_groups, (x0, y0), (x1, y1), (255, 0, 0), 3)

        x0, y0, x1, y1 = big_letter_box
        cv2.rectangle(vis_groups, (x0, y0), (x1, y1), (0, 255, 0), 3)

        show_images(
            [
                cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB),
                cv2.cvtColor(plate, cv2.COLOR_BGR2RGB),
                cv2.cvtColor(band, cv2.COLOR_BGR2RGB),
                bw,
                vis_groups,
            ],
            ["Input", "Plate ROI", "Character Band", "BW", "BIG Boxes: Digits (Blue) / Letters (Green)"],
            figsize_scale=4,
        )

        # View 2: individual boxes after split
        vis_split = cv2.cvtColor(bw, cv2.COLOR_GRAY2RGB)
        for x0, y0, x1, y1 in digits_boxes:
            cv2.rectangle(vis_split, (x0, y0), (x1, y1), (255, 0, 0), 2)
        for x0, y0, x1, y1 in letters_boxes:
            cv2.rectangle(vis_split, (x0, y0), (x1, y1), (0, 255, 0), 2)

        '''
        show_images([vis_split], ["AFTER SPLIT: Individual Boxes"], figsize_scale=6)
  
        # View 3: tight crops (this is what you want visually)
        if digits_tight:
            show_images(digits_tight, [f"Digit {i+1}" for i in range(len(digits_tight))], figsize_scale=4)
        if letters_tight:
            show_images(letters_tight, [f"Letter {i+1}" for i in range(len(letters_tight))], figsize_scale=4)
        '''


        # ------------------------------------------------------------
        # COMBINED ROW:
        # [ AFTER SPLIT | Digits... | Letters... ]
        # ------------------------------------------------------------

        row2_images = []
        row2_titles = []

        # 1) AFTER SPLIT visualization
        row2_images.append(vis_split)
        row2_titles.append("AFTER SPLIT")

        # 2) Digits (left → right)
        for i, d in enumerate(digits_tight):
            row2_images.append(d)
            row2_titles.append(f"Digit {i+1}")

        # 3) Letters (right → left already sorted)
        for i, l in enumerate(letters_tight):
            row2_images.append(l)
            row2_titles.append(f"Letter {i+1}")

        show_images(
            row2_images,
            row2_titles,
            figsize_scale=3
        )



    # Return both (tight for viewing, norm for recognition)
    return {
        "bw": bw,
        "digits_boxes": digits_boxes,
        "letters_boxes": letters_boxes,
        "digits_tight": digits_tight,
        "letters_tight": letters_tight,
        "digits_norm64": digits_norm,
        "letters_norm64": letters_norm,
    }


# =========================
# RUN (single sample)
# =========================
# img_bgr = cropped_Plate4   # example
# out = segment_egypt_plate_chars(img_bgr, debug=True)

# =========================
# RUN (all samples)
# =========================
# for i in range(1, 40):
#     name = f"cropped_Plate{i}"
#     if name in globals():
#         print("====", name, "====")
#         segment_egypt_plate_chars(globals()[name], debug=True)





#images = [cropped_Plate4, cropped_Plate7, cropped_Plate8, cropped_Plate10, cropped_Plate12, cropped_Plate5, cropped_Plate9, cropped_Plate11]
#images = [cropped_Plate1, cropped_Plate2, cropped_Plate3, cropped_Plate4, cropped_Plate5, cropped_Plate6, cropped_Plate7, cropped_Plate8, cropped_Plate9, cropped_Plate10, cropped_Plate11, cropped_Plate12, cropped_Plate13, cropped_Plate14, cropped_Plate15, cropped_Plate16, cropped_Plate17, cropped_Plate18, cropped_Plate19, cropped_Plate20, cropped_Plate21, cropped_Plate22, cropped_Plate23, cropped_Plate24, cropped_Plate25, cropped_Plate26, cropped_Plate27, cropped_Plate28, cropped_Plate29, cropped_Plate30, cropped_Plate31, cropped_Plate32, cropped_Plate33, cropped_Plate34, cropped_Plate37, cropped_Plate38, cropped_Plate39]

'''
images = [
    globals()[f"cropped_Plate{i}"]
    for i in range(1, 40)
    if f"cropped_Plate{i}" in globals()
]




for i, img in enumerate(images, start=1):
    print(f"==== PLATE {i} ====")
    segment_egypt_plate_chars(img, debug=True)

'''


def load_images_from_folder(folder_path):
    images = []
    names = []

    for fname in sorted(os.listdir(folder_path)):
        if fname.lower().endswith((".jpg", ".png", ".jpeg", ".bmp")):
            path = os.path.join(folder_path, fname)
            img = cv2.imread(path)
            if img is not None:
                images.append(img)
                names.append(fname)

    return images, names

# =========================
# RUN (from folder)
# =========================

folder = r"Localization Test/FinalCrops"   # adjust if needed

images, names = load_images_from_folder(folder)

for img, name in zip(images, names):
    print(f"==== {name} ====")
    segment_egypt_plate_chars(img, debug=True)









# =========================
# RUN
# =========================
####img_bgr = cropped_Plate12   # change

#work: 4, 7, 8, 10, 12
#dont: 5, 9, 11
#digits, letters = segment_egypt_plate_chars(img_bgr, debug=True)
####out = segment_egypt_plate_chars(img_bgr, debug=True)

# Run all samples if you have cropped_Plate1..cropped_Plate39 in globals()
# for i in range(1, 40):
#     name = f"cropped_Plate{i}"
#     if name in globals():
#         print("====", name, "====")
#         segment_egypt_plate_chars(globals()[name], debug=False)



'''
FORTNITE SEGMENT
'''

In [ ]:
'''
MAAW NEW SEG
'''


import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
import os
import glob


def extract_plate(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 80, 200)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    cnts, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(cnts, key=cv2.contourArea)

    x,y,w,h = cv2.boundingRect(cnt)
    return img[y:y+h, x:x+w]


def crop_digits_letters_region(plate):
    h, w = plate.shape[:2]

    # Egyptian plate layout: bottom ~65% contains digits + letters
    y0 = int(0.35 * h)
    y1 = int(0.95 * h)

    return plate[y0:y1, :]


#

def binarize_for_split(region_bgr):
    gray = cv2.cvtColor(region_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN,
                          cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)))
    return bw


def split_digits_letters(region_bgr):
    H, W = region_bgr.shape[:2]
    bw = binarize_for_split(region_bgr)

    # --- 1) Detect vertical separator line (strongest vertical structure) ---
    # Kernel عمودي طويل يلقط الخط الفاصل
    kH = max(15, int(0.7 * H))
    vert_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, kH))
    vert = cv2.morphologyEx(bw, cv2.MORPH_OPEN, vert_kernel)

    col_strength = np.sum(vert > 0, axis=0).astype(np.float32)

    # نبحث قريب من منتصف اللوحة فقط
    left = int(0.30 * W)
    right = int(0.80 * W)

    x_sep = left + int(np.argmax(col_strength[left:right]))
    sep_strength = col_strength[x_sep]

    # --- 2) Fallback: valley split (لو مفيش خط فاصل واضح) ---
    if sep_strength < 0.20 * H:  # لو الخط ضعيف
        col_sum = np.sum(bw > 0, axis=0).astype(np.float32)
        # smoothing
        win = max(15, W // 30)
        col_sum_s = np.convolve(col_sum, np.ones(win)/win, mode="same")

        left = int(0.35 * W)
        right = int(0.75 * W)
        x_sep = left + int(np.argmin(col_sum_s[left:right]))

    # قص حوالين الفاصل عشان ما يدخلش مع أي جانب
    pad = 4
    x_sep = max(10, min(W - 10, x_sep))

    digits = region_bgr[:, :max(0, x_sep - pad)]
    letters = region_bgr[:, min(W, x_sep + pad):]
    return digits, letters


#





def touches_border(x0, y0, x1, y1, W, H, margin=2):
    return (x0 <= margin) or (y0 <= margin) or (x1 >= W - margin) or (y1 >= H - margin)

#11
def binarize(region_bgr):
    gray = cv2.cvtColor(region_bgr, cv2.COLOR_BGR2GRAY)

    # Contrast boost WITHOUT exploding noise
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)

    # Median blur kills texture noise but keeps strokes
    gray = cv2.medianBlur(gray, 5)

    # Global threshold (stable for plates)
    _, bw = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # Remove pepper noise only
    bw = cv2.morphologyEx(
        bw, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    )

    return bw


def binarize_letters(region_bgr):
    gray = cv2.cvtColor(region_bgr, cv2.COLOR_BGR2GRAY)

    # mild contrast boost (don't overdo it)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)

    # small median (3) keeps dots better than 5
    gray = cv2.medianBlur(gray, 3)

    # OTSU is more stable than adaptive here
    bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    return bw


def build_group_mask_letters(bw):
    # specifically tuned to reconnect ج tail
    bw_g = cv2.morphologyEx(
        bw, cv2.MORPH_CLOSE,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5,3))
    )
    bw_g = cv2.dilate(
        bw_g,
        cv2.getStructuringElement(cv2.MORPH_RECT, (3,5)),
        iterations=1
    )
    return bw_g




def merge_close_letter_boxes(boxes, H):
    """
    Merge boxes that are likely parts of the same Arabic letter (fixes ج split).
    Rule: merge if x-overlap is decent OR x-gap small, and y ranges are close/overlapping.
    """
    if not boxes:
        return boxes

    boxes = sorted(boxes, key=lambda b: b[0])
    merged = [list(boxes[0])]

    for x0,y0,x1,y1 in boxes[1:]:
        px0,py0,px1,py1 = merged[-1]

        # overlaps / gaps
        x_overlap = max(0, min(px1, x1) - max(px0, x0))
        x_gap = max(0, x0 - px1)

        # vertical relation
        y_overlap = max(0, min(py1, y1) - max(py0, y0))
        y_gap = max(0, max(y0 - py1, py0 - y1))

        # thresholds (tuned for plate letters)
        close_in_x = (x_overlap > 0) or (x_gap < 0.06 * (px1 - px0 + x1 - x0))
        close_in_y = (y_overlap > 0) or (y_gap < 0.18 * H)

        if close_in_x and close_in_y:
            merged[-1][0] = min(px0, x0)
            merged[-1][1] = min(py0, y0)
            merged[-1][2] = max(px1, x1)
            merged[-1][3] = max(py1, y1)
        else:
            merged.append([x0,y0,x1,y1])

    return [tuple(b) for b in merged]


def find_letter_boxes_and_dots(bw, bw_group):
    H, W = bw.shape
    labeled = label(bw_group)

    big_boxes = []
    candidate_small = []  # dot candidates

    for r in regionprops(labeled):
        y0, x0, y1, x1 = r.bbox
        h = y1 - y0
        w = x1 - x0
        area = r.area

        if touches_border(x0, y0, x1, y1, W, H, margin=2):
            continue

        # DOTS: keep them generous (so ف dot doesn't vanish)
        if 20 <= area <= 400 and h <= 0.35 * H and w <= 0.35 * W:
            # roughly dot-ish (avoid long strokes)
            ar = w / max(h, 1)
            if 0.4 <= ar <= 2.5:
                candidate_small.append((x0, y0, x1, y1))
                continue

        # LETTER BODY (relaxed so we don't drop ج)
        if area < 120:
            continue
        if h < 0.16 * H:
            continue
        if w < 0.02 * W:
            continue

        big_boxes.append((x0, y0, x1, y1))

    big_boxes = merge_close_letter_boxes(big_boxes, H)
    big_boxes = sorted(big_boxes, key=lambda b: b[0])

    return big_boxes, candidate_small

def recover_missing_fa(letter_boxes, dot_boxes, H):
    """
    If a dot exists with no nearby letter body,
    create a synthetic body box under it (for ف).
    """
    recovered = list(letter_boxes)

    for dx0, dy0, dx1, dy1 in dot_boxes:
        dcy = 0.5 * (dy0 + dy1)

        attached = False
        for x0,y0,x1,y1 in letter_boxes:
            if x0 - 10 <= (dx0+dx1)/2 <= x1 + 10:
                attached = True
                break

        if not attached:
            # Create a virtual body under the dot
            body_h = int(0.35 * H)
            recovered.append((
                dx0 - 10,
                int(dcy),
                dx1 + 10,
                min(H, int(dcy + body_h))
            ))

    return recovered


def attach_dots_to_boxes(letter_boxes, dot_boxes):
    merged = [list(b) for b in letter_boxes]

    for dx0, dy0, dx1, dy1 in dot_boxes:
        dcx = 0.5 * (dx0 + dx1)
        dcy = 0.5 * (dy0 + dy1)

        best_i = None
        best_score = 1e18

        for i, (x0,y0,x1,y1) in enumerate(merged):
            # dot should be horizontally near the letter
            if dcx < x0 - 15 or dcx > x1 + 15:
                continue

            # prefer dots above upper-half of the letter (ف/ق), but don't hard-reject
            # score: distance with extra penalty if dot is too low
            bx = 0.5 * (x0 + x1)
            by = 0.5 * (y0 + y1)

            penalty_low = 0.0
            if dcy > by:  # dot below center is unlikely
                penalty_low = 2000.0

            score = (dcx - bx)**2 + 0.7*(dcy - by)**2 + penalty_low

            if score < best_score:
                best_score = score
                best_i = i

        if best_i is not None:
            merged[best_i][0] = min(merged[best_i][0], dx0)
            merged[best_i][1] = min(merged[best_i][1], dy0)
            merged[best_i][2] = max(merged[best_i][2], dx1)
            merged[best_i][3] = max(merged[best_i][3], dy1)

    return [tuple(b) for b in merged]


def get_components_letters(region_bgr):
    bw = binarize_letters(region_bgr)          # extraction mask (keeps holes/dots)
    bw_group = build_group_mask_letters(bw)    # grouping mask (connects ج)

    big_boxes, dot_boxes = find_letter_boxes_and_dots(bw, bw_group)
    
    boxes = attach_dots_to_boxes(big_boxes, dot_boxes)
    boxes = recover_missing_fa(boxes, dot_boxes, bw.shape[0])

    boxes = merge_close_letter_boxes(sorted(boxes, key=lambda b: b[0]), bw.shape[0])
    boxes = sorted(boxes, key=lambda b: b[0])

    return bw, boxes




def get_letter_boxes_and_dots(bw):
    H, W = bw.shape

    # group mask: connect broken pieces a bit, but don’t destroy holes in the ORIGINAL bw
    bw_group = cv2.dilate(
        bw, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 5)), iterations=1
    )
    # small close to reconnect weak gaps (helps ج) but still mild
    bw_group = cv2.morphologyEx(
        bw_group, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    )

    labeled = label(bw_group)

    big_boxes = []
    small_boxes = []  # dots / small fragments

    for r in regionprops(labeled):
        y0, x0, y1, x1 = r.bbox
        h = y1 - y0
        w = x1 - x0
        area = r.area

        if touches_border(x0, y0, x1, y1, W, H, margin=2):
            continue

        # classify small components (dots)
        is_dot = (
            40 < area < 180 and           # real dots only
            h < 0.22 * H and
            w < 0.22 * W and
            0.6 < (w / max(h, 1)) < 1.6   # circular-ish
        )


        if is_dot:
            small_boxes.append((x0, y0, x1, y1))
            continue


        # relaxed constraints for letters (so ج doesn’t get chopped)
        if area < 120:
            continue
        if h < 0.18 * H:
            continue
        if w < 0.02 * W:
            continue

        big_boxes.append((x0, y0, x1, y1))

    big_boxes = sorted(big_boxes, key=lambda b: b[0])
    return big_boxes, small_boxes



def attach_dots_to_letter_boxes(big_boxes, small_boxes):
    merged = [list(b) for b in big_boxes]

    for sx0, sy0, sx1, sy1 in small_boxes:
        scx = 0.5 * (sx0 + sx1)
        scy = 0.5 * (sy0 + sy1)

        for i, (x0, y0, x1, y1) in enumerate(merged):
            # Dot must be horizontally inside letter
            if not (x0 - 10 <= scx <= x1 + 10):
                continue

            # Dot must be ABOVE letter body (ف, ق)
            if sy1 > y0:
                continue

            # Attach dot
            merged[i][0] = min(merged[i][0], sx0)
            merged[i][1] = min(merged[i][1], sy0)
            merged[i][2] = max(merged[i][2], sx1)
            merged[i][3] = max(merged[i][3], sy1)
            break

    return [tuple(b) for b in merged]













def get_components(region_bgr, kind="digits"):
    if kind == "digits":
        # Preserve thin strokes (important for ٦)
        gray = cv2.cvtColor(region_bgr, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)   # NOT Gaussian
        _, bw = cv2.threshold(
            gray, 0, 255,
            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )
    else:
        bw = binarize(region_bgr)

    H, W = bw.shape

    # --- mask للتجميع فقط ---
    if kind == "letters":
        big_boxes, small_boxes = get_letter_boxes_and_dots(bw)
        boxes = attach_dots_to_letter_boxes(big_boxes, small_boxes)
        boxes = sorted(boxes, key=lambda b: b[0])
        return bw, boxes
    else:
        # digits: ممكن close صغير جدًا لو فيه انقطاع، أو سيبه bw_group=bw
        bw_group = cv2.morphologyEx(
            bw, cv2.MORPH_CLOSE,
            cv2.getStructuringElement(cv2.MORPH_RECT, (3,5))
        )

        min_area = 90        # was too aggressive
        min_h_ratio = 0.22   # allow shorter digits
        min_w_ratio = 0.02
        min_aspect = 0.07

    labeled = label(bw_group)
    boxes = []

    for r in regionprops(labeled):
        if r.area < min_area:
            continue

        y0, x0, y1, x1 = r.bbox
        h = y1 - y0
        w = x1 - x0

        # تجاهل أي شيء بيلمس الإطار (زي إطار اللوحة)
        if touches_border(x0, y0, x1, y1, W, H, margin=2):
            continue

        if h < min_h_ratio * H:
            continue
        if w < min_w_ratio * W:
            continue

        aspect = w / float(h)
        if aspect < min_aspect:
            continue

        boxes.append((x0, y0, x1, y1))

    boxes = sorted(boxes, key=lambda b: b[0])

    # رجّع الاتنين: الأصلي + البوكسات
    return bw, boxes


def extract_chars_from_mask(mask_bw, boxes, reverse=False, pad=2):
    # نقص من الـ mask الأصلي (مش bw_group) علشان holes تفضل موجودة
    boxes = sorted(boxes, key=lambda b: b[0], reverse=reverse)
    chars = []
    H, W = mask_bw.shape

    for x0,y0,x1,y1 in boxes:
        x0 = max(0, x0 - pad); y0 = max(0, y0 - pad)
        x1 = min(W, x1 + pad); y1 = min(H, y1 + pad)
        chars.append(mask_bw[y0:y1, x0:x1])

    return chars

#11


def draw_boxes(mask, boxes):
    out = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    for x0,y0,x1,y1 in boxes:
        cv2.rectangle(out,(x0,y0),(x1,y1),(255,0,0),2)
    return out




def trim_borders(img, px=6):
    h, w = img.shape[:2]
    return img[px:h-px, px:w-px]



#img_bgr = cropped_Plate1
#img_bgr = cropped_Plate2
#img_bgr = cropped_Plate3
#img_bgr = cropped_Plate4 #DONE
#img_bgr = cropped_Plate5
#img_bgr = cropped_Plate6
#img_bgr = cropped_Plate7
#img_bgr = cropped_Plate8
#img_bgr = cropped_Plate9
#img_bgr = cropped_Plate10
#img_bgr = cropped_Plate11
#img_bgr = cropped_Plate12
#img_bgr = cropped_Plate13
#img_bgr = cropped_Plate14
#img_bgr = cropped_Plate15
#img_bgr = cropped_Plate16
#img_bgr = cropped_Plate17
#img_bgr = cropped_Plate18
#img_bgr = cropped_Plate19
#img_bgr = cropped_Plate20
#img_bgr = cropped_Plate21
#img_bgr = cropped_Plate22
#img_bgr = cropped_Plate23
#img_bgr = cropped_Plate24
#img_bgr = cropped_Plate25
#img_bgr = cropped_Plate26
#img_bgr = cropped_Plate27
#img_bgr = cropped_Plate28
#img_bgr = cropped_Plate29
#img_bgr = cropped_Plate30
#img_bgr = cropped_Plate31
#img_bgr = cropped_Plate32
#img_bgr = cropped_Plate33
#img_bgr = cropped_Plate34
#img_bgr = cropped_Plate37
#img_bgr = cropped_Plate38
#img_bgr = cropped_Plate39

##########################################LOOOOOOOOOOP


def process_one_plate(img_bgr, plate_name="plate", save=True, show=True):
    """
    Process ONE Egyptian plate image end-to-end
    """

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    plate = extract_plate(img_bgr)

    dl_region = crop_digits_letters_region(plate)
    dl_region = trim_borders(dl_region, px=4)

    # Split digits / letters
    digits_region, letters_region = split_digits_letters(dl_region)

    # Extract components
    d_mask, d_boxes = get_components(digits_region, kind="digits")
    l_mask, l_boxes = get_components_letters(letters_region)

    # Keep at most 4 digits
    if len(d_boxes) > 4:
        d_boxes = sorted(
            d_boxes,
            key=lambda b: (b[2]-b[0])*(b[3]-b[1]),
            reverse=True
        )[:4]
        d_boxes = sorted(d_boxes, key=lambda b: b[0])

    # Keep at most 3 letters
    if len(l_boxes) > 3:
        l_boxes = sorted(
            l_boxes,
            key=lambda b: (b[2]-b[0])*(b[3]-b[1]),
            reverse=True
        )[:3]
        l_boxes = sorted(l_boxes, key=lambda b: b[0])

    digits  = extract_chars_from_mask(d_mask, d_boxes, reverse=False)
    letters = extract_chars_from_mask(l_mask, l_boxes, reverse=True)

    # ---------- SAVE ----------
    if save:
        BASE_OUT_DIR = "output_chars"
        DIGITS_DIR = os.path.join(BASE_OUT_DIR, "digits", plate_name)
        LETTERS_DIR = os.path.join(BASE_OUT_DIR, "letters", plate_name)

        os.makedirs(DIGITS_DIR, exist_ok=True)
        os.makedirs(LETTERS_DIR, exist_ok=True)

        TARGET_SIZE = (150, 150)

        for i, d in enumerate(digits):
            d_resized = cv2.resize(d, TARGET_SIZE, interpolation=cv2.INTER_AREA)
            cv2.imwrite(os.path.join(DIGITS_DIR, f"digit_{i}.png"), d_resized)

        for i, l in enumerate(letters):
            l_resized = cv2.resize(l, TARGET_SIZE, interpolation=cv2.INTER_AREA)
            cv2.imwrite(os.path.join(LETTERS_DIR, f"letter_{i}.png"), l_resized)

    # ---------- VISUALIZE ----------
    if show:
        d_boxes_img = draw_boxes(d_mask, d_boxes)
        l_boxes_img = draw_boxes(l_mask, l_boxes)

        show_images(
            [
                img_rgb,
                cv2.cvtColor(plate, cv2.COLOR_BGR2RGB),
                cv2.cvtColor(dl_region, cv2.COLOR_BGR2RGB),
                cv2.cvtColor(digits_region, cv2.COLOR_BGR2RGB),
                cv2.cvtColor(letters_region, cv2.COLOR_BGR2RGB),
            ],
            ["Original", "Plate Crop", "Digits+Letters", "Digits Region", "Letters Region"]
        )

        show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
        show_images([d_boxes_img, l_boxes_img], ["Digits Boxes", "Letters Boxes"])

        if digits:
            show_images(digits, [f"Digit {i+1}" for i in range(len(digits))])
        if letters:
            show_images(letters, [f"Letter {i+1}" for i in range(len(letters))])

    return {
        "digits": digits,
        "letters": letters,
        "d_boxes": d_boxes,
        "l_boxes": l_boxes
    }






'''
images = [
    globals()[f"cropped_Plate{i}"]
    for i in range(1, 40)
    if f"cropped_Plate{i}" in globals()
]

for i, img in enumerate(images, start=1):
    print(f"\n===== Processing Plate {i} =====")
    process_one_plate(
        img,
        plate_name=f"plate_{i}",
        save=True,
        show=True
    )
'''


def load_images_from_folder(folder_path):
    images = []
    names = []

    for fname in sorted(os.listdir(folder_path)):
        if fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
            path = os.path.join(folder_path, fname)
            img = cv2.imread(path)
            if img is not None:
                images.append(img)
                names.append(os.path.splitext(fname)[0])  # name without extension

    return images, names


# =========================
# RUN FROM FOLDER
# =========================

INPUT_FOLDER = r"Localization Test/FinalCrops"   # change if needed

images, names = load_images_from_folder(INPUT_FOLDER)

for img, name in zip(images, names):
    print(f"\n===== Processing {name} =====")
    process_one_plate(
        img,
        plate_name=name,   # used for output folder naming
        save=True,
        show=True
    )





#############################################LOOOOOOOOOOOOP






'''#REMOVE TO FIX






img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plate = extract_plate(img_bgr)

dl_region = crop_digits_letters_region(plate)
dl_region = trim_borders(dl_region, px=4)

# FINAL robust split
digits_region, letters_region = split_digits_letters(dl_region)

d_mask, d_boxes = get_components(digits_region, kind="digits")
l_mask, l_boxes = get_components_letters(letters_region)




# Keep at most 4 digits: choose by area (largest blobs)
if len(d_boxes) > 4:
    d_boxes = sorted(d_boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)[:4]
    d_boxes = sorted(d_boxes, key=lambda b: b[0])

# Keep at most 3 letters: choose by area (largest blobs)
if len(l_boxes) > 3:
    l_boxes = sorted(l_boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)[:3]
    l_boxes = sorted(l_boxes, key=lambda b: b[0])





BASE_OUT_DIR = "output_chars"
#BASE_OUT_DIR = "Image-Processing-Project"
DIGITS_DIR = os.path.join(BASE_OUT_DIR, "digits")
LETTERS_DIR = os.path.join(BASE_OUT_DIR, "letters")

os.makedirs(DIGITS_DIR, exist_ok=True)
os.makedirs(LETTERS_DIR, exist_ok=True)

def clear_old_files(folder, pattern):
    for f in glob.glob(os.path.join(folder, pattern)):
        os.remove(f)

# Clear previous results
clear_old_files(DIGITS_DIR, "digit_*.png")
clear_old_files(LETTERS_DIR, "letter_*.png")





digits  = extract_chars_from_mask(d_mask, d_boxes, reverse=False)
letters = extract_chars_from_mask(l_mask, l_boxes, reverse=True)


TARGET_SIZE = (150, 150)

# Save digits
for i, d in enumerate(digits):
    d_resized = cv2.resize(d, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    save_path = os.path.join(DIGITS_DIR, f"digit_{i}.png")
    cv2.imwrite(save_path, d_resized)

# Save letters
for i, l in enumerate(letters):
    l_resized = cv2.resize(l, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    save_path = os.path.join(LETTERS_DIR, f"letter_{i}.png")
    cv2.imwrite(save_path, l_resized)





d_boxes_img = draw_boxes(d_mask, d_boxes)
l_boxes_img = draw_boxes(l_mask, l_boxes)


show_images(
    [
        img_rgb,
        cv2.cvtColor(plate, cv2.COLOR_BGR2RGB),
        cv2.cvtColor(dl_region, cv2.COLOR_BGR2RGB),
        cv2.cvtColor(digits_region, cv2.COLOR_BGR2RGB),
        cv2.cvtColor(letters_region, cv2.COLOR_BGR2RGB)
    ],
    ["Original", "Plate Crop", "Digits+Letters", "Digits Region", "Letters Region"]
)


show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
show_images([d_boxes_img, l_boxes_img], ["Digits Boxes", "Letters Boxes"])

show_images(digits, [f"Digit {i+1}" for i in range(len(digits))])
show_images(letters, [f"Letter {i+1}" for i in range(len(letters))])





'''#REMOVE TO FIX




'''
MAAW NEW SEG
'''

In [ ]:
'''
MAAW SEGMENTATION
'''

'''



import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import (
    remove_small_holes,
    remove_small_objects,
    binary_closing,
    square,
)
from skimage.measure import label, regionprops
from skimage.transform import resize
from skimage.color import gray2rgb


def crop_bottom(img, top_ratio=0.40):
    h, w = img.shape[:2]
    return img[int(h * top_ratio):, :]


def split_digits_letters(bottom):
    h, w = bottom.shape[:2]
    mid = w // 2
    return bottom[:, :mid], bottom[:, mid:]


def get_mask(region):
    gray = rgb2gray(region)

    T = threshold_otsu(gray)
    mask = gray < T

    mask = remove_small_objects(mask, 25)

    return mask






def touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.005):
    mr = int(margin_ratio * H)
    mc = int(margin_ratio * W)

    if r0 <= mr: return True        # top
    if c0 <= mc: return True        # left
    if r1 >= H - mr: return True    # bottom
    if c1 >= W - mc: return True    # right

    return False



def extract_digits(digits_region, max_chars=4, debug=False):
    H0, W0 = digits_region.shape[:2]
    border_ratio = 0.04
    by = int(border_ratio * H0)
    bx = int(border_ratio * W0)
    sub = digits_region[by:H0 - by, bx:W0 - bx]
    
    mask = get_mask(sub)
    H, W = mask.shape
    
    def remove_frame_components(mask):
        H, W = mask.shape
        lbl = label(mask)

        for p in regionprops(lbl):
            r0, c0, r1, c1 = p.bbox
            h = r1 - r0
            w = c1 - c0

            wr = w / (W + 1e-6)
            hr = h / (H + 1e-6)

            near_edge = (r0 < 0.20 * H) or (r1 > 0.80 * H) #very wide, thin, near top or bottom

            if near_edge and wr > 0.75 and hr < 0.12:
                mask[lbl == p.label] = 0

        return mask

    mask = remove_frame_components(mask)
    
    def touches_top_or_bottom(r0, r1, H, margin=2):
        return r0 <= margin or r1 >= H - margin

    def is_horizontal_frame(h, w, H, W, r0, r1):
        very_wide = w > 0.65 * W
        very_thin = h < 0.12 * H
        near_edge = r0 < 0.15 * H or r1 > 0.85 * H
        return very_wide and very_thin and near_edge



    lbl = label(mask)

    candidates = []
    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        if touches_top_or_bottom(r0, r1, H):
            continue

        h = r1 - r0
        w = c1 - c0
        area = p.area
        wr = w / W
        ar = area / (H * W)

        if is_horizontal_frame(h, w, H, W, r0, r1):
            continue

        if wr < 0.035 or wr > 0.75:
            continue

        if ar < 0.001:
            continue

        candidates.append({"bbox": [r0, c0, r1, c1], "area": area})

    candidates.sort(key=lambda c: c["bbox"][1])


    merged = []
    for c in candidates:
        if not merged:
            merged.append(c)
            continue

        r0, c0, r1, c1 = c["bbox"]
        r0m, c0m, r1m, c1m = merged[-1]["bbox"]

        overlap_x = min(c1, c1m) - max(c0, c0m)
        overlap_y = min(r1, r1m) - max(r0, r0m)

        #only merge if they overlap in X and Y or very close vertically
        vertical_gap = max(r0, r0m) - min(r1, r1m) #negative means it overlaps

        if overlap_x > 0 and (overlap_y > 0 or vertical_gap < 0.08 * H):
            merged[-1]["bbox"] = [
                min(r0, r0m), min(c0, c0m),
                max(r1, r1m), max(c1, c1m)
            ]
        else:
            merged.append(c)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    chars = []
    for m in merged:
        r0, c0, r1, c1 = m["bbox"]
        if (r1 - r0) < 0.22 * H:
            continue

        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        chars.append((c0 + bx, crop))

    chars.sort(key=lambda x: x[0])

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]



def extract_letters(letters_region, max_chars=3, debug=False):
    H0, W0 = letters_region.shape[:2]
    by = int(0.04 * H0)
    bx = int(0.04 * W0)
    sub = letters_region[by:H0 - by, bx:W0 - bx]

    mask = get_mask(sub)
    H, W = mask.shape
    lbl = label(mask)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    big_blobs = []   #letter body
    small_blobs = [] #dots


    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        if touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.02):
            continue
    
        h = r1 - r0
        w = c1 - c0
        area = p.area
        cy = p.centroid[0] / H
        wr = w / W
        ar = area / (H * W)

        #dot (area < 0.015)
        if ar < 0.015:
            small_blobs.append((r0, c0, r1, c1))
            continue

        #main letter filtering
        if wr < 0.08 or wr > 0.75:
            continue
        if ar > 0.22:
            continue
        if cy > 0.75:
            continue

        big_blobs.append([r0, c0, r1, c1])


    for dr0, dc0, dr1, dc1 in small_blobs:
        dc = (dc0 + dc1) / 2
        dr = (dr0 + dr1) / 2

        best_i = -1
        best_dx = 1e9

        for i, box in enumerate(big_blobs):
            r0, c0, r1, c1 = box
            bc = (c0 + c1) / 2
            br = (r0 + r1) / 2

            dx = abs(dc - bc)
            dy = abs(dr - br)

            #vertically and horizontally closest
            if dx < best_dx and dy < 120:
                best_dx = dx
                best_i = i

        #attach dot to nearest valid letter ONLY
        if best_i != -1:
            box = big_blobs[best_i]
            box[0] = min(box[0], dr0)
            box[1] = min(box[1], dc0)
            box[2] = max(box[2], dr1)
            box[3] = max(box[3], dc1)


    big_blobs.sort(key=lambda b: b[1])


    #new
    # ---------- MERGE BROKEN LETTER BODIES (e.g. س) ----------
    merged = []
    for box in sorted(big_blobs, key=lambda b: b[1]):
        if not merged:
            merged.append(box)
            continue

        r0, c0, r1, c1 = box
        pr0, pc0, pr1, pc1 = merged[-1]

        # horizontal gap
        gap = c0 - pc1

        # vertical overlap
        overlap = min(r1, pr1) - max(r0, pr0)

        if gap < 0.15 * W and overlap > 0.3 * min(r1 - r0, pr1 - pr0):
            # merge
            merged[-1] = [
                min(r0, pr0),
                min(c0, pc0),
                max(r1, pr1),
                max(c1, pc1)
            ]
        else:
            merged.append(box)

    big_blobs = merged

    #new





    chars = []
    for r0, c0, r1, c1 in big_blobs:
        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        chars.append((c0 + bx, crop))

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]



def process_plate(img):
    bottom = crop_bottom(img)
    digits_region, letters_region = split_digits_letters(bottom)

    digits, d_mask, d_boxes = extract_digits(digits_region, debug=True)
    letters, l_mask, l_boxes = extract_letters(letters_region, debug=True)

    return digits_region, letters_region, digits, letters, d_mask, d_boxes, l_mask, l_boxes


img22 = imread('Q1.jpg')
#img22 = imread('Dataset/Buses & government vehicles.png')
#img22 = imread('Dataset/Diplomatic vehicles.png')
#img22 = imread('Dataset/image.png')
#img22 = imread('Dataset/Limousines & tourist buses.png')
#img22 = imread('Dataset/Police vehicles.png')
# img22 = rgba2rgb(imread('Dataset/Private vehicles & motorcycles.png'))
#img22 = rgba2rgb(imread('Dataset/Taxis.png'))
#img22 = imread('Dataset/Trucks.png')
#img22 = imread('Dataset/Vehicles with unpaid customs.png')

#etsh cropped tests########################
# img22 = cropped_Plate1
# img22 = cropped_Plate2 #blur problem
#img22 = rgba2rgb(imread('Q1C.png'))
# img22 = cropped_Plate3 #blur problem
# img22 = cropped_Plate5 #FIXED frame counting as letter from left
# img22 = cropped_Plate11 #FIXED digit border on top detected  
# img22 = cropped_Plate12 #FIXED border in digit detected
# img22 = cropped_Plate13 #FIXED border detected as letter
#img22 = cropped_Plate15 #blur problem
#img22 = cropped_Plate111 ##
#img22 = cropped_Plate112 #blur problem
#img22 = cropped_Plate113 #blur problem
#img22 = cropped_Plate115 ##
#img22 = cropped_Plate1111 #hard angle
#img22 = cropped_Plate1112 #hard angle
#img22 = cropped_Plate1113 #blur problem
#img22 = cropped_Plate1115 #blur
#img22 = cropped_Plate11113 #blur
#img22 = cropped_Plate11114 ##
#img22 = cropped_Plate11115 ##
#img22 = cropped_Plate11116 ##
#img22 = cropped_Plate11117 ##

#etsh cropped tests########################




import os

def clear_previous_outputs():
    files = [
        "digit_0.png", "digit_1.png", "digit_2.png", "digit_3.png", "digit_4.png", "letter_0.png", "letter_1.png", "letter_2.png"
    ]

    for f in files:
        if os.path.exists(f):
            os.remove(f)


def save_characters(digits, letters):
    #digits
    for i, img in enumerate(digits):
        out = (img * 255).astype(np.uint8) if img.dtype != np.uint8 else img
        cv2.imwrite(f"digit_{i}.png", out)

    #letters
    for i, img in enumerate(letters):
        out = (img * 255).astype(np.uint8) if img.dtype != np.uint8 else img
        cv2.imwrite(f"letter_{i}.png", out)


clear_previous_outputs()

d_reg, l_reg, digits, letters, d_mask, d_boxes, l_mask, l_boxes = process_plate(img22)






show_images([d_reg, l_reg], ["Digits Region", "Letters Region"])

show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])

show_images([d_boxes, l_boxes], ["Digits Mask + Boxes", "Letters Mask + Boxes"])

digit_titles = [f"Digit {i}" for i in range(len(digits))]
show_images(digits, digit_titles)

letter_titles = [f"Letter {i}" for i in range(len(letters))]
show_images(letters, letter_titles)


save_characters(digits,letters)


#RUN LOOP

for i in range(1, 40):
    img = globals()[f"cropped_Plate{i}"]

    (
        d_reg, l_reg,
        digits, letters,
        d_mask, d_boxes,
        l_mask, l_boxes
    ) = process_plate(img)

    show_images([d_reg, l_reg], ["Digits Region", "Letters Region"])
    show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
    show_images([d_boxes, l_boxes], ["Digits Mask + Boxes", "Letters Mask + Boxes"])

    digit_titles = [f"Digit {j}" for j in range(len(digits))]
    show_images(digits, digit_titles)

    letter_titles = [f"Letter {j}" for j in range(len(letters))]
    show_images(letters, letter_titles)


#RUN LOOP


'''
'''
MAAW SEGMENTATION
'''
        

In [ ]:
'''
MAAW DIGIT AND LETTER RECOGNITION
'''
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os


english_to_arabic = {
    1: '١',
    2: '٢',
    3: '٣',
    4: '٤',
    5: '٥',
    6: '٦',
    7: '٧',
    8: '٨',
    9: '٩'
    }


def normalize_digit(img):
    img = img.copy()

    if np.mean(img) > 127:
        img = cv2.bitwise_not(img)

    ys, xs = np.where(img > 0)
    if len(xs) == 0:
        return cv2.resize(img, (64, 64))

    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    img = img[y0:y1+1, x0:x1+1]

    img = cv2.copyMakeBorder(img, 12, 12, 12, 12, cv2.BORDER_CONSTANT, value=0)

    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_NEAREST)
    return img.astype(np.uint8)


def match_digit(img, refs_folder="digits_reference"):
    img = normalize_digit(img)

    scores = {}

    for d in range(1,10):
        digit_folder = os.path.join(refs_folder, str(d))
        if not os.path.exists(digit_folder):
            continue

        best_local = -1

        for fname in os.listdir(digit_folder):
            ref = cv2.imread(os.path.join(digit_folder, fname), 0)
            ref = normalize_digit(ref)

            score = ssim(img, ref)
            best_local = max(best_local, score)

        scores[d] = best_local

    best_digit = max(scores, key=scores.get)

    return best_digit

final_digits_english = []
final_digits_arabic  = []

for d in digits:
    d_img = d.astype(np.uint8) 
    digit = match_digit(d_img)

    final_digits_english.append(str(digit))
    final_digits_arabic.append(english_to_arabic[digit])

print("Plate Digits In English:", "".join(final_digits_english))
print("Plate Digits In Arabic: ", "".join(final_digits_arabic))



letter_id_to_arabic = {
    1: 'أ',
    2: 'ب',
    3: 'ج',
    4: 'د',
    5: 'ر',
    6: 'ز',
    7: 'س',
    8: 'ص',
    9: 'ض',
    10: 'ط',
    11: 'ع',
    12: 'ف',
    13: 'ق',
    14: 'ك',
    15: 'ل',
    16: 'م',
    17: 'ن',
    18: 'ه',
    19: 'و',
    20: 'ي',
    21: 'ى'
}


def normalize_letter(img):
    img = img.copy()

    if np.mean(img) > 127:
        img = cv2.bitwise_not(img)

    ys, xs = np.where(img > 0)
    if len(xs) == 0:
        return cv2.resize(img, (64, 64))

    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    img = img[y0:y1+1, x0:x1+1]

    img = cv2.copyMakeBorder(
        img, 12, 12, 12, 12,
        cv2.BORDER_CONSTANT, value=0
    )

    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_NEAREST)
    return img.astype(np.uint8)



def match_letter(img, refs_folder="letters_reference"):
    img = normalize_letter(img)

    scores = {}

    for letter_id in range(1, 22):
        letter_folder = os.path.join(refs_folder, str(letter_id))
        if not os.path.exists(letter_folder):
            continue

        best_local = -1

        for fname in os.listdir(letter_folder):
            ref = cv2.imread(os.path.join(letter_folder, fname), 0)
            if ref is None:
                continue

            ref = normalize_letter(ref)
            score = ssim(img, ref)
            best_local = max(best_local, score)

        scores[letter_id] = best_local

    best_letter_id = max(scores, key=scores.get)
    return best_letter_id


final_letters_arabic = []

for l in letters:
    l_img = l.astype(np.uint8)   
    letter_id = match_letter(l_img)


    final_letters_arabic.append(letter_id_to_arabic[letter_id])

final_plate_letters = " ".join(final_letters_arabic[::-1])

print("Plate Letters: ", final_plate_letters)



'''
MAAW DIGIT AND LETTER RECOGNITION
'''

In [ ]:
import pytesseract
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import os
import easyocr
os.environ['TESSDATA_PREFIX'] = os.path.abspath("Dataset")
# os.environ.pop('TESSDATA_PREFIX', None)
def add_padding(img, pad=10):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=255
    )

def add_paddingblack(img, pad=50):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=0
    )


def pad_for_easyocr(img):
    h, w = img.shape
    new_img = np.zeros((h, w*3), dtype=np.uint8)  # black background
    new_img[:, w:w*2] = img  # center the character
    return new_img


def recognize_arabic_digit(image_path, show_preprocessed=False):
    """
    Recognize a single Arabic digit from an image.
    
    Args:
        image_path (str): Path to image file.
        show_preprocessed (bool): If True, shows the binary preprocessed image.
    
    Returns:
        str: Detected Arabic digit or empty string if not recognized.
    """
    
    # 1. Load image
    img = io.imread(image_path)
    # img=add_paddingblack(img)
    # img=pad_for_easyocr(img)
    
    # img=add_padding(img)
    # img=d_mask
    if img.dtype == bool:
        img = (img * 255).astype(np.uint8)

    # # Ensure grayscale, not weird format
    # if len(img.shape) == 2:
    #     img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    # # 2. Convert to grayscale if needed
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    
    # # 3. Resize image up to help OCR
    # gray = cv2.resize(gray, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
    
    # # 4. Smooth image to reduce noise
    # blur = cv2.GaussianBlur(gray, (3,3), 0)
    
    # # 5. Threshold to get binary image
    # _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # # 6. Invert if background is dark
    # if np.mean(thresh) < 127:
    #     thresh = cv2.bitwise_not(thresh)
    
    # # Optional: show preprocessed image
    if show_preprocessed:
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.show()
    
    # 7. Configure Tesseract for single character + Arabic digits only
    # config = f'--oem 3 --psm 10 -c tessedit_char_whitelist={ARABIC_DIGITS}'
    
    # 8. Run OCR
    # text = pytesseract.image_to_string(thresh, lang='ara')

    reader = easyocr.Reader(['ar'])
    results = reader.readtext(img)
    texts = [text for bbox, text, prob in results]


    # 9. Clean result
    # text = text.strip()
    return texts

# Example usage:
digit = recognize_arabic_digit("letter_0.png", show_preprocessed=True)
print("Detected Arabic digit EASYOCR:", repr(digit))
# print("Detected Arabic digit:", repr(digit))


In [ ]:
import easyocr
import cv2
import numpy as np

reader = easyocr.Reader(['ar'], gpu=False)  # make this global or at module level

def prepare_for_easyocr_char(img):
    """
    img: numpy array (grayscale or RGB) containing ONE character.
    Returns a padded, upscaled image suitable for EasyOCR.
    """

    # If bool -> uint8
    if img.dtype == bool:
        img = (img * 255).astype(np.uint8)

    # Convert to grayscale
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img = img.astype(np.uint8)

    # Normalize range
    if img.max() <= 1:
        img = (img * 255).astype(np.uint8)

    # Binarize a bit to stabilize
    _, img = cv2.threshold(img, 0, 255,
                           cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Make sure text is dark on light (helps EasyOCR)
    if img.mean() < 127:      # mostly dark => likely white on black
        img = 255 - img       # invert to black text on white

    # ---- FIX #1: add margin so glyph doesn't touch borders ----
    border = 20
    img = cv2.copyMakeBorder(
        img, border, border, border, border,
        borderType=cv2.BORDER_CONSTANT,
        value=255  # white background
    )

    # ---- FIX #2: upscale ----
    img = cv2.resize(img, None, fx=5, fy=5,
                     interpolation=cv2.INTER_LINEAR)

    return img


def recognize_single_char_easyocr(image_path, show=False):
    # load from file
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    prep = prepare_for_easyocr_char(img)

    if show:
        import matplotlib.pyplot as plt
        plt.imshow(prep, cmap='gray')
        plt.axis('off'); plt.show()

    # detail=0 -> just texts; paragraph=False so it doesn't merge things
    texts = reader.readtext(prep, detail=0, paragraph=False)

    return texts[0] if texts else ""

digit0 = recognize_single_char_easyocr("clean_digits.png", show=True)
letter0 = recognize_single_char_easyocr("clean_letters.png", show=True)
print("digit_0:", repr(digit0))
print("letter_0:", repr(letter0))


In [ ]:
'''
MAAW FORTNITE BATTLE ROYALE
'''

import numpy as np
import matplotlib.pyplot as plt
import cv2                         ### <<< you were using cv2 in save_characters

from skimage.io import imread
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import (
    remove_small_holes,
    remove_small_objects,
    binary_closing,
    square,
)
from skimage.measure import label, regionprops
from skimage.transform import resize
from skimage.color import gray2rgb


# ---------------------------------------------------
# 1) SIMPLE GEOMETRIC UTILITIES
# ---------------------------------------------------

def crop_bottom(img, top_ratio=0.40):
    """
    Keep the lower part of the plate (digits + letters).
    top_ratio is the fraction of height we cut from the top.
    """
    h, w = img.shape[:2]
    return img[int(h * top_ratio):, :]


def split_digits_letters(bottom):
    """
    Egyptian plate: digits on the left half, letters on the right half.
    """
    h, w = bottom.shape[:2]
    mid = w // 2
    return bottom[:, :mid], bottom[:, mid:]


def get_mask(region):
    gray = rgb2gray(region)
    T = threshold_otsu(gray)
    mask = gray < T

    # ✅ KEEP holes & dots (light cleaning only)
    mask = remove_small_objects(mask, 25)
    return mask


# ---------------------------------------------------
# Helper: create a crop that is friendly for OCR
# ---------------------------------------------------
def make_ocr_char_crop(gray_sub, r0, c0, r1, c1,
                       pad=3, scale=3):
    """
    gray_sub : grayscale version of sub-image (0–1)
    (r0,c0,r1,c1) : bbox in sub coordinates
    pad : extra pixels around character
    scale : upscaling factor (so text is not tiny)
    """
    H, W = gray_sub.shape

    rr0 = max(r0 - pad, 0)
    cc0 = max(c0 - pad, 0)
    rr1 = min(r1 + pad, H)
    cc1 = min(c1 + pad, W)

    crop = gray_sub[rr0:rr1, cc0:cc1]        # 0–1 float from rgb2gray

    # upscale (don’t DOWNsize to 40×40)
    h, w = crop.shape
    crop = resize(crop, (int(h * scale), int(w * scale)),
                  anti_aliasing=True)

    return crop   # still float 0–1


# ---------------------------------------------------
# 2) DIGITS EXTRACTION
# ---------------------------------------------------

def touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.005):
    mr = int(margin_ratio * H)
    mc = int(margin_ratio * W)

    if r0 <= mr: return True        # top
    if c0 <= mc: return True        # left
    if r1 >= H - mr: return True    # bottom
    if c1 >= W - mc: return True    # right

    return False


def extract_digits(digits_region, max_chars=4, debug=False):
    H0, W0 = digits_region.shape[:2]
    border_ratio = 0.04
    by = int(border_ratio * H0)
    bx = int(border_ratio * W0)
    sub = digits_region[by:H0 - by, bx:W0 - bx]

    # segmentation is done on mask
    mask = get_mask(sub)
    gray_sub = rgb2gray(sub)          ### <<< NEW: grayscale for OCR crops
    H, W = mask.shape

    lbl = label(mask)

    candidates = []
    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        touches = touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.01)
        if touches:
            h = r1 - r0
            w = c1 - c0
            ar = p.area / (H * W)
            # ✅ only reject if it looks like a FRAME, not a digit
            if h > 0.9 * H or w < 0.02 * W or ar < 0.002:
                continue

        h = r1 - r0
        w = c1 - c0
        area = p.area
        wr = w / W
        ar = area / (H * W)

        if wr < 0.035 or wr > 0.75:
            continue
        if ar < 0.001:
            continue

        candidates.append({"bbox": [r0, c0, r1, c1], "area": area})

    # sort left → right
    candidates.sort(key=lambda c: c["bbox"][1])

    # merge overlapping boxes
    merged = []
    for c in candidates:
        if not merged:
            merged.append(c)
            continue

        r0, c0, r1, c1 = c["bbox"]
        r0m, c0m, r1m, c1m = merged[-1]["bbox"]
        overlap_x = min(c1, c1m) - max(c0, c0m)

        if overlap_x > 0:
            merged[-1]["bbox"] = [
                min(r0, r0m), min(c0, c0m),
                max(r1, r1m), max(c1, c1m)
            ]
        else:
            merged.append(c)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    chars = []
    for m in merged:
        r0, c0, r1, c1 = m["bbox"]
        if (r1 - r0) < 0.22 * H:
            continue

        # draw box for debug
        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        # <<< OLD:
        # crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        # chars.append((c0 + bx, crop))

        # >>> NEW: OCR-friendly grayscale crop with padding + upscaling
        crop = make_ocr_char_crop(gray_sub, r0, c0, r1, c1,
                                  pad=3, scale=3)
        chars.append((c0 + bx, crop))

    chars.sort(key=lambda x: x[0])

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]


# ---------------------------------------------------
# 3) LETTERS EXTRACTION
# ---------------------------------------------------

def extract_letters(letters_region, max_chars=3, debug=False):
    H0, W0 = letters_region.shape[:2]
    by = int(0.04 * H0)
    bx = int(0.04 * W0)
    sub = letters_region[by:H0 - by, bx:W0 - bx]

    mask = get_mask(sub)
    gray_sub = rgb2gray(sub)            ### <<< NEW: grayscale for OCR crops
    H, W = mask.shape
    lbl = label(mask)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    big_blobs = []   # main letter bodies
    small_blobs = [] # dots

    # -----------------------------
    # 1) SPLIT BIG BLOBS & DOTS
    # -----------------------------
    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        if touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.02):
            continue

        h = r1 - r0
        w = c1 - c0
        area = p.area
        cy = p.centroid[0] / H
        wr = w / W
        ar = area / (H * W)

        # 👉 DOT = any small area < 0.015
        if ar < 0.015:
            small_blobs.append((r0, c0, r1, c1))
            continue

        # 👉 MAIN LETTER FILTER (no lower area limit now)
        if wr < 0.08 or wr > 0.75:
            continue
        if ar > 0.22:
            continue
        if cy > 0.75:
            continue

        big_blobs.append([r0, c0, r1, c1])

    # -----------------------------------------
    # 2) ATTACH EACH DOT TO NEAREST LETTER BOX
    # -----------------------------------------
    for dr0, dc0, dr1, dc1 in small_blobs:
        dc = (dc0 + dc1) / 2
        dr = (dr0 + dr1) / 2

        best_i = -1
        best_dx = 1e9

        for i, box in enumerate(big_blobs):
            r0, c0, r1, c1 = box
            bc = (c0 + c1) / 2
            br = (r0 + r1) / 2

            dx = abs(dc - bc)
            dy = abs(dr - br)

            # ✅ must be vertically close AND horizontally closest
            if dx < best_dx and dy < 120:
                best_dx = dx
                best_i = i

        # ✅ attach dot ONLY to the nearest valid letter
        if best_i != -1:
            box = big_blobs[best_i]
            box[0] = min(box[0], dr0)
            box[1] = min(box[1], dc0)
            box[2] = max(box[2], dr1)
            box[3] = max(box[3], dc1)

    # -----------------------------------------
    # 3) SORT & CROP FINAL LETTERS
    # -----------------------------------------
    big_blobs.sort(key=lambda b: b[1])

    chars = []
    for r0, c0, r1, c1 in big_blobs:
        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        # <<< OLD:
        # crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        # chars.append((c0 + bx, crop))

        # >>> NEW: grayscale + padding + upscaling
        crop = make_ocr_char_crop(gray_sub, r0, c0, r1, c1,
                                  pad=3, scale=3)
        chars.append((c0 + bx, crop))

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]


# ---------------------------------------------------
# 4) FULL PIPELINE FOR ONE IMAGE
# ---------------------------------------------------
def process_plate(img):
    bottom = crop_bottom(img)
    digits_region, letters_region = split_digits_letters(bottom)

    digits, d_mask, d_boxes = extract_digits(digits_region, debug=True)
    letters, l_mask, l_boxes = extract_letters(letters_region, debug=True)

    return digits_region, letters_region, digits, letters, d_mask, d_boxes, l_mask, l_boxes


# Choose your plate image
img22 = imread('Dataset/Limousines & tourist buses.png')

def save_characters(digits, letters):
    # digits / letters are floats 0–1 → uint8 0–255
    for i, img in enumerate(digits):
        out = (img * 255).astype(np.uint8)
        cv2.imwrite(f"digit_{i}.png", out)

    for i, img in enumerate(letters):
        out = (img * 255).astype(np.uint8)
        cv2.imwrite(f"letter_{i}.png", out)


d_reg, l_reg, digits, letters, d_mask, d_boxes, l_mask, l_boxes = process_plate(img22)

# (show_images assumed defined elsewhere)
show_images([d_reg, l_reg], ["Digits Region", "Letters Region"])
show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
show_images([d_boxes, l_boxes], ["Digits Mask + Boxes", "Letters Mask + Boxes"])

digit_titles = [f"Digit {i}" for i in range(len(digits))]
show_images(digits, digit_titles)

letter_titles = [f"Letter {i}" for i in range(len(letters))]
show_images(letters, letter_titles)

save_characters(digits, letters)
